In [1]:
import numpy as np
import logging
import pyLDAvis.gensim
import json
import warnings

In [2]:
# add log
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarit

In [3]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array

In [4]:
# set up logging
# logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)
# logging.debug("test")

In [5]:
#set up corpus
texts = [['o', 'ov', 'ooh', 'vh', 'oab', 'aoh', 'vaao', 'o', 'hjb', 'ohhv', 'aah', 'v', 'o', 'hvvohh', 'a', 'ao', 'c', 'ovh', 'v', 'bhvhvo', 'vjx', 'aav', 'avh', 'hac', 'a', 'v', 'oxjoo', 'hooa', 'ha', 'hh', 'a', 'aow', 'aah', 'hq', 'voj', 'o', 'hoh', 'hh', 'vv', 'vpov', 'h', 'hoqao', 'b', 'vhoxao', 'hov', 'ah', 'axvv', 'ovva', 'hh', 'vhoooh', 'o', 'hhv', 'oao', 'oha', 'v', 'v', 'vo', 'h', 'ho', 'h', 'h', 'h', 'ohh', 'v', 'ah', 'vh', 'wh', 'avc', 'vvah', 'h', 'a', 'a', 'aov', 'j', 'qha', 'aoa', 'hhoaha', 'o', 'hh', 'v', 'vao', 'ooha', ],
['vh', 'ao', 'iv', 'ih', 'hao', 'oa', 'a', 'oaoo', 'ow', 'hho', 'aha', 'h', 'ohav', 'v', 'hovvh', 'vvaho', 'a', 'h', 'wj', 'hoho', 'vhhh', 'h', 'aa', 'x', 'ahh', 'vho', 'aah', 'aav', 'ooo', 'aaoq', 'ooa', 'vw', 'oa', 'hh', 'a', 'a', 'ah', 'a', 'h', 'av', 'o', 'hvo', 'hao', 'aqi', 'vo', 'oh', 'vho', 'hov', 'hoav', 'voh', 'hhh', 'ioov', 'hpo', 'a', 'hh', 'hh', 'a', 'v', 'ha', 'vaa', 'hqwhv', 'vhv', 'vvoh', 'avahv', 'havh', 'hvo', 'hh', 'oaavv', 'avvvh', 'vv', 'hvovhx', 'bh', 'hawo', 'ah', 'avh', ],
['av', 'w', 'ohoi', 'aa', 'vaovh', 'o', 'ho', 'vvv', 'ohawv', 'hh', 'x', 'whhv', 'vavhqo', 'ovv', 'oh', 'vhvaa', 'vvvv', 'aho', 'oo', 'av', 'avv', 'oh', 'aah', 'haov', 'hv', 'a', 'ahav', 'hh', 'qoa', 'h', 'wav', 'oo', 'ho', 'o', 'ho', 'aah', 'vh', 'vv', 'hvhaa', 'o', 'v', 'oo', 'pah', 'vhho', 'vh', 'vvi', 'h', 'vox', 'ahv', 'hvah', 'aoa', 'ahhv', 'v', 'qho', 'hoo', 'o', 'vhov', 'aaow', 'v', 'hova', 'oav', 'ah', 'hvvo', 'hvvavv', 'aov', 'hb', 'o', 'hhi', 'ahvv', 'ovav', 'hoh', 'hvv', 'vo', 'ah', 'vhooaah', ],
['ho', 'oviv', 'h', 'ohaa', 'oo', 'o', 'v', 'hoh', 'h', 'hhqo', 'hov', 'voo', 'johi', 'w', 'hh', 'hvo', 'hph', 'ajhaa', 'vao', 'ooj', 'aaha', 'ho', 'hooa', 'aaaov', 'vah', 'ha', 'a', 'ooo', 'xoaa', 'a', 'ohva', 'ov', 'hv', 'ah', 'o', 'v', 'o', 'v', 'voi', 'a', 'o', 'hv', 'avoa', 'hohv', 'o', 'ohv', 'vah', 'jo', 'aah', 'vhaap', 'ovvh', 'oo', 'aahhoo', 'h', 'v', 'hvoh', 'ovpoa', 'ha', 'hoa', 'vhoq', 'viav', 'hv', 'o', 'av', 'ha', 'hvo', 'hha', 'oa', 'oahh', 'ho', 'cov', 'oaov', 'vaaw', 'vivh', 'ava', 'ovoohoa', 'qhh', 'av', 'a', 'va', ],
['h', 'h', 'hha', 'vvvvh', 'ovoa', 'pa', 'jhoo', 'hhv', 'o', 'oavo', 'hvo', 'a', 'vjaa', 'ao', 'v', 'vhohav', 'aooaho', 'ov', 'hh', 'aaoo', 'aa', 'vvao', 'a', 'ooowh', 'avhha', 'iia', 'ah', 'vbcha', 'vvoov', 'vvoav', 'a', 'ov', 'ovh', 'ha', 'ao', 'avvv', 'oh', 'a', 'vooh', 'v', 'vhhx', 'ava', 'hao', 'xv', 'pa', 'vh', 'ia', 'a', 'havo', 'oh', 'hhov', 'avoa', 'hha', 'cvhqa', 'hho', 'wva', 'iqvwo', 'hhhh', 'v', 'o', 'vvhv', 'oao', 'oq', 'hbo', 'vah', 'j', 'a', 'oahh', 'avhiv', 'oaao', 'aaco', 'oahh', 'aoh', 'woov', 'xhh', 'hv', 'ajavvah', ],
['av', 'ovo', 'oha', 'hahva', 'o', 'v', 'oi', 'hhh', 'ao', 'vhh', 'pq', 'ahov', 'vv', 'vh', 'h', 'avjv', 'o', 'ovvaa', 'a', 'vwh', 'oho', 'ahx', 'vvoav', 'oqv', 'v', 'ohao', 'avo', 'vbjhv', 'hvhoxv', 'hh', 'hov', 'v', 'ao', 'a', 'cv', 'bqaa', 'hvh', 'aoh', 'o', 'o', 'oaooq', 'xao', 'aahh', 'vo', 'hh', 'ao', 'who', 'hva', 'o', 'ho', 'vh', 'a', 'av', 'vva', 'va', 'chavq', 'w', 'hao', 'v', 'avho', 'vvaoah', 'oa', 'ha', 'x', 'ohv', 'aaohh', 'vov', 'avvaa', 'vv', 'a', 'hohvv', 'hv', 'ao', 'hoox', 'p', ],
['o', 'oav', 'vvc', 'ao', 'o', 'vvcv', 'oojhh', 'vvha', 'wa', 'avh', 'oo', 'havb', 'ahv', 'vav', 'vhv', 'hhoa', 'avvv', 'vhho', 'vh', 'hohv', 'oahhh', 'ooc', 'v', 'ho', 'hoooh', 'oohah', 'hjvav', 'o', 'hvv', 'oo', 'avv', 'hvao', 'i', 'vhqiao', 'ahvvo', 'v', 'ahhav', 'a', 'haa', 'ho', 'hbv', 'oovp', 'ohov', 'vva', 'o', 'aa', 'ho', 'ah', 'v', 'vv', 'vh', 'aoo', 'ha', 'v', 'av', 'hva', 'hhaoa', 'vvvv', 'ovv', 'h', 'ahoqvv', 'v', 'ooo', 'vo', 'hhh', 'oavov', 'vvv', 'voih', 'pobaovvo', 'h', 'vh', 'va', 'hhavh', 'oov', ],
['ch', 'bhoao', 'hhh', 'hvv', 'oo', 'a', 'oppiv', 'ova', 'ooa', 'aoho', 'vo', 'vp', 'vv', 'ahoa', 'vav', 'ovaha', 'a', 'ohavcv', 'h', 'vo', 'vvh', 'xh', 'aaio', 'ohovvv', 'ahhha', 'ahaavv', 'aqv', 'a', 'vwoao', 'ahh', 'h', 'h', 'a', 'h', 'vqh', 'ao', 'xv', 'h', 'vo', 'wo', 'ao', 'a', 'oaa', 'aah', 'oa', 'v', 'h', 'ahv', 'hhvo', 'av', 'abo', 'ca', 'avvhvh', 'aaq', 'hv', 'va', 'vh', 'vawhx', 'o', 'o', 'oao', 'ooc', 'oa', 'ov', 'haa', 'vvo', 'oa', 'hvvoj', 'ooaho', 'avvavv', 'ovba', 'hoxho', 'ah', 'avo', 'a', 'vh', 'ov', 'aahhovv', 'avo', 'hh', 'jhv', 'o', 'vvoaa', 'co', 'aoho', 'ova', 'h', ],
['h', 'a', 'ha', 'ox', 'oh', 'avh', 'vva', 'hvhvvoj', 'aao', 'v', 'hv', 'vva', 'bvh', 'ba', 'vav', 'ho', 'o', 'v', 'hh', 'a', 'va', 'hhh', 'hphh', 'qa', 'ooho', 'aoh', 'wh', 'aav', 'o', 'o', 'vhvaa', 'vpv', 'av', 'oo', 'vho', 'vavv', 'ohv', 'haa', 'v', 'apavv', 'v', 'oaah', 'vaowh', 'vooavvv', 'vh', 'vhv', 'o', 'vhhaa', 'oa', 'hvhho', 'oohh', 'vov', 'vxaa', 'paw', 'vh', 'oaooh', 'vhvh', 'oa', 'av', 'vvac', 'vhxav', 'vv', 'a', 'b', 'ov', 'aa', 'qhh', 'oao', 'ahaa', 'oho', 'o', 'va', ],
['xaa', 'ovovja', 'oax', 'ovoov', 'v', 'vv', 'vv', 'ovoo', 'hvvov', 'o', 'hvhv', 'hah', 'ooh', 'vavhah', 'havao', 'o', 'ovoho', 'aao', 'cav', 'ohihva', 'ihvahow', 'havv', 'oavo', 'hhh', 'vhoa', 'ah', 'ahoooh', 'aavh', 'aah', 'hhv', 'ao', 'ah', 'ao', 'av', 'aiphh', 'aovoo', 'vha', 'hvaahv', 'ha', 'haa', 'v', 'hv', 'woqv', 'o', 'avh', 'o', 'v', 'vavhh', 'ov', 'o', 'v', 'oaq', 'a', 'apaoh', 'oqa', 'hv', 'avvh', 'vho', 'vh', 'ohvvo', 'vva', 'o', 'oboaa', 'ohvv', 'hh', 'abhh', 'vhob', 'ha', 'ahao', 'ab', 'avovh', 'hwv', 'o', 'vviavvb', ],
['aho', 'vh', 'hoho', 'bo', 'ov', 'a', 'v', 'a', 'ahh', 'av', 'ahaoch', 'vavh', 'a', 'v', 'hhavhao', 'oh', 'hovo', 'ohv', 'v', 'javoa', 'vhapo', 'aah', 'ah', 'ojai', 'aajv', 'aq', 'hhvva', 'aav', 'vv', 'a', 'hjvbohao', 'hv', 'woo', 'vo', 'vo', 'o', 'aahv', 'o', 'cqoh', 'hhh', 'av', 'pp', 'hohb', 'ava', 'bv', 'oapa', 'avav', 'o', 'aoo', 'a', 'a', 'ai', 'ho', 'ahh', 'hvv', 'avahaa', 'phaa', 'oha', 'vh', 'oha', 'vvaoov', 'ha', 'haoo', 'aoaho', 'avva', 'h', 'ovv', 'hov', 'voh', 'oh', 'vw', 'aqo', ],
['ah', 'aavh', 'ooova', 'ho', 'aacv', 'vaoo', 'hoo', 'ovvaoao', 'vp', 'ovoaa', 'vhaov', 'oho', 'o', 'hvhqo', 'vw', 'ov', 'vvo', 'ovh', 'aov', 'oh', 'vo', 'hh', 'ohvh', 'h', 'oa', 'vvhav', 'hha', 'hoah', 'hvv', 'wov', 'aoa', 'jhvh', 'aav', 'hw', 'hovo', 'ahhb', 'hoh', 'voha', 'hbvai', 'cvbh', 'oavoha', 'v', 'o', 'wo', 'ohjv', 'phhh', 'oho', 'aa', 'hoo', 'whaa', 'vv', 'vpvv', 'o', 'av', 'ah', 'hj', 'oh', 'ja', 'hj', 'h', 'o', 'vhjhow', 'h', 'avo', 'v', 'hoa', 'v', 'vh', 'ov', 'vv', 'vooo', 'vp', 'ao', 'hvo', 'vho', 'oa', ],
['a', 'o', 'h', 'vq', 'boh', 'v', 'woa', 'oo', 'ovh', 'xaahao', 'h', 'hvho', 'voiov', 'hhha', 'qa', 'ha', 'oh', 'abaav', 'vah', 'v', 'haha', 'a', 'av', 'ao', 'hohh', 'oa', 'ovh', 'q', 'h', 'ha', 'aahhaa', 'ahoo', 'v', 'a', 'hjv', 'vvv', 'v', 'ooh', 'oh', 'oh', 'a', 'ph', 'oabh', 'aa', 'ao', 'v', 'hh', 'a', 'vhoo', 'vh', 'vva', 'oxha', 'oaaovah', 'hao', 'oaa', 'hv', 'vv', 'bh', 'xa', 'o', 'ao', 'vavahav', 'ohh', 'v', 'hahov', 'bcah', 'a', 'hva', 'hh', 'oh', 'hh', 'oaha', 'h', 'owao', ],
['phvho', 'hoh', 'aj', 'aoa', 'ohov', 'aov', 'vvoaa', 'avwav', 'hhwa', 'h', 'ohova', 'avhv', 'o', 'aa', 'ova', 'v', 'hohh', 'av', 'hv', 'aovv', 'vao', 'h', 'ah', 'aavo', 'v', 'ooho', 'vhvahv', 'o', 'hov', 'qo', 'oo', 'haha', 'ohv', 'vow', 'h', 'vvvo', 'aohh', 'xh', 'iho', 'vhvaahah', 'aa', 'a', 'a', 'aoaaav', 'a', 'av', 'a', 'oa', 'hh', 'hvvoo', 'av', 'vohh', 'ai', 'hva', 'ohv', 'oc', 'o', 'aovv', 'v', 'vab', 'qovoa', 'v', 'ho', 'ph', 'av', 'vo', 'hbo', 'ah', 'voh', 'oc', 'hv', 'ova', 'ov', 'ahvov', 'vihh', 'a', ],
['vvo', 'ov', 'va', 'hv', 'hhh', 'hah', 'vav', 'ahvch', 'j', 'aavah', 'v', 'hv', 'ahv', 'hoovo', 'oboh', 'whao', 'haao', 'hvqao', 'ovhv', 'ahoohw', 'ja', 'a', 'oqvvo', 'vov', 'oavoaoa', 'hab', 'oojaj', 'avhha', 'oo', 'aoa', 'hq', 'av', 'oovh', 'o', 'voo', 'o', 'a', 'hvpa', 'voh', 'h', 'bvhhhvp', 'hwh', 'ooahv', 'haii', 'b', 'hhv', 'o', 'va', 'vo', 'ph', 'a', 'a', 'oh', 'vh', 'h', 'o', 'vhaa', 'iooo', 'hhhv', 'ovvh', 'vvvph', 'hv', 'hphao', 'a', 'avhoo', 'v', 'hoj', 'vv', 'vvov', 'ovho', 'oojooa', 'ooav', 'ohvhqo', 'ah', 'vo', 'avaav', 'aiba', 'hvoha', 'aahvh', 'vah', 'oh', 'aova', 'hhh', 'a', 'v', ],
['oavx', 'abooa', 'v', 'hiaava', 'hahvo', 'ph', 'ovv', 'a', 'haooh', 'aa', 'h', 'hoh', 'ac', 'a', 'haoh', 'oahvj', 'haa', 'o', 'vw', 'hvhhoo', 'booh', 'ov', 'a', 'hah', 'hh', 'w', 'oq', 'ivhh', 'ao', 'aaho', 'vah', 'hoo', 'vhv', 'oao', 'av', 'vh', 'qopv', 'oaa', 'hvav', 'vaq', 'ha', 'aa', 'v', 'o', 'a', 'ohv', 'obv', 'aovoqv', 'va', 'aa', 'io', 'hhvo', 'vaho', 'ho', 'oav', 'choaah', 'pvv', 'v', 'vh', 'oh', 'aia', 'oooao', 'hvvoo', 'hvh', 'ao', 'a', 'o', 'vaqva', 'v', 'baoj', 'oh', 'aahp', 'ao', 'wh', 'a', 'oo', 'h', 'hva', 'oavc', 'xvv', 'hoo', 'a', 'oah', 'oo', 'aah', ],
['ah', 'ooavp', 'hv', 'o', 'ovo', 'vjo', 'o', 'bh', 'ah', 'a', 'a', 'ohh', 'aa', 'aovhv', 'aao', 'va', 'xqa', 'a', 'oohh', 'vqa', 'hoov', 'voavhh', 'hhoa', 'a', 'o', 'vo', 'oa', 'vo', 'ah', 'v', 'vo', 'hvv', 'aha', 'hvpo', 'o', 'oh', 'h', 'vhxa', 'o', 'o', 'xh', 'a', 'hvv', 'hvhavh', 'ao', 'avo', 'voh', 'vhoa', 'va', 'oav', 'v', 'aoao', 'vv', 'hq', 'hva', 'aq', 'aha', 'v', 'h', 'o', 'vv', 'v', 'vvvo', 'oaoh', 'av', 'v', 'hvava', 'aava', 'v', 'o', 'a', 'oowo', 'hj', 'o', 'av', 'avvoq', 'a', 'ooh', 'vaxh', ],
['vv', 'vo', 'ov', 'h', 'o', 'o', 'v', 'jhh', 'ha', 'vaaa', 'qo', 'a', 'hvoj', 'aoh', 'oo', 'xvoox', 'oha', 'vaa', 'vv', 'qov', 'oa', 'hoha', 'pvo', 'a', 'vvwov', 'vooa', 'o', 'hv', 'v', 'aaivo', 'ahoa', 'ah', 'voh', 'a', 'boh', 'ov', 'a', 'h', 'a', 'a', 'o', 'hvo', 'hha', 'ohah', 'v', 'oo', 'avvhj', 'paoavva', 'vao', 'hv', 'aox', 'hah', 'ova', 'baohaoh', 'hh', 'o', 'vvvvovi', 'avv', 'o', 'a', 'hic', 'hh', 'x', 'coah', 'vhv', 'whohv', 'po', 'hvavvoi', 'ovoh', 'oav', 'ao', 'ooohohv', 'oqabo', 'ovv', 'hvoba', 'wvaoha', 'aaa', 'haha', 'va', 'oah', 'ooha', 'aa', 'qh', 'jo', ],
['ov', 'hv', 'o', 'aho', 'o', 'aoh', 'v', 'haa', 'o', 'voa', 'voo', 'oo', 'vao', 'va', 'aaaa', 'aaav', 'aho', 'oh', 'qoah', 'ao', 'xv', 'ahxv', 'va', 'oaov', 'ao', 'aa', 'vvao', 'xavavv', 'vhh', 'o', 'wa', 'hq', 'o', 'h', 'v', 'xhh', 'haa', 'ahh', 'hvvh', 'oph', 'hhh', 'oooa', 'ajhj', 'vo', 'v', 'ovq', 'ha', 'oiaao', 'a', 'ovox', 'o', 'oo', 'poh', 'oao', 'hvov', 'av', 'v', 'qa', 'o', 'ohciox', 'ooa', 'avov', 'o', 'ooh', 'ho', 'oha', 'jo', 'xo', 'o', 'oa', 'v', 'h', 'vo', 'hbob', 'hvjhhv', 'vv', 'oh', 'vhva', 'a', 'ihhi', 'hho', ],
['vh', 'vhoc', 'ooo', 'h', 'v', 'aqhao', 'hahj', 'va', 'ao', 'hvah', 'hhoaa', 'v', 'v', 'aq', 'ooaa', 'avo', 'ha', 'ov', 'v', 'vvva', 'hahv', 'wvvv', 'ov', 'oho', 'ao', 'hbhapa', 'chav', 'o', 'ohv', 'aa', 'hva', 'aaa', 'vh', 'oo', 'pa', 'ovaa', 'avow', 'vvoh', 'ao', 'vhvaa', 'phchv', 'viabho', 'oaovo', 'ohhv', 'ahh', 'ohho', 'va', 'aovh', 'oav', 'vhhh', 'hawhh', 'hh', 'o', 'hhxhv', 'aovov', 'ahvo', 'hv', 'h', 'o', 'vjvbvo', 'o', 'ovv', 'a', 'v', 'ha', 'hov', 'ba', 'vah', 'vov', 'ohi', 'avvo', 'oh', 'ovb', 'o', 'o', 'v', 'va', 'oaoaa', 'avowa', 'hv', 'ovhaj', 'h', 'v', 'oovvvox', ],
['oaoh', 'o', 'h', 'o', 'j', 'vhv', 'oh', 'vvhh', 'v', 'aapap', 'voho', 'o', 'aoavh', 'ooi', 'o', 'aaao', 'vab', 'c', 'ohoh', 'caahv', 'v', 'oq', 'vah', 'ao', 'oho', 'v', 'vhao', 'vohva', 'ao', 'hvahv', 'vh', 'h', 'vhohh', 'oav', 'v', 'oaaa', 'hvhoow', 'voa', 'hhx', 'qao', 'w', 'o', 'avvaahav', 'a', 'haa', 'ahah', 'vh', 'a', 'ha', 'ohhv', 'ovhoa', 'vhah', 'oqvv', 'aha', 'vi', 'h', 'ahohvoh', 'bo', 'ovhha', 'oh', 'ha', 'aho', 'boao', 'x', 'oow', 'a', 'hi', 'qv', 'j', 'hba', 'h', 'oa', 'h', 'v', 'vxah', 'haav', 'hohq', 'jaoa', 'vhaa', 'ovha', 'ahaaav', ],
['ahaa', 'oo', 'vh', 'oja', 'hvvoao', 'hah', 'hhohvovo', 'apv', 'hahx', 'aaq', 'hv', 'vahw', 'qv', 'qa', 'o', 'haov', 'avao', 'cav', 'v', 'qooo', 'ah', 'apa', 'ovvq', 'aao', 'ov', 'jovava', 'h', 'hh', 'avhh', 'haax', 'hov', 'a', 'aov', 'op', 'o', 'q', 'aaavhv', 'hq', 'hh', 'ah', 'opqvoo', 'avvh', 'vha', 'iv', 'av', 'h', 'a', 'j', 'vo', 'vbiav', 'voao', 'axh', 'a', 'ovha', 'aqa', 'iaaoao', 'h', 'vh', 'hoh', 'o', 'vo', 'i', 'hh', 'i', 'vav', 'av', 'aaoq', 'ovoo', 'v', 'ho', 'v', 'a', 'oh', 'haa', 'v', 'hhh', 'hv', ],
['aooo', 'vohh', 'h', 'voa', 'oh', 'vqo', 'ohhao', 'hava', 'ohh', 'aoo', 'ao', 'o', 'hoo', 'ohao', 'haa', 'o', 'vvoha', 'jhha', 'vh', 'aoh', 'hv', 'ahc', 'ha', 'v', 'qooh', 'ho', 'ho', 'vva', 'cho', 'vaahh', 'o', 'ahh', 'hhh', 'ovhaha', 'haqq', 'boa', 'avvo', 'oh', 'ophh', 'aa', 'v', 'wpo', 'hi', 'hvahc', 'h', 'ooo', 'vx', 'avoh', 'aav', 'vhvo', 'aaaoo', 'ao', 'v', 'a', 'aoh', 'h', 'avaoh', 'hv', 'av', 'voaoaa', 'hc', 'hv', 'va', 'oo', 'hvv', 'p', 'voaa', 'aaoo', 'o', 'hv', 'hov', 'oa', 'ooih', 'hov', 'wv', 'oa', 'hahvha', 'avh', 'hhx', 'hhh', 'hho', 'ovav', 'a', ],
['o', 'v', 'v', 'a', 'oohova', 'ho', 'vph', 'oo', 'v', 'vh', 'hvh', 'oi', 'h', 'v', 'o', 'a', 'i', 'ajh', 'va', 'oava', 'o', 'vv', 'ovpx', 'v', 'h', 'h', 'aooaa', 'hvvhhow', 'ho', 'vhj', 'vaoah', 'ovaaohooa', 'j', 'haop', 'o', 'vova', 'ahvoaq', 'a', 'hvvo', 'haav', 'h', 'hhav', 'h', 'hh', 'h', 'voa', 'o', 'haoa', 'o', 'ahh', 'oo', 'oi', 'p', 'a', 'vv', 'hv', 'vov', 'qohoaw', 'hv', 'poh', 'v', 'ah', 'acav', 'aooa', 'ahioh', 'oa', 'v', 'avhvw', 'vv', 'v', 'avh', 'ov', 'oo', 'ahvv', 'wvo', 'hva', ],
['hao', 'vaoa', 'vvh', 'hhhvi', 'oaaoa', 'ovh', 'a', 'vvvv', 'xovva', 'o', 'o', 'o', 'jao', 'oah', 'owv', 'hva', 'ahhvv', 'oaa', 'vhq', 'v', 'oav', 'vbj', 'o', 'qaa', 'vo', 'aqo', 'vhh', 'av', 'a', 'ov', 'o', 'vai', 'vh', 'oh', 'hvoah', 'aovo', 'aho', 'h', 'vv', 'opv', 'vaaa', 'hhhvv', 'hvva', 'vohh', 'aa', 'h', 'opa', 'o', 'hoovov', 'h', 'aa', 'aoc', 'ohvah', 'ovhx', 'aao', 'h', 'hahao', 'oaohv', 'pa', 'a', 'o', 'o', 'a', 'ah', 'hh', 'vx', 'vva', 'hovivao', 'ow', 'oho', 'h', 'j', 'aah', 'ho', 'a', 'hhvov', 'hahba', 'hoop', 'ho', 'jvv', 'wv', ],
['c', 'av', 'avo', 'hoo', 'hx', 'av', 'o', 'hoopav', 'vooa', 'oqo', 'a', 'hha', 'ha', 'oa', 'ov', 'ohaoao', 'ajoov', 'ah', 'ha', 'hqo', 'ooa', 'vv', 'pah', 'aovavh', 'vo', 'vao', 'avavoq', 'j', 'h', 'hhbo', 'hho', 'oh', 'x', 'vo', 'hvhav', 'cvhv', 'vhv', 'jca', 'vh', 'ahva', 'vv', 'hvv', 'oho', 'qaoo', 'vv', 'ovho', 'ha', 'v', 'hoha', 'hoa', 'ha', 'aqvho', 'wi', 'o', 'hah', 'vh', 'aaa', 'hvva', 'h', 'hah', 'ov', 'hih', 'va', 'va', 'ohhhho', 'ohav', 'phhvhv', 'hqoh', 'h', 'a', 'ohj', 'aa', 'a', ],
['hh', 'ov', 'oov', 'a', 'vvh', 'h', 'vvva', 'o', 'hh', 'v', 'aoao', 'h', 'avh', 'ooa', 'a', 'h', 'hhoa', 'oovvab', 'bvh', 'vaphaav', 'hhh', 'v', 'ahh', 'hh', 'oavv', 'ovh', 'ovoo', 'aova', 'avaao', 'hj', 'q', 'ov', 'hx', 'o', 'vh', 'oh', 'oav', 'ah', 'ohhvo', 'vaaq', 'ohhvv', 'va', 'va', 'ooj', 'ov', 'ovovo', 'hov', 'b', 'v', 'aov', 'vvv', 'h', 'aov', 'vvaov', 'a', 'aav', 'vqvvov', 'a', 'oaav', 'ao', 'aohv', 'ov', 'va', 'a', 'h', 'vvai', 'x', 'oaao', 'wq', 'hwvva', 'av', 'a', 'aaah', 'ohh', 'vh', 'oi', 'hh', 'bvjoi', 'vh', 'o', ],
['ha', 'ahv', 'aoio', 'hv', 'ahv', 'ovvi', 'vvoh', 'a', 'o', 'ov', 'o', 'voah', 'hhhvv', 'va', 'hvho', 'o', 'o', 'pahja', 'hh', 'o', 'hhvvo', 'o', 'aj', 'a', 'v', 'vvahao', 'vaqv', 'vi', 'hhhvh', 'o', 'x', 'aav', 'v', 'ioh', 'o', 'hav', 'vhv', 'oho', 'h', 'va', 'oh', 'ahva', 'vv', 'v', 'h', 'ao', 'h', 'ohv', 'ahhx', 'ow', 'aohhav', 'cboa', 'o', 'h', 'vah', 'vvxo', 'aho', 'v', 'o', 'vhh', 'ovo', 'aoh', 'ovov', 'x', 'vh', 'ahoa', 'ao', 'owco', 'ho', 'h', 'vav', 'ho', 'va', 'vvo', 'o', 'hv', 'v', 'oov', 'oaooo', 'oaah', 'o', 'vhvaa', ],
['ah', 'aa', 'vo', 'hha', 'hh', 'ahh', 'v', 'vh', 'a', 'o', 'hvaahvha', 'vavh', 'aah', 'xa', 'vao', 'v', 'aoah', 'v', 'v', 'h', 'hvho', 'aahoh', 'v', 'o', 'ov', 'vvax', 'oh', 'ao', 'haphvav', 'v', 'a', 'apvj', 'h', 'ooh', 'hav', 'xvao', 'vv', 'hahahh', 'avv', 'ha', 'v', 'v', 'hv', 'ho', 'oaq', 'hov', 'q', 'h', 'vh', 'vhovp', 'abocvv', 'oh', 'oa', 'ox', 'oav', 'aov', 'bha', 'av', 'b', 'i', 'h', 'va', 'va', 'oo', 'vho', 'vaoooo', 'obo', 'aia', 'aa', 'hj', 'av', 'voa', 'aa', 'oaav', 'ho', 'b', 'vvvov', 'ja', ],
['aa', 'cbhhv', 'ho', 'oav', 'hh', 'hq', 'v', 'hv', 'ao', 'v', 'ao', 'awpahah', 'ovv', 'vva', 'vi', 'oo', 'aoao', 'aaa', 'awaho', 'avxvaaa', 'a', 'qhw', 'ohvoh', 'ba', 'ov', 'voo', 'v', 'va', 'va', 'jai', 'a', 'va', 'a', 'bvha', 'aho', 'aoav', 'pvh', 'a', 'vo', 'vha', 'aov', 'avxj', 'a', 'voo', 'ooahv', 'avvh', 'o', 'pvaaa', 'abh', 'hv', 'oav', 'ho', 'avha', 'v', 'v', 'oqa', 'vao', 'avv', 'ho', 'bva', 'ho', 'ovho', 'hah', 'vo', 'oa', 'o', 'oo', 'v', 'oahvo', ],
['oovvv', 'av', 'ahva', 'vo', 'oha', 'aa', 'o', 'vvbh', 'v', 'o', 'h', 'aaxa', 'bb', 'aav', 'avhv', 'oa', 'vhh', 'v', 'wh', 'av', 'ah', 'a', 'o', 'voa', 'bah', 'v', 'v', 'h', 'o', 'ah', 'vaaha', 'apvhv', 'cv', 'hqavo', 'vxavh', 'a', 'ovah', 'hao', 'avooo', 'av', 'h', 'ohvh', 'a', 'xhxv', 'a', 'p', 'v', 'a', 'vaaa', 'ohov', 'ho', 'va', 'vov', 'oaoio', 'aaohv', 'o', 'ahvh', 'oawoo', 'aaah', 'vhv', 'aaov', 'vo', 'ah', 'vh', 'av', 'hvov', 'hahhvv', 'hov', 'ohoho', 'ahvha', 'ha', 'oavv', 'oh', 'h', ],
['qhbvp', 'aca', 'c', 'vo', 'o', 'pcwo', 'voajh', 'haavh', 'hh', 'a', 'bov', 'hhaov', 'hv', 'aa', 'ao', 'jao', 'havih', 'h', 'o', 'a', 'ab', 'ov', 'o', 'aa', 'oohv', 'ha', 'vv', 'o', 'hh', 'h', 'ov', 'a', 'hii', 'hao', 'ohoc', 'o', 'oaa', 'vvvc', 'vho', 'h', 'hhvh', 'o', 'vv', 'a', 'aovioa', 'haa', 'avohoha', 'hovh', 'oahho', 'o', 'v', 'jhhha', 'ca', 'ohovvav', 'vaa', 'vaoo', 'hh', 'avjv', 'va', 'vo', 'aa', 'v', 'ha', 'vvhhhc', 'hvvvh', 'vo', 'a', 'voo', 'vv', ],
['voao', 'cvh', 'ovovv', 'hoa', 'h', 'hawwh', 'ov', 'aov', 'vao', 'o', 'ao', 'ovvvav', 'ho', 'a', 'hiah', 'hh', 'avvvah', 'aah', 'v', 'ovh', 'oahv', 'o', 'hoa', 'hvovoa', 'va', 'avhv', 'bh', 'oaojh', 'a', 'av', 'aavvp', 'boo', 'vqv', 'o', 'bh', 'hoohaq', 'v', 'vhhov', 'hhh', 'vaova', 'ohaaa', 'vh', 'a', 'ahva', 'v', 'hv', 'ao', 'ahoha', 'ov', 'vh', 'va', 'avv', 'oav', 'bhwo', 'va', 'oh', 'ocvaa', 'h', 'v', 'ovo', 'o', 'oh', 'vhc', 'hvv', 'hhoa', 'ava', 'o', 'a', 'hp', 'aovoov', 'aaaoaw', 'ooava', 'c', 'haooa', 'hh', 'v', 'ovh', 'ha', ],
['a', 'opv', 'v', 'oa', 'ovhhov', 'ha', 'ovvav', 'hhj', 'o', 'v', 'vv', 'hahhp', 'ahhv', 'ahoha', 'ohpoo', 'avvp', 'oh', 'aa', 'ov', 'aoah', 'vavva', 'h', 'vahv', 'a', 'ov', 'hhvho', 'vh', 'aoah', 'a', 'ho', 'aoovoh', 'av', 'i', 'h', 'ovi', 'a', 'co', 'ovo', 'ovq', 'vcvqvv', 'hhhah', 'ohv', 'vhha', 'v', 'v', 'aoo', 'va', 'va', 'oh', 'vh', 'av', 'h', 'xahx', 'avh', 'aa', 'vvvaa', 'ha', 'a', 'h', 'voa', 'vow', 'vo', 'havooh', 'ca', 'hha', 'ho', 'vh', 'oo', 'xho', 'av', 'oahaoa', 'aaa', 'ova', 'v', 'aho', 'oa', 'aaava', 'avvh', 'hqah', 'ih', 'hh', 'pv', 'aa', 'v', ],
['a', 'vap', 'oo', 'o', 'ajxvv', 'aai', 'oh', 'ahhw', 'jhqaohh', 'oavao', 'o', 'boooho', 'oov', 'v', 'o', 'oha', 'aoh', 'ovh', 'ovah', 'hha', 'h', 'hovo', 'x', 'ohovv', 'phohc', 'pohava', 'vqha', 'o', 'jo', 'ovhao', 'hhhh', 'oo', 'oh', 'oahhha', 'vvj', 'aoa', 'aoovvh', 'aaa', 'ov', 'q', 'hh', 'oahvo', 'vahao', 'ahb', 'aa', 'oo', 'v', 'aia', 'a', 'hov', 'hv', 'hhoov', 'ahoh', 'hva', 'a', 'hooao', 'va', 'ha', 'vo', 'vo', 'v', 'ah', 'hao', 'oov', 'va', 'h', 'hohvhov', 'vhho', 'ohov', ],
['bhvh', 'vaooj', 'v', 'oahoa', 'vaa', 'va', 'hhh', 'xooh', 'hh', 'va', 'x', 'v', 'aaoo', 'a', 'ao', 'avw', 'i', 'hohhh', 'oao', 'oa', 'ohvha', 'ohv', 'h', 'ov', 'p', 'ah', 'ohh', 'xoh', 'aaaoo', 'ooo', 'avaihh', 'avh', 'hhhhv', 'ovv', 'coo', 'v', 'o', 'v', 'co', 'vvv', 'ooo', 'ovvaa', 'oxv', 'vvvo', 'ho', 'hoo', 'h', 'oho', 'ohvahvao', 'vao', 'oaa', 'aaao', 'aa', 'v', 'i', 'v', 'hhv', 'ooah', 'h', 'oa', 'phav', 'hhahha', 'hv', 'vchoh', 'h', 'ahhac', 'ph', 'oaqo', 'cqoa', 'h', 'oahh', 'ohao', 'aohav', 'hhhh', 'ba', ],
['vaoj', 'v', 'oo', 'oo', 'hh', 'aa', 'vo', 'hvoo', 'hvv', 'aoooh', 'voav', 'ha', 'vo', 'o', 'h', 'hav', 'h', 'vaoo', 'ovhh', 'avhh', 'hhvv', 'ovv', 'vhav', 'ha', 'hh', 'oa', 'hvaa', 'aqav', 'vhvao', 'hh', 'hoov', 'vva', 'vv', 'qhva', 'a', 'ov', 'avv', 'v', 'ao', 'va', 'ah', 'hv', 'jvaio', 'aoa', 'jha', 'v', 'ooa', 'vohh', 'haowhq', 'hvva', 'hh', 'o', 'xha', 'vvavvv', 'hhvo', 'hh', 'ovh', 'ha', 'v', 'av', 'chi', 'a', 'av', 'va', 'o', 'qahv', 'hovohw', 'o', 'j', 'hv', 'woo', 'ov', 'oo', 'v', ],
['oavhh', 'ahvv', 'hoho', 'p', 'p', 'o', 'v', 'o', 'ahhoova', 'hi', 'vh', 'v', 'joho', 'xvo', 'hh', 'oa', 'ohvo', 'ha', 'ova', 'oqao', 'qo', 'oha', 'h', 'hovo', 'hx', 'h', 'haa', 'ov', 'vav', 'ovoh', 'aaaha', 'x', 'hvva', 'x', 'jaa', 'aa', 'avo', 'c', 'abo', 'oo', 'ajvhv', 'pov', 'ah', 'oopoav', 'hh', 'pv', 'ca', 'vavhoo', 'c', 'hvooov', 'aaoav', 'avha', 'haho', 'aqa', 'aa', 'v', 'oa', 'a', 'j', 'hva', 'vv', 'v', 'hj', 'v', 'cha', 'ho', 'va', 'vo', 'aoo', 'vhav', 'hp', 'o', 'vo', 'va', 'ovoa', 'ov', 'hhhox', 'hvo', 'vvh', 'waho', ],
['hahv', 'o', 'o', 'ap', 'hvv', 'oa', 'aovh', 'a', 'a', 'bo', 'vv', 'hh', 'aa', 'qoaxo', 'oh', 'v', 'oh', 'o', 'iaa', 'avavv', 'oa', 'vva', 'aoh', 'hohhoh', 'aah', 'h', 'ah', 'ahh', 'hh', 'aavv', 'c', 'o', 'oho', 'j', 'ohhai', 'abh', 'hv', 'oooa', 'hvv', 'hoavaa', 'oahh', 'h', 'a', 'wvhv', 'o', 'vh', 'oaav', 'av', 'b', 'ovoh', 'aovao', 'oo', 'oa', 'o', 'ovv', 'aw', 'vooho', 'a', 'hh', 'hvhoa', 'vvao', 'ha', 'ovv', 'h', 'bav', 'jxoa', 'va', 'o', 'ooh', 'v', 'v', 'o', 'vah', 'ov', 'b', ],
['hia', 'hhvov', 'vap', 'h', 'hao', 'hvoo', 'ha', 'o', 'hxh', 'ovo', 'a', 'h', 'jhooao', 'vhh', 'hhh', 'a', 'aavc', 'h', 'oav', 'v', 'ahp', 'vvv', 'ooo', 'i', 'ov', 'oohov', 'ov', 'avohhov', 'vwco', 'oo', 'ohh', 'aao', 'aoh', 'oahhv', 'a', 'vhv', 'aovvhh', 'o', 'o', 'oha', 'hh', 'viwa', 'a', 'aavo', 'vv', 'va', 'ha', 'i', 'oaaha', 'vv', 'ovhoa', 'vohaa', 'avo', 'p', 'aoah', 'ha', 'aa', 'oaq', 'hho', 'oa', 'ahvo', 'xa', 'haoav', 'oqoha', 'j', 'hao', 'va', 'ovv', 'o', 'vh', 'ho', 'h', 'ooh', 'vo', 'avvhh', 'ajva', 'o', ],
['oa', 'aaoovh', 'v', 'vv', 'a', 'v', 'v', 'xv', 'hv', 'vhoh', 'o', 'h', 'ahavhh', 'hvvqa', 'oqv', 'ah', 'ova', 'oaa', 'covv', 'v', 'i', 'v', 'ahh', 'ovh', 'ovh', 'oaoo', 'ovhj', 'h', 'avaaai', 'aohw', 'hvv', 'jovhaao', 'h', 'ooh', 'ha', 'vq', 'vha', 'a', 'vvo', 'hv', 'oo', 'o', 'aohvv', 'aj', 'joaah', 'vo', 'pa', 'ap', 'oo', 'vvvh', 'aovo', 'ocooa', 'oav', 'hhoh', 'hao', 'hha', 'vv', 'oo', 'o', 'vo', 'va', 'vvh', 'h', 'aahao', 'oo', 'hvh', 'av', 'paha', 'ahh', 'h', 'ao', 'vva', 'v', 'avojhwv', 'o', 'w', ],
['vhvho', 'hvvao', 'vahoo', 'bvvv', 'o', 'johao', 'ov', 'oohv', 'v', 'owah', 'h', 'voahvv', 'b', 'oa', 'aao', 'avv', 'vwj', 'vo', 'h', 'a', 'h', 'hv', 'caho', 'ohj', 'hhaoa', 'hh', 'aa', 'vh', 'o', 'v', 'vhh', 'vvh', 'ha', 'vvoa', 'v', 'v', 'a', 'vv', 'h', 'aoavv', 'j', 'ah', 'aa', 'vhov', 'ovva', 'ooh', 'v', 'vohh', 'vho', 'ov', 'hv', 'ajaapa', 'vvaav', 'oaa', 'via', 'ao', 'v', 'oqoa', 'hovv', 'o', 'vah', 'a', 'avoh', 'v', 'hvah', 'o', 'h', 'vhaoo', 'o', 'a', 'hvh', 'bav', 'ao', 'i', 'va', 'haob', 'voh', 'o', 'jo', 'haqwvv', ],
['vhhvh', 'oaa', 'pov', 'vaw', 'hhh', 'wvoh', 'hj', 'aah', 'p', 'vv', 'a', 'v', 'av', 'o', 'aah', 'oao', 'jo', 'av', 'aavba', 'v', 'qv', 'ahva', 'hoh', 'oa', 'a', 'hav', 'a', 'oov', 'ophva', 'hvv', 'ovva', 'hav', 'h', 'ah', 'havai', 'oooo', 'ha', 'vh', 'ho', 'a', 'oha', 'oaovh', 'o', 'hajaovao', 'a', 'hv', 'hhoi', 'havh', 'vh', 'aoooa', 'aavahv', 'aa', 'a', 'a', 'avvo', 'h', 'ava', 'cva', 'aoaaa', 'hhoo', 'avvxx', 'a', 'owhvh', 'aho', 'aj', 'aahhv', 'oa', 'a', 'vhvho', 'vva', 'vavhv', 'ovhvv', 'voohvov', 'vvoov', 'oob', 'oa', 'v', ],
['ovv', 'vh', 'ho', 'hhv', 'h', 'vo', 'a', 'vh', 'ooooajw', 'h', 'vahh', 'q', 'haa', 'a', 'a', 'oao', 'oooaov', 'hao', 'vvaoh', 'hh', 'hh', 'vo', 'vva', 'vxa', 'vch', 'a', 'hv', 'jxa', 'voa', 'a', 'vvv', 'oao', 'vovo', 'vvo', 'avvv', 'vvhv', 'pbh', 'vv', 'hah', 'a', 'q', 'avohb', 'avav', 'oh', 'hvav', 'oa', 'ohavoa', 'a', 'ha', 'ao', 'vaoa', 'voov', 'w', 'ahv', 'ha', 'oo', 'hva', 'vhv', 'aav', 'oh', 'oav', 'hao', 'haj', 'ov', 'bv', 'hv', 'vo', 'hvvah', 'hvv', ],
['aaaa', 'vvav', 'vxo', 'oo', 'vao', 'vio', 'voooo', 'cav', 'avho', 'ohv', 'bha', 'ah', 'vbh', 'oaoh', 'hvh', 'v', 'ah', 'ao', 'va', 'hohw', 'o', 'vv', 'aho', 'x', 'aovh', 'hoo', 'wov', 'vahoh', 'vvv', 'ao', 'o', 'ao', 'aoaa', 'hhvavh', 'opah', 'oojo', 'ah', 'vhhh', 'oo', 'oov', 'q', 'ao', 'o', 'apoh', 'oh', 'ohhx', 'whoap', 'o', 'v', 'ao', 'hhvoaa', 'avhvoh', 'x', 'oh', 'oaaah', 'vah', 'ojovv', 'ahhih', 'ovhvh', 'voh', 'oh', 'hohco', 'ahoa', 'ooooa', 'havo', 'o', 'oaoo', 'ah', 'cho', 'oah', 'a', 'vo', 'o', 'ovhq', 'a', 'av', 'va', 'wooa', 'qvoa', 'v', 'ah', ],
['hv', 'oj', 'hhovab', 'hvo', 'ahah', 'aa', 'hoaaa', 'vv', 'a', 'oo', 'h', 'vhv', 'h', 'vha', 'avhhh', 'oajao', 'a', 'a', 'oavv', 'v', 'o', 'oaa', 'bqooava', 'v', 'ho', 'vhobo', 'o', 'ohhhvvh', 'hvo', 'v', 'vha', 'ohq', 'xv', 'ao', 'vvha', 'vaahha', 'vvv', 'jab', 'o', 'o', 'vha', 'h', 'aj', 'hv', 'hh', 'ahvvh', 'xh', 'ohoo', 'aaa', 'hoj', 'av', 'oh', 'vavaox', 'wh', 'oi', 'ah', 'a', 'ahq', 'ao', 'hoob', 'aai', 'oq', 'av', 'ho', 'ovvvo', 'hao', 'ava', 'oo', 'haqo', 'vaow', 'oo', 'wi', 'hwoovvh', 'bhc', 'ahah', 'ca', 'caha', 'coha', 'h', 'vh', ],
['vo', 'aoo', 'jh', 'ivo', 'oo', 'hova', 'oa', 'op', 'vwp', 'ov', 'a', 'aoh', 'o', 'ao', 'oho', 'ohoo', 'hjvoh', 'a', 'voh', 'ho', 'q', 'v', 'aa', 'hvoh', 'aob', 'hioahp', 'ov', 'a', 'xoh', 'a', 'haavo', 'v', 'ah', 'o', 'vahvw', 'hhvv', 'ia', 'ovia', 'oohoa', 'ha', 'o', 'b', 'o', 'va', 'oxhv', 'oaaah', 'voa', 'oav', 'aha', 'b', 'c', 'vv', 'hvja', 'ho', 'hoov', 'obo', 'ooo', 'ajw', 'i', 'oovoh', 'o', 'h', 'qhchv', 'vvoa', 'av', 'ov', 'vwqaa', 'o', 'oaaqx', 'a', 'oh', 'vhahooo', 'o', 'av', 'o', 'ohao', 'o', 'vvwa', 'aovhh', 'a', 'ha', 'ojoa', ],
['aooo', 'avvv', 'aoha', 'oo', 'ovav', 'oah', 'oa', 'hhooo', 'v', 'xavha', 'aaahjh', 'h', 'p', 'ha', 'b', 'avaa', 'oooa', 'h', 'ha', 'ovo', 'oopah', 'ah', 'h', 'ahva', 'oaa', 'aaho', 'ovh', 'hoa', 'oa', 'hh', 'o', 'p', 'ha', 'h', 'ha', 'o', 'hx', 'hhh', 'o', 'ah', 'vhao', 'avao', 'oo', 'va', 'axa', 'aoc', 'ah', 'hav', 'o', 'hv', 'oo', 'h', 'oha', 'ob', 'h', 'ovh', 'o', 'v', 'abav', 'ohbha', 'awao', 'haooo', 'vav', 'h', 'vaa', 'pv', 'hvo', 'ia', 'oavooq', 'q', 'o', 'hhovaav', 'ahh', 'hawa', 'iho', 'oa', 'vb', 'vc', 'vvoo', ],
['bivh', 'vav', 'ia', 'ooaa', 'oohv', 'bo', 'ha', 'hw', 'ovhhh', 'pv', 'hovhvhh', 'jvao', 'bo', 'hhv', 'vavih', 'ahvhv', 'aha', 'o', 'vo', 'o', 'v', 'ao', 'hx', 'ch', 'ha', 'a', 'vv', 'vv', 'oah', 'aa', 'a', 'a', 'i', 'avhb', 'vha', 'q', 'ahhh', 'ooh', 'ha', 'hh', 'h', 'vo', 'a', 'voa', 'ahv', 'vvv', 'ov', 'hhavovv', 'a', 'hha', 'ph', 'voh', 'vo', 'hava', 'oahv', 'hh', 'ohh', 'vhoah', 'hh', 'v', 'ohah', 'v', 'oav', 'wao', 'aaho', 'o', 'ov', 'oa', 'avov', 'qho', ],
['hh', 'pv', 'vh', 'jvhc', 'oa', 'vv', 'vh', 'avo', 'hh', 'ivw', 'h', 'vabo', 'ha', 'aao', 'avaoav', 'oohv', 'ho', 'h', 'oj', 'ovv', 'ahbv', 'vaavv', 'o', 'ja', 'oahoov', 'voq', 'vo', 'o', 'ohpv', 'hh', 'hbooh', 'hvo', 'ow', 'o', 'a', 'h', 'a', 'hh', 'haovvhh', 'oa', 'ah', 'avo', 'voaw', 'vavova', 'ao', 'vc', 'oaov', 'a', 'vh', 'x', 'op', 'av', 'hv', 'hv', 'aavh', 'vphv', 'v', 'aa', 'aahvaa', 'a', 'aa', 'voaqv', 'voo', 'cvao', 'avhav', 'oaa', 'av', 'v', ],
['jbpo', 'qvpwoa', 'pp', 'oi', 'pw', 'pp', 'oj', 'wipah', 'bhbi', 'abaw', 'ipi', 'bb', 'bv', 'b', 'v', 'wab', 'obw', 'pvx', 'vq', 'iop', 'vbpiv', 'wajv', 'woq', 'wppwhai', 'apvb', 'iwiww', 'wb', 'wbhp', 'pbavv', 'ijpv', 'woh', 'whhb', 'i', 'o', 'v', 'aip', 'ob', 'pip', 'bvibw', 'iopp', 'iopw', 'hvv', 'appvibv', 'vp', 'b', 'ph', 'v', 'po', 'ba', 'o', 'i', 'vc', 'joivh', 'h', 'iwia', 'x', 'pwp', 'wpbhp', 'wwwh', 'p', 'w', 'wpiivo', 'hbw', 'q', 'ob', 'bp', 'avhbp', 'iph', 'ac', 'a', 'a', 'w', 'ivop', 'bi', 'ap', 'bha', 'booxv', 'pbaw', 'bihahbw', ],
['paw', 'vhqwho', 'ppiaw', 'iiai', 'baivwb', 'pi', 'h', 'hw', 'piwi', 'wjwbph', 'b', 'q', 'w', 'wbip', 'iba', 'o', 'p', 'ihi', 'viwh', 'bpw', 'wo', 'iwwa', 'paib', 'bhw', 'w', 'p', 'j', 'pi', 'pivv', 'i', 'p', 'hb', 'pop', 'ho', 'iqh', 'ip', 'h', 'oaobq', 'pp', 'wwp', 'i', 'ia', 'bwv', 'aio', 'b', 'ah', 'wi', 'i', 'p', 'pi', 'j', 'iio', 'pph', 'wi', 'biw', 'xip', 'ibb', 'pbp', 'biw', 'p', 'ai', 'pv', 'pi', 'phbp', 'hv', 'boi', 'wv', 'iw', 'cw', 'aa', 'ihp', 'bv', 'wppw', 'ih', 'hp', 'wv', 'va', ],
['b', 'bopww', 'vbii', 'j', 'wiw', 'wvwwwb', 'jwipx', 'iiq', 'b', 'bix', 'h', 'o', 'baphb', 'vqoiic', 'hbp', 'qppa', 'oobivb', 'pi', 'a', 'hia', 'bhaiqww', 'ajopb', 'wv', 'q', 'apw', 'iq', 'i', 'hb', 'b', 'wpwha', 'aap', 'hw', 'wp', 'a', 'w', 'v', 'wp', 'hw', 'b', 'bib', 'pvibiwa', 'wib', 'c', 'hbha', 'qb', 'viwvo', 'vvx', 'pacxp', 'bvhw', 'wiipv', 'iio', 'wi', 'hio', 'wpaiw', 'pijiiw', 'p', 'wbb', 'p', 'i', 'ibw', 'boc', 'owb', 'vio', 'pppx', 'chbi', 'bp', 'iichp', 'x', 'w', 'pw', 'piwp', 'wpx', 'pv', 'cw', 'v', 'w', 'p', 'qv', 'ivwo', 'wp', 'qva', 'i', ],
['i', 'o', 'p', 'w', 'vc', 'bhpaq', 'i', 'pwpb', 'bip', 'bi', 'wvx', 'aiii', 'bbq', 'hw', 'ih', 'bwoi', 'jiij', 'o', 'ib', 'iw', 'pwa', 'ohhhw', 'pvhh', 'pb', 'oq', 'hbvi', 'wbhb', 'b', 'h', 'a', 'ovb', 'hpw', 'vbpb', 'pvpi', 'pww', 'aow', 'oahopb', 'pb', 'ww', 'bb', 'bo', 'b', 'boqpx', 'wo', 'xib', 'ahb', 'wo', 'ai', 'xa', 'hpppq', 'i', 'wp', 'w', 'wi', 'ipb', 'p', 'ha', 'hw', 'viv', 'ap', 'w', 'vopii', 'i', 'w', 'wibaw', 'i', 'a', 'vbi', 'iwbi', 'aaw', 'bhiv', 'x', 'bw', 'b', 'w', 'hi', 'bb', ],
['ip', 'ph', 'ixv', 'p', 'bva', 'iowb', 'o', 'hc', 'obw', 'bbpo', 'hhwh', 'vpxi', 'o', 'hp', 'ib', 'o', 'jiiw', 'baxiwa', 'iph', 'piibo', 'vpi', 'bwa', 'vwa', 'ap', 'xw', 'a', 'pa', 'wwi', 'ii', 'pjpq', 'px', 'w', 'c', 'ib', 'bpw', 'i', 'xaw', 'ao', 'ia', 'i', 'ph', 'b', 'ip', 'vo', 'v', 'a', 'ij', 'cwb', 'wh', 'cb', 'qi', 'pv', 'p', 'hw', 'bqb', 'p', 'qpq', 'iix', 'ah', 'w', 'i', 'iw', 'b', 'opbp', 'phviwoi', 'bpa', 'ip', 'bb', 'i', 'vvi', 'o', 'ijah', 'b', 'cpbi', 'piiv', 'b', 'o', 'avaba', 'ow', 'i', ],
['wivp', 'opp', 'wbo', 'ip', 'iwxpw', 'wwi', 'wbbajph', 'wiw', 'bww', 'wbp', 'w', 'avv', 'hbopw', 'wi', 'jabi', 'piw', 'pbao', 'i', 'viww', 'vhooo', 'pjv', 'vvpwop', 'a', 'hw', 'a', 'wa', 'pviipa', 'a', 'iwwhb', 'o', 'bi', 'vawwwj', 'iappw', 'bhbv', 'bii', 'ib', 'wbb', 'pwvc', 'pawpo', 'w', 'bihv', 'pa', 'bwii', 'iw', 'iqapov', 'pw', 'oabpw', 'w', 'ww', 'bcii', 'vhw', 'xw', 'wpca', 'biw', 'wi', 'qw', 'c', 'ov', 'ipi', 'pibw', 'v', 'a', 'jp', 'ba', 'bbh', 'iaaih', 'v', 'bvwwi', 'o', 'iawb', 'oihp', 'p', 'wipa', 'hi', 'ipibj', 'v', 'bap', 'ppw', 'bapqo', ],
['vb', 'wbb', 'qpb', 'biqbh', 'appw', 'po', 'bxpiw', 'ox', 'avwb', 'pb', 'cqpph', 'c', 'ivp', 'aib', 'i', 'p', 'owwib', 'b', 'wx', 'phbp', 'pjow', 'wb', 'iqw', 'qbppcpab', 'aowabi', 'iwa', 'aixpwa', 'wp', 'b', 'pcbbi', 'bobawh', 'wohb', 'hh', 'ijpa', 'bp', 'ipw', 'iiw', 'baa', 'qa', 'ohop', 'pwpwi', 'pi', 'owip', 'ww', 'ob', 'pxp', 'i', 'bax', 'a', 'baqi', 'hb', 'hpiaiii', 'ow', 'w', 'q', 'bw', 'pbawwp', 'ij', 'vbabp', 'hibj', 'bpjhq', 'ip', 'ob', 'whi', 'ow', 'obp', 'pjbw', 'xh', 'i', 'o', 'xvip', 'bii', 'wipiw', 'wobipi', 'wvp', 'bvpph', 'io', 'pjvpvhi', 'ppbx', 'oi', ],
['b', 'b', 'owiiowi', 'pwhp', 'bwi', 'b', 'wibpp', 'c', 'h', 'pwbw', 'bpo', 'a', 'b', 'ib', 'pw', 'w', 'iwp', 'oiw', 'iwp', 'wh', 'h', 'ow', 'h', 'ipq', 'ppocw', 'ba', 'h', 'qibwh', 'bwpxw', 'b', 'b', 'wha', 'p', 'pbapj', 'h', 'hiwwpw', 'pvb', 'bi', 'wabo', 'iviw', 'bvpw', 'bvwbx', 'wwo', 'pw', 'h', 'bo', 'ppoaa', 'iiq', 'x', 'vp', 'p', 'i', 'v', 'cwh', 'h', 'ciwv', 'c', 'v', 'vhi', 'axvqbp', 'phvi', 'vvb', 'baw', 'qbvv', 'oiv', 'b', 'appv', 'wwpwhb', 'pbi', 'bobb', 'o', 'b', 'vwow', 'p', 'ia', 'b', 'pwvw', 'bp', 'bqiw', ],
['pv', 'p', 'ipbb', 'ojwp', 'w', 'hw', 'wi', 'p', 'jiwi', 'pibbp', 'oqo', 'wwj', 'wo', 'op', 'biow', 'aob', 'p', 'iw', 'hhhb', 'obw', 'w', 'pc', 'vaaip', 'o', 'w', 'a', 'wwo', 'iiw', 'biq', 'op', 'iw', 'hwii', 'pppibi', 'ai', 'wbi', 'abo', 'pa', 'b', 'pq', 'pbwi', 'jxviwv', 'pibp', 'ppaj', 'b', 'owi', 'bbb', 'bi', 'wiw', 'ibbh', 'biih', 'ho', 'i', 'jpbo', 'pbc', 'o', 'jav', 'wbhw', 'c', 'xcq', 'b', 'ai', 'op', 'biaw', 'v', 'qhv', 'hppbab', 'wipiv', 'pi', 'v', 'hbi', 'o', 'pvbi', 'vpwb', 'vx', 'iib', ],
['pwvoc', 'woq', 'wv', 'pxb', 'aw', 'i', 'cpwh', 'piwbb', 'i', 'b', 'b', 'oi', 'ip', 'i', 'w', 'hp', 'cw', 'apw', 'xo', 'jbx', 'jo', 'p', 'i', 'bvw', 'ww', 'bwp', 'hi', 'b', 'hobi', 'wi', 'jv', 'bwo', 'xpiaowq', 'a', 'v', 'ppw', 'viwiw', 'p', 'wb', 'a', 'ohc', 'b', 'b', 'iiw', 'bhbppq', 'ivbpi', 'h', 'bcxbwxp', 'icb', 'cow', 'b', 'oh', 'awqa', 'qiw', 'wqvpia', 'b', 'pjpchi', 'b', 'iw', 'iw', 'i', 'h', 'owb', 'h', 'ohqo', 'wbac', 'obw', 'bb', ],
['oawib', 'bcjw', 'jiox', 'vpopwiw', 'pv', 'p', 'wqiiwpbvi', 'p', 'hpbb', 'vpw', 'vab', 'b', 'awaw', 'wb', 'pvbiq', 'bbi', 'pw', 'o', 'wxv', 'boi', 'p', 'ap', 'wpwi', 'hvow', 'h', 'x', 'poo', 'i', 'avaqbpp', 'ihwo', 'hiw', 'i', 'ioip', 'aiowx', 'aoaow', 'ahh', 'pih', 'w', 'bcb', 'w', 'wcbia', 'jix', 'pbii', 'p', 'i', 'hpb', 'ph', 'wwwii', 'wiwci', 'pi', 'bpajw', 'pa', 'bwj', 'hb', 'haq', 'bbwo', 'obvh', 'ip', 'bbaw', 'ooi', 'poph', 'wvpiih', 'c', 'bb', 'xw', 'jpwa', 'aiwp', 'wiv', 'iiib', 'b', 'pb', 'vp', 'bw', 'iw', ],
['iooi', 'w', 'opbb', 'aix', 'bipoi', 'pvv', 'ha', 'iq', 'obp', 'wbi', 'bi', 'biwvvb', 'hwb', 'hvix', 'pcopwa', 'iw', 'phibba', 'b', 'bbh', 'h', 'h', 'ahw', 'hob', 'wa', 'ww', 'a', 'apa', 'o', 'b', 'howjo', 'xpocb', 'ibiw', 'obbap', 'b', 'h', 'v', 'oi', 'cbb', 'oab', 'pap', 'pi', 'bb', 'bp', 'awiww', 'pw', 'bbwb', 'iwoip', 'w', 'bbpba', 'wibix', 'iiwo', 'wh', 'poi', 'vobh', 'h', 'wvb', 'wib', 'bo', 'pwx', 'iwc', 'pvip', 'w', 'b', 'obi', 'ihw', 'bwi', 'pbpoo', 'voio', 'wpwb', 'b', 'ov', 'wobcob', 'w', 'va', 'b', 'hahwi', 'i', 'oi', 'hipib', 'pvqhaw', 'pw', 'hbw', 'c', 'bbxp', ],
['aqpp', 'hihh', 'bpvp', 'bw', 'bv', 'pbvvv', 'icqpb', 'pbp', 'w', 'ppv', 'wbb', 'oo', 'oipv', 'hi', 'p', 'vibw', 'hv', 'ib', 'iq', 'ixwaa', 'pp', 'pa', 'qcbi', 'jb', 'b', 'b', 'hv', 'iwo', 'ippwva', 'ohp', 'h', 'i', 'awi', 'ipai', 'biv', 'hcoi', 'wip', 'opo', 'pph', 'oobpih', 'hw', 'ipp', 'p', 'j', 'wpo', 'iw', 'ib', 'axp', 'vv', 'wpp', 'owboph', 'q', 'bvhi', 'vw', 'ob', 'ovw', 'qbiw', 'pwp', 'b', 'bji', 'pxiv', 'hop', 'wip', 'ibi', 'iiw', 'wb', 'ooa', 'ojhbv', 'ivi', 'cqvi', 'appc', ],
['w', 'ooa', 'wbph', 'bow', 'pb', 'obxp', 'bbqo', 'ib', 'voi', 'w', 'ai', 'vwi', 'b', 'iwi', 'owiwp', 'i', 'ao', 'wivp', 'b', 'wv', 'haa', 'wi', 'bwv', 'o', 'h', 'vi', 'i', 'pihwq', 'pop', 'aib', 'bpap', 'bowv', 'ap', 'wbaj', 'wwbvo', 'vw', 'phcp', 'hpjp', 'piiw', 'ijo', 'awibii', 'pb', 'wv', 'bw', 'w', 'c', 'b', 'ipab', 'ihjov', 'bbqooh', 'i', 'a', 'wbwabp', 'cbah', 'oi', 'h', 'h', 'iah', 'pj', 'i', 'bh', 'pwwb', 'pwbwii', 'obp', 'o', 'bvbo', 'ivpb', 'pv', 'viwc', 'pw', 'vo', 'wbh', 'viwo', 'bwpi', 'hiw', 'p', 'hw', 'xpib', 'woab', 'b', ],
['hc', 'poba', 'pi', 'wp', 'iov', 'ia', 'o', 'ipivjb', 'bvi', 'iwvb', 'iiwp', 'bi', 'ibhow', 'o', 'h', 'hiwpbb', 'jo', 'ab', 'i', 'iphh', 'p', 'va', 'o', 'vocpw', 'ivb', 'p', 'i', 'hvvbaoi', 'w', 'pxip', 'bwo', 'poob', 'wci', 'wiphw', 'b', 'ai', 'hb', 'bp', 'bwp', 'bbw', 'hh', 'qpq', 'wipi', 'iw', 'whpb', 'obbawa', 'wv', 'w', 'ww', 'vaw', 'a', 'pw', 'ia', 'iiw', 'hpbw', 'b', 'h', 'bb', 'obw', 'bpvo', 'a', 'ihvwap', 'ibp', 'bp', 'bw', 'ib', 'iwpb', 'hioa', 'h', 'wiai', 'ivjp', 'ih', 'pwp', 'ipp', 'bo', 'ivvw', 'bop', 'haph', 'pb', 'opaaa', 'p', ],
['o', 'wcbp', 'wivipi', 'x', 'ioi', 'ipbo', 'v', 'ao', 'xh', 'v', 'whwoa', 'b', 'h', 'bpa', 'pi', 'bai', 'jic', 'wpi', 'wwphc', 'aobi', 'w', 'apa', 'aiw', 'wpw', 'pwo', 'bwbo', 'p', 'pw', 'ii', 'pv', 'p', 'p', 'ib', 'ca', 'owbi', 'biba', 'wq', 'hpcw', 'bb', 'baaiio', 'appbw', 'a', 'bipxhi', 'ip', 'ha', 'hic', 'w', 'i', 'wj', 'wobb', 'ivbiai', 'bio', 'wi', 'ahwhpw', 'a', 'qvopw', 'opb', 'pavhqp', 'ij', 'b', 'bwb', 'hbwipb', 'pocp', 'wii', 'ibi', 'pwwav', 'pb', 'p', 'viwj', 'woo', 'bihab', 'q', 'i', 'b', 'b', 'w', 'ih', ],
['iiip', 'ipibpbo', 'viw', 'pvboo', 'jibbpj', 'w', 'jibbw', 'pww', 'b', 'aqvw', 'a', 'ppvb', 'h', 'o', 'bap', 'qv', 'i', 'b', 'bip', 'iabwiwb', 'pi', 'wbw', 'wpwpb', 'iiwv', 'b', 'bp', 'h', 'ob', 'bhobb', 'a', 'i', 'o', 'wvwih', 'w', 'aow', 'ip', 'vhi', 'piovp', 'o', 'jaa', 'cxp', 'qwaihp', 'ib', 'piwvb', 'hbi', 'j', 'hv', 'wov', 'vwv', 'oviiqb', 'w', 'bb', 'a', 'iao', 'p', 'jo', 'waaw', 'a', 'acih', 'wbbpp', 'w', 'ivhb', 'wh', 'pb', 'vo', 'o', 'ibbh', 'v', 'cbwb', 'wi', 'hoo', 'bh', 'w', 'bb', ],
['oiap', 'pw', 'pi', 'wbbb', 'wv', 'aapb', 'oi', 'qx', 'wp', 'pwv', 'vv', 'hwo', 'iob', 'w', 'wo', 'bqw', 'bb', 'ah', 'w', 'biba', 'boip', 'p', 'p', 'pb', 'j', 'w', 'pp', 'pv', 'bav', 'vip', 'vi', 'b', 'vbb', 'hppi', 'vphaa', 'wh', 'o', 'bxiwi', 'wph', 'i', 'c', 'a', 'pivh', 'ow', 'wi', 'wb', 'ohv', 'hvw', 'b', 'w', 'bww', 'hjb', 'piah', 'i', 'j', 'ibihxh', 'hb', 'ho', 'w', 'bib', 'hww', 'oi', 'wix', 'pwwvh', 'ax', 'aqq', 'i', 'wwwi', 'abo', 'i', 'ivw', 'o', 'hh', 'wb', 'wbw', 'whp', 'b', 'q', 'wihpi', 'ax', 'phwb', 'bj', 'a', 'ai', ],
['iw', 'wpi', 'wpi', 'wpvw', 'ovab', 'a', 'bb', 'iow', 'hxiip', 'wcix', 'bbwx', 'bapbb', 'ciopcwp', 'qo', 'vh', 'vjho', 'w', 'wbi', 'aab', 'cw', 'p', 'o', 'p', 'wpi', 'iwwviib', 'bqai', 'bib', 'vij', 'a', 'b', 'iv', 'bh', 'p', 'bi', 'ppww', 'i', 'vv', 'vwpow', 'h', 'bapo', 'w', 'bq', 'wi', 'cw', 'whi', 'w', 'ahbop', 'biwi', 'iba', 'bvw', 'iwpi', 'bpbw', 'b', 'bp', 'p', 'vbbvha', 'b', 'ih', 'vp', 'iibbw', 'hw', 'bw', 'bh', 'wch', 'aibi', 'bbwb', 'qbvhh', 'obia', 'xwhp', 'ippv', 'j', 'iwba', 'q', 'vow', 'bhwp', 'vvc', 'pw', 'bvp', 'bopi', 'wpw', 'ivi', 'b', ],
['bh', 'hiiw', 'p', 'opbwb', 'q', 'p', 'ha', 'i', 'io', 'pwa', 'hvpb', 'b', 'woai', 'wv', 'v', 'pwabjb', 'iw', 'hib', 'bvb', 'vjaww', 'i', 'jq', 'jibb', 'xo', 'i', 'oaihi', 'bopb', 'h', 'ppi', 'iix', 'aww', 'ib', 'xh', 'cppah', 'wii', 'hibi', 'vh', 'viv', 'wbw', 'wapv', 'wb', 'joao', 'whb', 'b', 'co', 'h', 'oipa', 'ci', 'o', 'bbbj', 'ipih', 'iipi', 'hwao', 'pba', 'o', 'w', 'wpioai', 'w', 'bvpww', 'vbb', 'bv', 'vawa', 'io', 'b', 'wc', 'qah', 'wv', 'wp', 'bwiop', 'poa', 'xi', 'bvh', 'p', 'iobwa', 'bv', 'vvhibbb', 'wpvp', 'ppw', 'aw', 'o', 'wabbbw', 'a', 'bap', 'apiw', 'iio', ],
['opx', 'pb', 'iwbb', 'w', 'pihohq', 'xhbxii', 'p', 'wpw', 'pbpxh', 'pj', 'pi', 'wp', 'wih', 'wo', 'wiibp', 'oab', 'bb', 'xova', 'ppv', 'bv', 'i', 'hjb', 'hx', 'aab', 'po', 'i', 'jab', 'pppi', 'aw', 'vobhb', 'awobh', 'ahp', 'bippa', 'ipb', 'pa', 'h', 'i', 'xhpvb', 'bj', 'i', 'vvwi', 'wbbb', 'wbbb', 'ba', 'ii', 'p', 'ioh', 'oh', 'bbq', 'vqb', 'pva', 'bj', 'pvwpi', 'w', 'ppjv', 'iw', 'pb', 'ivwbi', 'xhbo', 'aba', 'wppx', 'opvi', 'opbp', 'v', 'wpbo', 'ivai', 'hpw', 'wbw', 'ivw', 'i', 'ihv', 'ih', 'j', 'bww', ],
['bpp', 'i', 'pi', 'jap', 'pv', 'w', 'oipp', 'hai', 'ab', 'bcb', 'w', 'vpwwio', 'i', 'c', 'pp', 'vx', 'pihji', 'ib', 'pwp', 'b', 'pwc', 'bbh', 'j', 'wp', 'bwxpi', 'o', 'wpbba', 'iaj', 'bh', 'jop', 'i', 'qpp', 'bhw', 'p', 'ib', 'ahhi', 'xi', 'bwi', 'iiwpwp', 'ob', 'pxio', 'p', 'o', 'iic', 'hvww', 'bbi', 'op', 'v', 'awwwa', 'v', 'pbpop', 'ip', 'b', 'b', 'p', 'hbpwwph', 'ivbqp', 'pjww', 'wo', 'pwp', 'ibh', 'abp', 'hjb', 'ov', 'wpi', 'voi', 'av', 'ppp', 'v', 'c', 'qp', 'hbb', 'hpvb', 'bpowab', 'ipj', 'boviowb', 'bovai', 'aciw', ],
['w', 'ipa', 'oiaa', 'ap', 'ibqh', 'bi', 'wxb', 'pb', 'bvxbwi', 'ow', 'j', 'aoabw', 'apa', 'ahc', 'ob', 'w', 'woi', 'icww', 'ib', 'ai', 'ibx', 'iiih', 'p', 'b', 'b', 'oiihbip', 'povx', 'aiv', 'xiji', 'whbbq', 'h', 'ippwh', 'hiob', 'pbi', 'ph', 'bbw', 'hpa', 'pp', 'b', 'boi', 'w', 'b', 'ibip', 'wwpi', 'vbo', 'i', 'wip', 'wjb', 'hx', 'ii', 'h', 'oi', 'vbv', 'oc', 'cpqi', 'v', 'ba', 'pcvh', 'aip', 'bhbip', 'pv', 'hxw', 'pa', 'wab', 'w', 'bopxi', 'iiwi', 'paao', 'ib', 'bivp', 'op', 'ippiavp', 'w', 'woqc', 'bx', 'wq', ],
['wbi', 'wb', 'vpiw', 'ww', 'vw', 'i', 'phoija', 'vw', 'ibq', 'pb', 'wwi', 'vpx', 'o', 'a', 'bv', 'a', 'ip', 'biw', 'b', 'ibhwxi', 'hhiw', 'bphb', 'hoiap', 'aipqb', 'ap', 'ah', 'ib', 'wqp', 'wow', 'bp', 'avivb', 'h', 'wpboo', 'p', 'p', 'b', 'pb', 'cvvbp', 'ip', 'i', 'wppbh', 'bw', 'aqw', 'i', 'ibvia', 'cp', 'h', 'w', 'v', 'wwb', 'vihb', 'bb', 'ajc', 'ip', 'vh', 'bvo', 'piavw', 'qiwb', 'ip', 'bbaii', 'i', 'w', 'hjbc', 'h', 'c', 'iiowaa', 'io', 'hippj', 'i', 'phib', 'bb', 'pv', 'b', 'hpb', 'w', 'vwwaw', 'wwv', 'ia', 'i', 'wh', ],
['hip', 'poc', 'pj', 'wiiwp', 'ww', 'pa', 'bo', 'wpb', 'cab', 'wwj', 'hbbbbiw', 'ib', 'wh', 'wi', 'xbhap', 'pwch', 'w', 'pp', 'vp', 'wwpb', 'ahpbi', 'oib', 'iww', 'bc', 'vw', 'c', 'jh', 'bw', 'qo', 'vip', 'bw', 'pvih', 'bawawpw', 'hw', 'wv', 'qwpb', 'q', 'jow', 'hpp', 'w', 'w', 'w', 'owa', 'iwwq', 'ac', 'wca', 'wbhvoi', 'i', 'wp', 'p', 'hp', 'bpb', 'ia', 'vxb', 'hwbv', 'bob', 'aw', 'bpwv', 'xahvb', 'v', 'w', 'wbbwq', 'vhqipa', 'ibw', 'pwbwv', 'xw', 'app', 'ihh', 'bvq', 'b', 'wob', 'ia', ],
['pa', 'bb', 'qooc', 'ww', 'qhb', 'iiv', 'opcab', 'ia', 'bjpp', 'hbbi', 'b', 'aphw', 'hvpwww', 'ioboo', 'woxoobi', 'wv', 'bh', 'phbv', 'h', 'obc', 'qbhq', 'op', 'apx', 'i', 'ii', 'oi', 'obaw', 'w', 'i', 'owbhb', 'o', 'bpx', 'bp', 'v', 'pb', 'oov', 'wwp', 'iboc', 'ippb', 'iwbiv', 'i', 'io', 'pa', 'q', 'hw', 'xhviv', 'ibhww', 'hp', 'av', 'axw', 'hw', 'bj', 'wiwa', 'pwx', 'p', 'pxph', 'w', 'pxwww', 'iiapo', 'i', 'hb', 'xhvi', 'pa', 'bv', 'c', 'bw', 'abb', 'hbia', 'ixowaaqi', 'p', 'ibwo', 'bwi', 'ob', 'pb', 'icipc', 'ww', 'bv', 'vpa', 'wcpiw', 'ihpwpv', 'ihbv', 'b', 'i', 'ow', 'o', ],
['viw', 'jwpph', 'bo', 'w', 'wbw', 'ocvw', 'wba', 'w', 'o', 'oqi', 'hpw', 'i', 'qopp', 'io', 'vpi', 'viv', 'h', 'ai', 'oi', 'wwv', 'ppiw', 'i', 'bq', 'w', 'i', 'ihpii', 'ii', 'vbi', 'p', 'biw', 'bph', 'ip', 'pp', 'i', 'awiwv', 'w', 'p', 'h', 'qi', 'ho', 'wp', 'w', 'i', 'b', 'xj', 'vwwb', 'iiaw', 'bpp', 'bp', 'hvibw', 'o', 'b', 'bw', 'v', 'wvbw', 'o', 'hwhp', 'pbp', 'pvb', 'b', 'bbpap', 'ppaai', 'i', 'ji', 'wv', 'o', 'bp', 'bp', 'bh', 'howaw', ],
['wbv', 'iaw', 'apw', 'bxb', 'oipbo', 'pp', 'w', 'pi', 'wp', 'p', 'o', 'bp', 'io', 'ibvbpa', 'pb', 'vab', 'aio', 'oi', 'wvc', 'biw', 'bip', 'wvpb', 'oiw', 'ib', 'wiwb', 'ppww', 'ii', 'apap', 'o', 'pi', 'ihb', 'bx', 'xvb', 'pipi', 'w', 'b', 'iwii', 'vwpwc', 'vp', 'cv', 'bxiwh', 'p', 'vo', 'jw', 'baopbw', 'hbx', 'bx', 'ipiwbo', 'bwv', 'a', 'bxi', 'v', 'vpvo', 'pwj', 'bip', 'hbib', 'ocpbc', 'owi', 'vo', 'bib', 'bpavvh', 'vaj', 'i', 'wawbb', 'bab', 'pbi', 'hw', 'vpvaww', 'oovj', 'apc', 'a', 'hp', 'wo', 'pppaa', 'pw', 'i', 'hicow', 'cp', 'vph', 'h', ],
['p', 'hiwpbb', 'i', 'xhq', 'q', 'p', 'aqi', 'piwb', 'hww', 'apoo', 'bh', 'ohowp', 'h', 'w', 'iww', 'v', 'ob', 'bcp', 'ic', 'aah', 'vw', 'aibiioi', 'h', 'ipj', 'ph', 'b', 'a', 'wpih', 'wcij', 'pi', 'bhvbo', 'icav', 'ipa', 'paib', 'b', 'aawj', 'a', 'vawb', 'p', 'pph', 'cwi', 'wbqii', 'p', 'iohpab', 'xwv', 'b', 'pj', 'hbw', 'ojia', 'qp', 'j', 'jov', 'awp', 'hvb', 'xoi', 'pbb', 'hwboh', 'aawppoi', 'hvi', 'vwiwaa', 'bpo', 'poiwph', 'bbwi', 'pba', 'wwqiw', 'ha', 'b', 'p', 'oip', 'w', 'w', ],
['bj', 'p', 'w', 'wc', 'pbv', 'ci', 'ijcia', 'wppi', 'ipii', 'ww', 'avv', 'ibiib', 'hcq', 'vic', 'i', 'cbwo', 'wwh', 'ap', 'bh', 'ii', 'whh', 'obp', 'w', 'abvvhio', 'w', 'ib', 'ophvvo', 'ipw', 'w', 'bwbi', 'vb', 'b', 'wo', 'bia', 'ob', 'phip', 'ipbav', 'qbpw', 'o', 'apiii', 'pa', 'ba', 'wqpp', 'oo', 'ww', 'bi', 'ovq', 'wwp', 'jpi', 'aioi', 'p', 'wpp', 'ipbbai', 'wii', 'vpb', 'x', 'w', 'b', 'bw', 'wbi', 'o', 'ww', 'pwop', 'awq', 'ciioi', 'vqi', 'ipw', 'i', 'phca', 'qwv', 'p', 'ipw', 'i', 'vai', 'wobp', 'b', 'i', 'app', 'p', ],
['whibpi', 'b', 'ip', 'w', 'ahc', 'wia', 'pvj', 'oohhb', 'wibb', 'iiwbb', 'jhp', 'cbv', 'ib', 'ppb', 'v', 'ww', 'b', 'wv', 'ih', 'iib', 'abba', 'owp', 'wp', 'hiq', 'pbi', 'w', 'ovwobi', 'bva', 'qwiah', 'bww', 'haw', 'jbiwwp', 'h', 'pba', 'wbp', 'vhvia', 'pavw', 'ob', 'hvcb', 'v', 'io', 'wbw', 'hppi', 'aii', 'appv', 'wp', 'pbv', 'wbppwa', 'wbbpb', 'bhw', 'jw', 'bcpoi', 'aivpcoha', 'p', 'vi', 'ajjao', 'cbpb', 'wap', 'w', 'bac', 'p', 'wi', 'awv', 'bv', 'hpih', 'wvp', 'iiw', 'wp', 'whb', 'v', 'ppoaiw', 'i', 'pboi', 'ii', 'b', 'i', ],
['wiipw', 'phh', 'bbba', 'iv', 'pppph', 'vw', 'wvb', 'obpbi', 'iaw', 'jxp', 'waw', 'b', 'vwo', 'qao', 'whiiiv', 'ahw', 'v', 'pppa', 'ho', 'bo', 'whwvpob', 'iop', 'vi', 'bw', 'hp', 'xwab', 'piwa', 'p', 'wip', 'bh', 'i', 'oop', 'hb', 'wq', 'h', 'w', 'hph', 'w', 'v', 'ci', 'bwp', 'vpwa', 'wbb', 'p', 'p', 'b', 'piw', 'w', 'i', 'o', 'bwwx', 'i', 'oop', 'o', 'ww', 'i', 'hp', 'qb', 'w', 'v', 'bb', 'w', 'o', 'cii', 'w', 'pii', 'iiobv', 'h', 'wb', 'pvvx', 'q', 'bia', ],
['aviw', 'wip', 'pip', 'vw', 'w', 'bcbab', 'wbhpc', 'piwh', 'hji', 'hpv', 'bhx', 'hwb', 'oiiiw', 'wx', 'hhioo', 'i', 'jpibop', 'vwp', 'a', 'ib', 'wipwo', 'bpbpp', 'appb', 'vjwb', 'cpcw', 'whap', 'p', 'o', 'ib', 'vv', 'ip', 'hi', 'i', 'o', 'woa', 'hwop', 'c', 'i', 'x', 'o', 'ih', 'ibhvv', 'ahiiaah', 'iipw', 'ohh', 'pwb', 'qap', 'opwpqc', 'pvb', 'vp', 'wiw', 'o', 'ix', 'pv', 'hiap', 'p', 'vv', 'o', 'xbi', 'i', 'o', 'ahp', 'qai', 'cvc', 'ao', 'v', 'v', 'w', 'oap', 'oi', 'x', 'bi', 'q', 'ppi', 'o', 'pcpp', 'w', 'i', 'va', 'v', 'abqij', 'ibw', 'ac', 'iwb', 'op', ],
['abjw', 'ip', 'wjcp', 'bpvp', 'how', 'ib', 'hpw', 'w', 'vwov', 'i', 'ipab', 'v', 'pqwwap', 'b', 'pqo', 'bi', 'iiw', 'opp', 'wvq', 'w', 'ooac', 'ai', 'waap', 'phi', 'poo', 'ii', 'jbbc', 'i', 'vxai', 'w', 'wbwipc', 'iobwv', 'ivq', 'ip', 'p', 'pbxc', 'i', 'vwwp', 'ihwiba', 'apxi', 'bw', 'xbib', 'hpobo', 'biviw', 'hb', 'w', 'ohx', 'b', 'pvw', 'v', 'whc', 'abp', 'bqh', 'pi', 'hw', 'h', 'ih', 'w', 'bw', 'wb', 'iipv', 'pxq', 'ii', 'hpwb', 'aix', 'iphjip', 'hwo', 'ibbva', 'h', 'iiaawp', 'i', 'oovip', 'p', 'x', 'pp', 'ivii', 'vpwi', 'abxbx', 'ab', ],
['h', 'iwjb', 'bppvca', 'ao', 'hqb', 'b', 'civp', 'owbvvv', 'o', 'haqi', 'bo', 'po', 'x', 'wpq', 'w', 'p', 'ibibi', 'owawb', 'wbii', 'cppp', 'bi', 'hbwcap', 'boi', 'p', 'ob', 'bh', 'p', 'cbbph', 'woi', 'w', 'ivai', 'woao', 'biwb', 'pao', 'wp', 'pi', 'bwo', 'q', 'x', 'ia', 'h', 'opi', 'ih', 'ihpwv', 'owpb', 'o', 'vo', 'ap', 'b', 'pb', 'api', 'whh', 'bwcb', 'pv', 'ivww', 'ic', 'bppio', 'bb', 'xco', 'ioow', 'ihp', 'vo', 'vp', 'wpw', 'hwb', 'oiv', 'xpbai', 'hw', 'pipwi', 'hooh', 'vpw', 'b', 'qqi', 'pp', 'p', 'o', 'v', 'vpqhih', ],
['pphh', 'ovh', 'aovi', 'wvw', 'pp', 'whoio', 'wb', 'hjcx', 'wo', 'piqb', 'jv', 'bjpi', 'ah', 'h', 'vi', 'bi', 'b', 'pb', 'o', 'wi', 'ha', 'bwib', 'bj', 'biawp', 'w', 'vbih', 'ohbpi', 'q', 'xxa', 'v', 'b', 'wbh', 'iov', 'poa', 'p', 'v', 'wopxw', 'xva', 'aw', 'cwoh', 'opp', 'cia', 'pa', 'bc', 'ib', 'iap', 'qp', 'iib', 'hbw', 'hi', 'pa', 'i', 'wb', 'aba', 'iwbioi', 'qoi', 'w', 'aiboi', 'ho', 'p', 'php', 'w', 'vaph', 'b', 'ax', 'wbwpbv', 'ob', 'bi', 'pi', 'hc', 'wpib', 'p', 'h', 'w', 'p', 'pv', 'wab', 'wboi', 'pbopbb', 'aa', 'wwp', ],
['ch', 'ixab', 'pj', 'v', 'iwiio', 'jp', 'vpp', 'qipc', 'bbwwc', 'bvap', 'x', 'jvh', 'phbjoo', 'bvhbbw', 'iib', 'xbb', 'bjp', 'cvi', 'cobiop', 'b', 'oba', 'bwha', 'ab', 'bhipo', 'v', 'ib', 'po', 'px', 'awv', 'i', 'j', 'hvao', 'wab', 'va', 'wbhq', 'ioiic', 'awpw', 'iwv', 'ww', 'a', 'pw', 'bbiwi', 'xv', 'w', 'vbbo', 'p', 'xp', 'w', 'bpqhbip', 'pi', 'w', 'w', 'ii', 'vv', 'vv', 'p', 'copiv', 'ab', 'pw', 'a', 'pcbw', 'c', 'bpo', 'i', 'bo', 'h', 'po', 'wbi', 'ia', 'pa', 'x', 'pvxx', 'w', 'xwib', 'iv', ],
['b', 'wio', 'wpx', 'bw', 'hvbp', 'i', 'op', 'ohpcoii', 'pbpvw', 'qb', 'x', 'a', 'v', 'qcvav', 'bv', 'pb', 'vi', 'cv', 'i', 'p', 'iho', 'ppo', 'b', 'vb', 'vp', 'ppb', 'pa', 'vbp', 'ao', 'a', 'wiv', 'owbb', 'q', 'cohww', 'oh', 'whh', 'vo', 'pvb', 'bvw', 'ai', 'bbj', 'qio', 'ixwb', 'ip', 'i', 'b', 'p', 'i', 'apwpq', 'qipw', 'obwviobp', 'pa', 'p', 'i', 'ocx', 'a', 'ab', 'avava', 'bi', 'o', 'bp', 'bpiw', 'jh', 'wipp', 'vh', 'hvp', 'wwv', 'bp', 'iw', 'bbwow', 'wb', 'b', 'oib', 'abw', 'q', 'wpw', 'hbv', ],
['a', 'aih', 'wpbwc', 'wxao', 'bvvww', 'wxavo', 'w', 'bqpo', 'poi', 'ip', 'bi', 'b', 'coppi', 'iph', 'o', 'p', 'bw', 'vbpwap', 'pwh', 'vivap', 'iw', 'xhiia', 'bbv', 'bq', 'wpiii', 'cwvjw', 'o', 'bbiwh', 'ww', 'w', 'pa', 'vh', 'v', 'h', 'bpipi', 'hwap', 'ab', 'ivwi', 'i', 'pp', 'bc', 'bwha', 'h', 'ovoxb', 'h', 'bibavpo', 'a', 'baaw', 'qo', 'wbp', 'iwpoh', 'ipaiw', 'qco', 'pb', 'hhvwoob', 'abpw', 'ppj', 'pbi', 'jwivh', 'piwa', 'bb', 'wh', 'xwbbi', 'w', 'o', 'w', 'xvpxh', 'iw', 'vi', 'bibb', 'ppbvo', 'aiwiv', 'wbi', 'oooh', 'wia', ],
['biwwwb', 'w', 'v', 'b', 'xibvhw', 'pjb', 'iw', 'wobjp', 'pp', 'ahova', 'bp', 'wi', 'viqabxw', 'wwva', 'i', 'wpjbpb', 'b', 'pqo', 'ia', 'vwii', 'a', 'vihp', 'popopp', 'ib', 'jvwa', 'v', 'xb', 'vaibow', 'hi', 'p', 'o', 'vb', 'axw', 'pi', 'ia', 'qw', 'pboap', 'ph', 'opbh', 'wpoi', 'bi', 'pi', 'b', 'w', 'ibp', 'b', 'aii', 'cbwpa', 'wi', 'w', 'ivpi', 'o', 'pw', 'hiv', 'ob', 'bho', 'pv', 'jp', 'op', 'ppbhb', 'wbi', 'ob', 'hpa', 'ivph', 'b', 'pw', 'cpbw', 'jh', 'ivb', 'aai', 'w', 'aipw', 'aaphbi', 'vww', 'pqv', 'ba', 'xq', 'iibo', 'ipw', ],
['hpx', 'ppp', 'o', 'pi', 'wwahi', 'wvi', 'biwj', 'obbw', 'hivwip', 'wp', 'w', 'pa', 'ppa', 'ibbawa', 'pop', 'w', 'p', 'bja', 'iq', 'obw', 'bxa', 'bb', 'piw', 'wa', 'h', 'oh', 'vxp', 'ov', 'wivi', 'h', 'bwp', 'vvhh', 'ip', 'bohx', 'pvbo', 'ii', 'wip', 'iji', 'wiowp', 'wi', 'vhoow', 'qw', 'biw', 'bpw', 'jpc', 'i', 'p', 'vppa', 'hwp', 'hahww', 'v', 'baqb', 'ohwi', 'ppb', 'p', 'pwj', 'i', 'ivbbhq', 'bpcw', 'aa', 'ww', 'hho', 'whp', 'ww', 'ovpi', 'h', 'bi', 'w', 'a', 'vb', 'v', 'apcxo', 'io', 'j', 'p', 'ap', 'bwhbx', 'bawow', ],
['b', 'wiwp', 'vhqbaw', 'aho', 'h', 'ic', 'v', 'hw', 'iw', 'b', 'wobb', 'w', 'iipiai', 'wb', 'bi', 'p', 'pbwbaib', 'ibp', 'bv', 'ibbia', 'ib', 'hihpb', 'cp', 'a', 'pb', 'qx', 'waw', 'wp', 'avwpbb', 'iw', 'xiw', 'pvhx', 'ph', 'wabi', 'o', 'ppb', 'obaw', 'ava', 'iapoiw', 'wviw', 'wa', 'ww', 'ibb', 'iw', 'h', 'wbcbw', 'hb', 'pob', 'w', 'q', 'iwvvib', 'wx', 'ihp', 'ivi', 'vwbc', 'bji', 'ow', 'obi', 'v', 'piwpi', 'vp', 'w', 'pachp', 'oo', 'vboo', 'i', 'a', 'aipaoh', 'iia', 'waio', 'wo', 'hbwaw', 'bv', 'i', 'x', 'owij', ],
['oqjo', 'h', 'ih', 'jho', 'i', 'vcbbo', 'xw', 'vpb', 'baip', 'h', 'iiwii', 'pb', 'b', 'pcp', 'vpv', 'p', 'pbiw', 'vwix', 'q', 'wbbv', 'pb', 'phbjp', 'op', 'bw', 'phja', 'pqiv', 'b', 'a', 'wh', 'hwv', 'i', 'ippawp', 'i', 'pww', 'abci', 'pwqwa', 'wb', 'w', 'paiib', 'ipowpb', 'ibpw', 'w', 'i', 'hwb', 'oppbb', 'bbi', 'cb', 'hba', 'whpc', 'aww', 'cww', 'pow', 'bavh', 'qw', 'wv', 'obav', 'p', 'bpiwpi', 'w', 'wqo', 'wciioj', 'iiop', 'bhvbi', 'b', 'abwhw', 'hb', 'h', 'vpbiwv', 'hopi', 'xbwhp', 'ww', 'iwpii', 'opvhab', 'qbhxpob', 'ip', 'w', 'i', 'haw', ],
['xiq', 'xwpq', 'piw', 'wwwaia', 'b', 'vbp', 'bw', 'wpabov', 'wpw', 'ivcv', 'v', 'pbw', 'phxw', 'wbb', 'whpipvb', 'i', 'b', 'p', 'wi', 'xji', 'iibiw', 'cipaw', 'ip', 'vxio', 'vxawi', 'wib', 'c', 'ww', 'cpi', 'i', 'bai', 'phpibw', 'c', 'wc', 'v', 'viw', 'w', 'xbwbv', 'ivh', 'bp', 'iibxp', 'oi', 'bpaji', 'j', 'abo', 'oobi', 'hi', 'bb', 'ibh', 'v', 'avxi', 'abh', 'aqw', 'pphw', 'pwhv', 'i', 'pbp', 'pava', 'vvww', 'ahp', 'bbv', 'hbpoip', 'av', 'bbowh', 'ihoh', 'b', 'bpvv', 'hw', 'vi', 'c', 'j', 'ib', 'w', 'iai', 'wbi', 'w', ],
['vbb', 'b', 'ao', 'hp', 'ji', 'ap', 'ipv', 'jii', 'i', 'iaibii', 'wb', 'aii', 'aw', 'wpb', 'i', 'ojibii', 'qiap', 'abwv', 'vao', 'c', 'ppp', 'bj', 'bv', 'wi', 'h', 'q', 'opob', 'aao', 'pvb', 'jv', 'bh', 'phpw', 'vc', 'p', 'pbhb', 'oioqhp', 'av', 'po', 'hiv', 'baw', 'oj', 'h', 'o', 'pq', 'jp', 'ii', 'bpbpi', 'baiv', 'joww', 'bb', 'pab', 'whvv', 'ii', 'i', 'apvwp', 'p', 'w', 'vpi', 'apch', 'vpi', 'bv', 'pa', 'wboci', 'wh', 'ipiw', 'h', 'hvxi', 'pia', 'hwvbb', 'apbp', 'hbvb', 'vbpwi', 'poaap', 'b', 'i', ],
['ih', 'bibwi', 'hw', 'bpop', 'w', 'vi', 'p', 'io', 'vwvpq', 'ipix', 'wiipp', 'wbxo', 'bi', 'vbh', 'v', 'iabo', 'wobaw', 'bpww', 'ivwihbw', 'o', 'a', 'po', 'pxhibv', 'bc', 'ww', 'phppwo', 'iphiv', 'iwiq', 'pvbb', 'jbwh', 'vhvi', 'chviai', 'h', 'ioi', 'ao', 'wb', 'cvaib', 'appjb', 'pw', 'p', 'w', 'pxii', 'pvwpp', 'h', 'bipi', 'bo', 'wppii', 'ih', 'iix', 'ibaahqa', 'wab', 'pb', 'p', 'h', 'aio', 'bwb', 'waii', 'iwp', 'aa', 'hv', 'v', 'php', 'bh', 'bppw', 'i', 'wc', 'b', 'wiwb', 'pv', 'hb', 'ph', 'cbb', 'w', 'habo', 'iii', 'owbb', 'i', 'p', 'wibo', 'hhwxp', 'hbp', 'wiabp', 'wb', 'oba', ],
['qb', 'ivbawv', 'w', 'ipqwi', 'ox', 'bwhp', 'wbpv', 'bpb', 'w', 'bovh', 'w', 'abipa', 'iw', 'h', 'oi', 'ab', 'hb', 'pip', 'ah', 'wiaai', 'ooovb', 'ii', 'vh', 'w', 'iaw', 'wvi', 'hwi', 'vpcbic', 'icw', 'ahb', 'b', 'v', 'xwha', 'iihow', 'bop', 'h', 'awbw', 'ioixi', 'owo', 'w', 'h', 'jjpi', 'a', 'bii', 'jppb', 'cihi', 'aoihb', 'wh', 'qwiib', 'i', 'p', 'wpb', 'bwwav', 'pw', 'h', 'bj', 'b', 'pw', 'wbiq', 'i', 'pibipw', 'w', 'bi', 'ppb', 'o', 'ihip', 'ibvvb', 'o', 'bab', 'b', 'p', 'v', 'bbbhvb', 'pob', 'p', 'bobi', 'a', 'ia', 'aia', 'wq', 'cpaaw', ],
['biijw', 'c', 'qba', 'oiv', 'ai', 'i', 'j', 'wi', 'a', 'w', 'cb', 'w', 'ia', 'bbwq', 'i', 'waiaxoq', 'b', 'icc', 'jvvv', 'pwqvi', 'jiw', 'vqw', 'ca', 'ap', 'piiiw', 'wa', 'bo', 'w', 'vai', 'j', 'h', 'jopi', 'oii', 'iw', 'iwp', 'w', 'i', 'w', 'bwpbch', 'ao', 'i', 'wpawi', 'wx', 'pxpiq', 'biv', 'bwaiwa', 'cw', 'cai', 'v', 'p', 'ib', 'b', 'i', 'bvbpb', 'v', 'i', 'hc', 'iipwh', 'w', 'bhbcb', 'v', 'wp', 'vww', 'ba', 'p', 'o', 'obi', 'wqb', 'awih', 'abx', 'pxc', 'bwiao', 'qi', 'whbpip', 'hbhobp', 'iw', 'ipixob', 'p', ],
['haqhh', 'vb', 'q', 'oh', 'o', 'wv', 'bbb', 'vjv', 'vbbiw', 'bpp', 'pwa', 'wh', 'oo', 'vvi', 'bibpp', 'bo', 'p', 'woaaa', 'ao', 'ho', 'pw', 'iqix', 'iwv', 'piw', 'q', 'v', 'o', 'opwq', 'bp', 'aiwp', 'pab', 'iwco', 'x', 'boh', 'ib', 'wb', 'ho', 'bvp', 'bi', 'ohwp', 'ppii', 'ia', 'wwhvw', 'a', 'iop', 'voo', 'ib', 'hv', 'wp', 'ibi', 'x', 'hx', 'paivbhp', 'aiv', 'wb', 'oav', 'h', 'iawa', 'qw', 'pibww', 'hva', 'b', 'baaj', 'pwo', 'va', 'w', 'hw', 'xwbaw', 'a', 'vjj', 'o', 'bp', 'ioap', 'cbpi', 'pvp', 'p', ],
['wib', 'wvi', 'qiab', 'pq', 'hb', 'q', 'bhw', 'wpwi', 'aipw', 'i', 'bho', 'qoi', 'pw', 'q', 'vbbw', 'aaph', 'b', 'b', 'pwwj', 'ivp', 'wo', 'ibai', 'pb', 'wa', 'owp', 'bxp', 'q', 'w', 'o', 'bb', 'p', 'b', 'www', 'pb', 'wb', 'wcvw', 'bvpph', 'oiob', 'q', 'abvw', 'wbhbw', 'cbip', 'bvvi', 'chip', 'wpvhi', 'iwi', 'pwpi', 'bib', 'aaawb', 'wpwb', 'wi', 'iwj', 'piwqp', 'jop', 'ji', 'ow', 'bap', 'wjp', 'bohv', 'vh', 'i', 'obpi', 'pbw', 'ibhi', 'oh', 'wpb', 'xbv', 'bh', 'h', 'ho', 'vpw', 'ib', 'hoi', 'h', 'wa', 'vxih', 'pwi', ],
['wbpw', 'ii', 'b', 'bb', 'bwppwp', 'iijbwb', 'wwbw', 'bppi', 'bwbipjiw', 'pwp', 'wb', 'pii', 'bibiji', 'pii', 'piii', 'vwq', 'qpbbpq', 'w', 'bi', 'wpi', 'wwpi', 'wp', 'wp', 'w', 'wpwiwi', 'ppp', 'wwxiii', 'iwwiw', 'p', 'wwi', 'ca', 'ip', 'pbvw', 'pbpw', 'j', 'p', 'ip', 'wwbw', 'pp', 'bixb', 'w', 'iip', 'pip', 'bib', 'w', 'ibpp', 'ibij', 'iopi', 'p', 'bp', 'i', 'ibww', 'pbi', 'wi', 'wbpip', 'pw', 'w', 'ipp', 'qiw', 'piw', 'pbp', 'ppbpw', 'ipw', 'qi', 'bipip', 'pixw', 'p', 'jp', 'ipbw', 'pjiwbw', 'jwpiq', 'wbpwii', 'wwpp', 'aww', 'bbpwb', 'ip', 'bwjap', 'ibbpb', ],
['p', 'p', 'wwbb', 'wbpp', 'qiwwiwp', 'bbb', 'ww', 'ipop', 'j', 'b', 'ibbw', 'ip', 'pb', 'xi', 'wwpwiwbbc', 'wpj', 'wbpbw', 'bjjw', 'wbi', 'bwc', 'i', 'iw', 'wxbpp', 'pix', 'p', 'ib', 'p', 'vib', 'b', 'xi', 'wwi', 'ipp', 'wiipi', 'pbpii', 'bp', 'wbiwb', 'p', 'bw', 'wbw', 'w', 'iww', 'wpw', 'abhwib', 'wapwbbb', 'ibp', 'pwwp', 'cii', 'b', 'b', 'i', 'b', 'qip', 'wwo', 'i', 'wpppp', 'ih', 'wi', 'jbb', 'pwii', 'i', 'iabwb', 'ibiqb', 'bpp', 'w', 'pbww', 'w', 'ippp', 'bp', 'bw', 'ibp', 'iw', 'bib', 'p', 'b', 'iww', 'pbp', 'jwp', 'wbibcp', 'p', 'wpi', 'ihp', 'bi', 'cpiib', 'pw', 'i', ],
['wbw', 'ip', 'qwi', 'ipbip', 'ib', 'p', 'bwiip', 'bb', 'iiiw', 'wwi', 'babx', 'bix', 'w', 'w', 'ipw', 'b', 'wbp', 'wiipp', 'w', 'bi', 'bpoi', 'pbb', 'bwi', 'jippi', 'wb', 'iiix', 'pi', 'b', 'p', 'wb', 'ipi', 'wwbi', 'bbpw', 'xop', 'bbb', 'ipiwiib', 'b', 'bi', 'ixp', 'pp', 'wi', 'iwipb', 'bba', 'bbiiq', 'bwip', 'w', 'wawbi', 'vwb', 'pb', 'wiwc', 'ciw', 'b', 'iibixp', 'bib', 'ii', 'i', 'i', 'wi', 'b', 'q', 'b', 'wq', 'p', 'bb', 'pp', 'i', 'b', 'wxvw', 'wwb', 'pw', 'b', 'ppi', 'wbp', 'wx', 'bp', 'biccb', 'ibbx', 'pwbi', 'bvib', 'ibi', 'b', 'p', ],
['wb', 'p', 'pipi', 'pw', 'pbwii', 'iiwpwp', 'bwpw', 'ipwp', 'xc', 'ibb', 'ibib', 'pwbb', 'cib', 'b', 'i', 'bww', 'a', 'wp', 'ip', 'wbb', 'pb', 'pbi', 'vwip', 'ww', 'iw', 'iip', 'qibpbw', 'wi', 'bpbpc', 'pwb', 'xpwq', 'wqp', 'ivwi', 'pi', 'bp', 'bippwip', 'wxip', 'pbwpw', 'iib', 'bhi', 'pi', 'pbp', 'ap', 'bip', 'pwip', 'bwb', 'ip', 'oi', 'wip', 'bpwc', 'wpb', 'wwb', 'wbw', 'pp', 'ip', 'qbw', 'bbw', 'ibww', 'p', 'pi', 'bw', 'wb', 'api', 'qwb', 'pb', 'ppppq', 'b', 'p', 'iw', 'wb', 'bwbp', 'wbwq', 'ip', 'ippi', 'wwww', 'p', 'iwiji', 'bbcxpwwbpb', 'pp', 'bip', 'ip', 'jq', 'ww', ],
['ip', 'ibw', 'bbi', 'ib', 'wwi', 'b', 'j', 'pibib', 'ji', 'ip', 'bwc', 'b', 'i', 'ixbb', 'bbpi', 'ibc', 'b', 'w', 'bbbww', 'pw', 'wi', 'bbcbw', 'aw', 'pbb', 'w', 'wpi', 'ww', 'ip', 'ib', 'ipp', 'ibi', 'ppbp', 'b', 'p', 'ip', 'ipb', 'bbi', 'pi', 'bpp', 'www', 'pi', 'wibxbbw', 'ipw', 'cwpc', 'bwwibi', 'jxwiiii', 'qoi', 'pipiip', 'phw', 'pww', 'aipwi', 'wb', 'c', 'ici', 'wiw', 'pbb', 'wb', 'pp', 'ww', 'bicw', 'oxb', 'pppi', 'iii', 'ii', 'bwi', 'i', 'p', 'p', 'wipp', 'wp', 'iiwwwp', 'p', 'wp', 'i', 'iwbibw', 'wbpc', 'pwwpi', 'ibiw', 'iiii', ],
['wibbihw', 'wppw', 'ibwpip', 'ib', 'ppbbpii', 'wpiibi', 'ip', 'pjii', 'i', 'iwb', 'bbw', 'ci', 'wjibi', 'biip', 'ib', 'jwp', 'wp', 'wibp', 'wqbbpp', 'iiiwc', 'wbb', 'b', 'b', 'iibbjp', 'wp', 'pbbw', 'c', 'iiiwp', 'wipp', 'wbi', 'ibb', 'wpbpb', 'ppp', 'cpp', 'pibb', 'j', 'bw', 'pppw', 'pbc', 'w', 'iw', 'icbw', 'wiwp', 'w', 'w', 'ibp', 'wxipb', 'ppwb', 'w', 'wii', 'wpbwib', 'i', 'pb', 'iq', 'bb', 'iiwjb', 'ibb', 'pi', 'bbp', 'pi', 'pbbj', 'iwwb', 'bi', 'pipj', 'bbp', 'pi', 'wiw', 'b', 'pbwb', 'pppibbi', 'p', 'pb', 'b', 'pipb', 'wb', 'wbpb', ],
['pp', 'pp', 'bcw', 'p', 'pb', 'hpipb', 'bjbi', 'bw', 'ip', 'bx', 'cp', 'pppp', 'bw', 'bxi', 'bpbp', 'iwb', 'wpiiw', 'ibwi', 'j', 'iwbjbp', 'wp', 'wib', 'pwcbi', 'qi', 'owb', 'iwi', 'i', 'w', 'wbxpb', 'wwp', 'iw', 'wbpwii', 'bbib', 'iwpi', 'i', 'w', 'o', 'i', 'ipw', 'ppwp', 'bpww', 'w', 'w', 'px', 'ppp', 'wwpwb', 'iw', 'cp', 'ibhii', 'xpibpp', 'iw', 'iji', 'wa', 'wbw', 'ppwpb', 'bppbpip', 'cbwp', 'ppp', 'bbwib', 'p', 'b', 'wixp', 'c', 'p', 'bbb', 'px', 'pw', 'iii', 'p', 'iw', 'b', 'p', 'ppbi', 'bwp', 'bi', ],
['pbi', 'bcw', 'b', 'iwwwp', 'wwiw', 'i', 'bb', 'bcbp', 'ivjp', 'wiiwbb', 'bi', 'ibww', 'bbwi', 'ibab', 'wbw', 'biw', 'p', 'pbbb', 'p', 'bww', 'ib', 'ixb', 'wjb', 'b', 'i', 'i', 'ioiap', 'i', 'bw', 'pb', 'ip', 'wi', 'bi', 'bpb', 'iwiw', 'pi', 'h', 'bpbi', 'ipipbwji', 'i', 'pi', 'bp', 'wwo', 'ipb', 'bibq', 'pwwp', 'bipip', 'p', 'i', 'bx', 'ipbwb', 'pp', 'ccbipwp', 'q', 'pwi', 'qbwbbb', 'jpqb', 'bbpbp', 'bciwb', 'wb', 'bqbp', 'bwbpw', 'wp', 'i', 'wwbi', 'bi', 'wqb', 'qwib', 'pwiww', 'iiwa', 'ipiabb', 'wwpwb', 'ppp', 'bb', 'bp', 'pwbj', 'bi', 'wp', 'ibibi', 'ii', 'bbib', 'pjbpip', 'bbi', ],
['bi', 'wwpii', 'p', 'ibii', 'ii', 'wwpc', 'w', 'ixbww', 'pp', 'w', 'pppwx', 'bpbi', 'xwxp', 'i', 'wijpc', 'wpib', 'j', 'p', 'biwci', 'c', 'bpppb', 'wwp', 'wiwwi', 'p', 'iwbi', 'bb', 'pbw', 'iwbi', 'p', 'pwpx', 'bb', 'wb', 'xi', 'pwi', 'w', 'ibbi', 'wbw', 'bqpibp', 'wxbw', 'wbwwp', 'wi', 'xpbb', 'cb', 'pwi', 'ipbi', 'bi', 'pbi', 'jbiiwi', 'wpxpb', 'ii', 'i', 'bpwipw', 'ip', 'ibpiic', 'ipp', 'iwi', 'xw', 'iiipbw', 'wxo', 'ip', 'pi', 'wpi', 'bi', 'wb', 'wbp', 'w', 'wbpi', 'ipiw', 'i', 'b', 'p', 'bpbw', 'b', 'bbpp', 'p', 'iwx', 'ibbww', 'phpbq', 'bpww', 'pwpqh', 'pbibw', 'p', 'bbipwb', 'ppwb', 'wbip', 'wip', 'wxp', 'i', ],
['iwwi', 'qq', 'w', 'iiibopi', 'bwb', 'p', 'ibiip', 'b', 'iwb', 'bb', 'bcp', 'b', 'p', 'piw', 'abww', 'pbq', 'i', 'ii', 'wj', 'wbvw', 'wpwi', 'pwbi', 'bbp', 'pbbp', 'p', 'w', 'b', 'iw', 'biwiba', 'wcwpwh', 'p', 'biwa', 'pp', 'bwx', 'wwiv', 'pw', 'piwiw', 'wpwwbp', 'pp', 'i', 'pbwpbw', 'pipj', 'xx', 'ba', 'iwi', 'ippbbbw', 'bpwi', 'ppcww', 'iq', 'p', 'i', 'bbw', 'ww', 'wbpp', 'iw', 'wpqb', 'bibpp', 'ibb', 'i', 'pcp', 'iwwi', 'ippb', 'w', 'ibwww', 'cb', 'bb', 'cxwii', 'pw', 'i', 'ipppw', 'bbwp', 'pi', 'i', 'pip', ],
['pi', 'bqvb', 'ipw', 'pbiipp', 'jpq', 'ic', 'i', 'bpi', 'ibpwpwi', 'wbw', 'i', 'wii', 'pbwiw', 'pwb', 'bp', 'bp', 'iijx', 'bjipii', 'i', 'xwibpjja', 'ww', 'ww', 'wbp', 'w', 'bp', 'cpp', 'ipbp', 'bwww', 'xiwbbp', 'piwi', 'bbbwb', 'bij', 'b', 'b', 'wi', 'pp', 'pw', 'w', 'q', 'pw', 'w', 'xiww', 'wii', 'pxi', 'b', 'bbb', 'bb', 'pw', 'pppp', 'w', 'bpi', 'b', 'i', 'wwio', 'wbbj', 'ipp', 'pbqp', 'wi', 'i', 'ixb', 'bxioipxi', 'p', 'pbwp', 'bw', 'w', 'pwqw', 'ip', 'pb', 'wwbp', 'wbiiq', 'ppi', 'iqb', 'bpi', 'bi', 'ipicbbpb', 'pibw', 'wbibb', 'wpbpi', 'cv', 'ippi', 'pwi', 'ip', 'ip', 'ib', 'ip', ],
['wbbi', 'iiii', 'i', 'ipj', 'ip', 'wb', 'ppbiwb', 'bbj', 'pwp', 'qw', 'ibi', 'cwbwoww', 'jqbw', 'bcp', 'iqb', 'bi', 'wxib', 'piipi', 'bpwpw', 'pw', 'qi', 'b', 'wwq', 'ixbwx', 'b', 'vp', 'iwjwbw', 'xbb', 'p', 'xjb', 'pi', 'bhw', 'ipib', 'iipx', 'pbw', 'w', 'w', 'bpp', 'p', 'ip', 'ib', 'wqbp', 'pq', 'cbw', 'ww', 'b', 'wp', 'bq', 'p', 'bw', 'pw', 'iwi', 'cbb', 'wpw', 'pxip', 'pwwiowb', 'pp', 'ji', 'iwbbiw', 'pa', 'wwb', 'iw', 'bbwp', 'wpwi', 'b', 'ibi', 'bbw', 'xciib', 'b', 'pb', 'bi', 'bpihp', 'iwi', 'bww', 'pbbw', 'iiw', 'biac', 'pwj', 'qi', 'pp', 'wpb', 'iiiw', 'b', 'bpipb', ],
['wwbiq', 'i', 'w', 'pqww', 'pij', 'hwibi', 'bbb', 'bpw', 'ibb', 'jbb', 'ibx', 'wwpiw', 'bwxp', 'bwb', 'bxbbb', 'ipp', 'p', 'wpwi', 'wwpb', 'wibwii', 'op', 'i', 'w', 'ww', 'ip', 'qb', 'bwijq', 'wpbww', 'bpc', 'wbqbj', 'ww', 'wbpb', 'bqv', 'pci', 'iiwp', 'p', 'bw', 'ww', 'pwi', 'bi', 'wipbibp', 'ii', 'pbpwp', 'wb', 'wi', 'b', 'bpx', 'cpbqp', 'i', 'wwwbw', 'w', 'bp', 'pix', 'bip', 'wwwjwwio', 'ibpbpi', 'qpwbp', 'p', 'p', 'bbb', 'wbxw', 'wbb', 'ij', 'j', 'p', 'bwi', 'p', 'pbp', 'pi', 'pipp', 'wicxp', 'p', 'qpc', 'wbp', 'opvbbpb', 'p', 'pp', 'pwbwcb', 'pwppw', 'wj', 'ibppip', 'w', 'qj', 'wpppb', ],
['pcbpip', 'p', 'ppp', 'cb', 'pb', 'ippq', 'pppp', 'qii', 'wbii', 'wb', 'pw', 'ipip', 'bbi', 'wb', 'wi', 'pi', 'wp', 'b', 'w', 'bb', 'ipiwp', 'iicp', 'iab', 'ix', 'iw', 'awpi', 'iiwpi', 'qb', 'pp', 'iwxbi', 'bb', 'pp', 'pqwp', 'i', 'wbjb', 'wiqbp', 'wpi', 'pijq', 'ipp', 'w', 'ppjww', 'ipb', 'wi', 'jbwqbb', 'ibwi', 'b', 'pbbv', 'ix', 'xp', 'pi', 'pbpc', 'bob', 'owb', 'wpj', 'p', 'pwpw', 'jbwpi', 'wwip', 'wbpp', 'ipw', 'bp', 'iipw', 'p', 'p', 'xq', 'bw', 'cpiw', 'cp', 'pb', 'w', 'iii', 'bi', 'ii', 'pw', 'wi', 'wip', 'pw', 'ibi', ],
['i', 'w', 'bb', 'wi', 'bwwb', 'ipipi', 'ibi', 'w', 'b', 'xp', 'bbw', 'pb', 'p', 'cwc', 'cbp', 'w', 'pjb', 'bw', 'w', 'bwbv', 'bb', 'pi', 'wwwp', 'wpq', 'ipi', 'pw', 'wb', 'wipw', 'ipppq', 'b', 'qbb', 'cppw', 'wpbiip', 'iw', 'xipwww', 'i', 'iwiwpwp', 'wqi', 'iq', 'ii', 'ipvp', 'wpb', 'iwwppw', 'bbjbb', 'cpi', 'pp', 'bb', 'pppbpb', 'bp', 'wb', 'p', 'iw', 'pbi', 'bwp', 'cpbb', 'wa', 'ipib', 'wipb', 'i', 'ibwi', 'pi', 'bw', 'ii', 'wibbb', 'wcwcip', 'x', 'j', 'pbp', 'i', 'p', 'i', 'wbb', 'ww', 'iopjb', 'xiip', 'bc', 'pp', 'pwb', 'wpiw', 'a', 'w', 'pbwp', 'wpi', 'pbai', 'bbw', 'ipw', ],
['ixw', 'bwb', 'ppi', 'ii', 'wipibwp', 'biwi', 'pib', 'whi', 'wi', 'pcbb', 'iqi', 'i', 'bwi', 'wp', 'pb', 'ppppp', 'bw', 'b', 'w', 'wpbbi', 'pwb', 'ii', 'i', 'iwb', 'cbw', 'wiw', 'i', 'wpi', 'bpibbc', 'bpiwp', 'wppb', 'i', 'wbww', 'bpbi', 'ob', 'p', 'bi', 'w', 'iw', 'bbjaw', 'jw', 'p', 'bbppb', 'wpw', 'ib', 'bwiww', 'wic', 'iwwi', 'wp', 'pbpii', 'bij', 'wqi', 'i', 'ib', 'pj', 'ipi', 'xpw', 'bbib', 'qi', 'i', 'ipppp', 'b', 'b', 'pxw', 'p', 'ipcbi', 'p', 'piw', 'bwbpb', 'w', 'p', 'ibbj', 'www', 'bww', 'pwiip', 'ipqqi', 'iiw', 'ipiwqp', 'iqji', 'q', 'ibp', 'wp', 'b', 'ppj', 'apbi', 'wi', ],
['ppbi', 'b', 'bwi', 'bbci', 'bii', 'owxi', 'awib', 'iwpi', 'i', 'i', 'ip', 'ii', 'w', 'a', 'iw', 'pb', 'iii', 'wpbbi', 'pb', 'pi', 'pwpwpi', 'pp', 'wwicwi', 'ppbp', 'b', 'iipw', 'ip', 'i', 'pv', 'pp', 'pb', 'wpp', 'iwwbi', 'iiwbi', 'bjpwi', 'wpwwoi', 'hw', 'pcj', 'b', 'ipp', 'iw', 'ipp', 'wi', 'iiw', 'pww', 'ppic', 'pp', 'bwh', 'pbpw', 'pohp', 'piipppic', 'wib', 'wcb', 'wpo', 'pbj', 'wb', 'vbp', 'qw', 'wppi', 'wpwii', 'v', 'wwwb', 'wwwpi', 'bip', 'j', 'biww', 'pww', 'b', 'bip', 'xbi', 'ippbi', 'wp', 'p', 'b', 'pwb', 'pbwbcb', 'ijbj', 'i', 'ippq', 'bb', 'wwp', 'i', 'pwi', 'p', 'iww', 'i', 'bbp', 'bjppb', 'iiwpww', 'iwi', 'w', 'qpb', 'ipic', 'x', 'pii', ],
['iwbi', 'j', 'bp', 'iwww', 'wiiw', 'ww', 'owp', 'j', 'bbp', 'p', 'pwbiw', 'i', 'piippw', 'wbbbi', 'bwx', 'iixib', 'bi', 'wb', 'pwpw', 'wwb', 'hi', 'bpi', 'pp', 'pi', 'www', 'ib', 'poi', 'b', 'p', 'bbwi', 'bbi', 'pwpi', 'wpw', 'wwb', 'iix', 'ii', 'ib', 'p', 'bwipc', 'i', 'b', 'bpw', 'pip', 'bibii', 'p', 'pp', 'wawpi', 'ipwhp', 'w', 'b', 'pwiiw', 'wp', 'iihoipb', 'biip', 'b', 'p', 'qii', 'iib', 'xpib', 'wpib', 'bibc', 'ibbii', 'wi', 'pwq', 'ibp', 'j', 'bw', 'wi', 'pwp', 'wp', 'cwpi', 'b', 'pw', 'ip', 'ppwii', 'pw', 'wbwb', 'wwi', 'wbii', 'wwp', 'p', 'ciii', 'iwij', 'bi', 'ij', ],
['pii', 'wi', 'w', 'ippbb', 'bp', 'cib', 'bp', 'w', 'wi', 'pwbbi', 'w', 'wib', 'wiw', 'wi', 'piibi', 'pwihbo', 'v', 'bi', 'wwbb', 'w', 'ibcpp', 'xb', 'iqbp', 'wpbpwwp', 'wbpi', 'bw', 'w', 'w', 'qww', 'bw', 'i', 'bbi', 'qp', 'wi', 'iib', 'bipp', 'i', 'wpibi', 'pqwx', 'ccw', 'iw', 'w', 'pi', 'bpiw', 'wca', 'bppbb', 'bpip', 'b', 'bpb', 'awib', 'wa', 'opww', 'bwbpp', 'j', 'iw', 'qw', 'ww', 'p', 'wjb', 'pb', 'wip', 'wwww', 'pi', 'ibwi', 'bb', 'p', 'b', 'pwch', 'wipw', 'pwp', 'pw', 'pbbi', 'iw', 'ppwwwp', ],
['bwbbwc', 'wppi', 'bwpp', 'bw', 'i', 'pipwbw', 'ib', 'bpw', 'wb', 'w', 'qbbbi', 'wbwbb', 'i', 'bwi', 'wiww', 'jwb', 'wbi', 'bwbi', 'hwpxpp', 'pb', 'pp', 'piw', 'wiq', 'cippi', 'i', 'bbpw', 'vp', 'bi', 'wja', 'wp', 'bpiwppw', 'w', 'iwp', 'biwi', 'ibwp', 'ivw', 'piap', 'pbp', 'w', 'wwiiw', 'b', 'bwbpp', 'xi', 'biw', 'wbw', 'wwwwb', 'wi', 'pjb', 'pwip', 'wibw', 'piiwwj', 'voqj', 'p', 'b', 'iipb', 'ww', 'b', 'ipi', 'iw', 'wbw', 'jip', 'pb', 'bipw', 'pppp', 'ixaw', 'w', 'p', 'ipbbbw', 'pw', 'pw', 'pbwi', 'b', 'pppw', 'iwpxw', 'pwbwb', 'i', ],
['wbp', 'w', 'bb', 'iwqw', 'pbij', 'bib', 'ii', 'iji', 'wp', 'bbb', 'iw', 'pb', 'wi', 'wbpppjwc', 'i', 'wbb', 'iibwq', 'p', 'w', 'p', 'wib', 'b', 'ciiiap', 'bxi', 'ipbw', 'p', 'ii', 'iwp', 'wpwj', 'iw', 'ip', 'xpbbb', 'p', 'cbi', 'pip', 'w', 'xi', 'w', 'i', 'iqb', 'q', 'wqw', 'pwbwb', 'pwpw', 'wwpbwpbi', 'piw', 'wi', 'pw', 'b', 'bb', 'pww', 'pb', 'i', 'p', 'bww', 'cww', 'pbp', 'wcbbip', 'wp', 'xb', 'ji', 'pbpbw', 'wwcp', 'w', 'iiww', 'pbivw', 'bi', 'wxpbwp', 'bpjiwi', 'bwwiw', 'pwi', 'pw', 'iwiwhpw', 'bi', 'b', 'cwp', 'ppbbcb', 'bbbw', 'wip', 'pw', 'bp', 'bi', ],
['bx', 'bw', 'bw', 'wbqi', 'b', 'pq', 'bi', 'bpj', 'piiw', 'wwwb', 'bjii', 'iipi', 'wbw', 'wi', 'wpwb', 'xpibib', 'x', 'bb', 'ppwbbp', 'jb', 'wbiiqbip', 'xpopi', 'piw', 'io', 'wppiqi', 'pji', 'wjwbi', 'pw', 'iwba', 'w', 'p', 'b', 'pwpp', 'pwiipv', 'wixbij', 'bv', 'p', 'i', 'pb', 'wp', 'wpi', 'w', 'bx', 'pwxi', 'p', 'pix', 'p', 'w', 'i', 'bpbppi', 'bwib', 'bpwpi', 'biwx', 'bcb', 'ip', 'pbi', 'i', 'b', 'picw', 'ww', 'wbwwi', 'i', 'qiw', 'i', 'ii', 'bw', 'i', 'v', 'iijp', 'wbii', 'wipp', 'j', 'pixpw', 'b', 'ipppjbp', 'ibc', 'xw', 'wbi', 'bbwp', 'b', 'pbi', 'b', 'p', 'piibp', 'pbpiwpp', 'wbpibqp', 'wp', 'iwh', 'p', 'pwpvw', ],
['wq', 'iwppw', 'ppwj', 'b', 'wp', 'i', 'bip', 'iii', 'ixpi', 'b', 'pwipbw', 'pp', 'wwpibp', 'p', 'w', 'bbaq', 'wbiw', 'ppw', 'ipb', 'jp', 'wwp', 'iq', 'pbbb', 'pih', 'cwbc', 'ip', 'q', 'pqbw', 'bwbi', 'iipi', 'qiib', 'pwi', 'q', 'pp', 'wbbqwb', 'wwwv', 'bw', 'wbw', 'bi', 'pw', 'wppi', 'pqbiw', 'pc', 'pi', 'w', 'i', 'bwbi', 'jibpiw', 'bqiw', 'b', 'wbb', 'pwbv', 'wv', 'wb', 'bb', 'qw', 'wpjj', 'ipiw', 'b', 'b', 'w', 'b', 'bpihp', 'pbbbi', 'i', 'qpbp', 'b', 'pi', 'i', 'iq', 'wi', 'x', 'iq', 'ibibp', 'wppi', 'bix', 'p', 'wpbwb', ],
['bic', 'wc', 'pbp', 'iwc', 'bpi', 'p', 'ipw', 'icwp', 'i', 'iwi', 'pb', 'pbpb', 'ww', 'oi', 'ihiib', 'j', 'bppbp', 'pppbw', 'j', 'xpwjb', 'xcp', 'cxpw', 'pwp', 'i', 'pwi', 'ppi', 'iooi', 'iq', 'wwpb', 'xwx', 'c', 'ipiw', 'bip', 'p', 'ww', 'hqb', 'wbpwbwb', 'ib', 'pbpw', 'pi', 'pi', 'ipc', 'wo', 'pwb', 'pwb', 'bpqw', 'wpww', 'wbppip', 'pwj', 'biwp', 'i', 'iw', 'cww', 'pwwcw', 'bib', 'ch', 'b', 'i', 'bcwb', 'bbw', 'biw', 'wi', 'xwwwi', 'iwi', 'bx', 'x', 'bbbi', 'pp', 'ipipbi', 'wib', 'b', 'bj', 'ia', 'pb', 'wpwb', 'bixwb', 'vj', 'wi', 'iwix', 'ppb', 'pbc', 'pbbb', 'vxbbw', 'i', 'pwp', 'iivw', 'b', 'wiwp', ],
['i', 'q', 'iwwibwi', 'pxb', 'wppb', 'pqwb', 'q', 'ij', 'biwob', 'iwwb', 'px', 'wiwpp', 'b', 'i', 'pippp', 'pp', 'iw', 'pibb', 'wb', 'bi', 'wip', 'i', 'ib', 'w', 'iw', 'i', 'bwi', 'cbq', 'ippbiib', 'q', 'apxp', 'ppi', 'iwjw', 'wb', 'iwx', 'wiw', 'pi', 'qibw', 'i', 'wpp', 'p', 'iwwpib', 'jp', 'iwii', 'i', 'xwwi', 'iw', 'wi', 'wiw', 'wibpw', 'ppxq', 'pw', 'wpwiw', 'ppji', 'wbp', 'bx', 'ib', 'ipwwppi', 'ic', 'pwbwi', 'ww', 'i', 'iipp', 'pbi', 'ppp', 'w', 'bipw', 'vp', 'bbw', 'picw', 'xwii', 'w', 'bqii', 'ippp', 'jibbiip', 'bj', 'ippp', 'bbbpibii', 'pij', 'b', 'app', 'pwi', 'iiwpb', 'iiwx', ],
['bwp', 'wib', 'q', 'pww', 'iwi', 'jw', 'w', 'pwb', 'qppi', 'biiwp', 'iwo', 'ip', 'w', 'w', 'wpw', 'ibw', 'pbipp', 'ibwxw', 'pwbi', 'pip', 'bbba', 'pibq', 'w', 'ibw', 'bbppwibwi', 'wx', 'p', 'bibi', 'i', 'bbib', 'pwpw', 'ii', 'wp', 'ibx', 'w', 'wpppp', 'pbw', 'wbwbiw', 'wp', 'bw', 'bxiiw', 'ppbiip', 'iwwipi', 'bjip', 'pap', 'wp', 'iwiw', 'wwpw', 'pi', 'pwpw', 'wvwbwc', 'i', 'iiw', 'x', 'pwipw', 'wppi', 'vbpv', 'jibib', 'bib', 'ppj', 'ciwpw', 'bw', 'ap', 'ibbb', 'wpb', 'w', 'wb', 'cwjwi', 'wbib', 'bwp', 'wpbiw', 'bioi', 'ibib', 'w', 'ii', 'aaw', 'wiajwp', 'ww', 'ppi', 'wi', 'wwpb', ],
['pbw', 'wpw', 'qjiii', 'iw', 'iibi', 'wxp', 'ibwwi', 'qbppi', 'b', 'ww', 'wb', 'wbi', 'wpi', 'wx', 'bbwb', 'c', 'xpi', 'pppw', 'b', 'hiwi', 'p', 'w', 'i', 'cbw', 'bpw', 'bpi', 'ibibwb', 'bpiiiw', 'www', 'b', 'iiibpx', 'ppw', 'pjx', 'bb', 'p', 'pbip', 'bi', 'bp', 'p', 'wbb', 'op', 'pbii', 'b', 'wwx', 'ic', 'x', 'pbc', 'bpp', 'bpi', 'wipp', 'bb', 'ib', 'ibb', 'wbp', 'pw', 'pwip', 'iii', 'piw', 'ww', 'cw', 'pwwpw', 'bp', 'jp', 'pi', 'ibbw', 'pib', 'ipp', 'i', 'ppwb', 'w', 'b', 'wbpwi', 'biwp', 'ibw', 'wp', 'p', 'wpw', 'ipp', 'pi', 'iiw', 'icpw', 'bpwb', 'bi', 'ip', 'ippi', 'bbb', ],
['iwibb', 'pixqib', 'iww', 'i', 'ppp', 'ibw', 'bwwb', 'www', 'b', 'w', 'wbp', 'b', 'bbxb', 'p', 'wwipw', 'bibip', 'piw', 'pwwc', 'piip', 'wpi', 'i', 'bbbpw', 'w', 'ibw', 'owiw', 'piii', 'i', 'bbii', 'bwwpw', 'i', 'iii', 'ii', 'bpwb', 'bbwb', 'iii', 'wwi', 'b', 'wbww', 'bbjw', 'iijii', 'pb', 'pp', 'w', 'pw', 'wxqi', 'bbww', 'bwpbw', 'pb', 'bww', 'wb', 'j', 'pp', 'pb', 'wbpi', 'iwxij', 'bp', 'ipwwwi', 'ib', 'bp', 'p', 'ppi', 'pj', 'iibi', 'c', 'pjwpi', 'wqjb', 'wbbb', 'wbj', 'i', 'b', 'wbb', 'ccbi', 'bpbwb', 'p', 'ci', 'xi', 'pw', 'c', 'b', ],
['bi', 'iowqbi', 'wwibi', 'pjw', 'wibq', 'b', 'bpbbp', 'pbb', 'p', 'bpw', 'bi', 'wwxbi', 'i', 'w', 'bw', 'i', 'iww', 'ibib', 'wwbw', 'pb', 'bwbwwpw', 'wpb', 'wi', 'b', 'pp', 'pcwbp', 'i', 'bcq', 'wi', 'b', 'pp', 'ib', 'xp', 'w', 'ppb', 'bwbbi', 'p', 'w', 'bbpi', 'ppj', 'bij', 'p', 'wb', 'bwbpp', 'ib', 'w', 'bjx', 'ipjp', 'b', 'ww', 'pi', 'ab', 'iji', 'ixp', 'ippb', 'iwib', 'pp', 'wb', 'wiw', 'iii', 'w', 'wibb', 'bjwi', 'xibip', 'pp', 'i', 'wi', 'bx', 'qb', 'bp', 'iw', 'wibbw', 'w', 'wpb', 'xp', 'bbwji', 'pwp', 'bbb', 'piw', 'pbpwwb', 'i', 'jii', 'cbwb', ],
['bp', 'i', 'pppi', 'bpipw', 'bwwwwwb', 'bbi', 'b', 'wb', 'ciw', 'bp', 'p', 'cip', 'pb', 'iwwcp', 'bi', 'iwwww', 'biibq', 'iiwbippc', 'i', 'pv', 'ipj', 'iwbb', 'bpib', 'pip', 'ww', 'p', 'j', 'pbpb', 'bw', 'xipipb', 'ai', 'ibio', 'pwww', 'jpbw', 'iwi', 'bpw', 'iwcwip', 'b', 'pppp', 'bi', 'vc', 'b', 'bwc', 'p', 'wbipwi', 'bippp', 'pipwi', 'wppipp', 'ipqci', 'wbpb', 'hpiwp', 'bwpb', 'bw', 'pb', 'wibw', 'biiip', 'pb', 'wpj', 'b', 'wi', 'bbbw', 'w', 'bwip', 'w', 'xbbp', 'b', 'ibp', 'c', 'ww', 'pqb', 'bqb', 'bp', 'wi', 'q', 'bb', 'iibb', 'bibphi', 'w', 'pppp', 'w', 'q', 'i', ],
['biwppp', 'i', 'wp', 'ibpp', 'bpwwi', 'bpp', 'w', 'i', 'bbii', 'bpiwp', 'wbpiq', 'wh', 'bppjiw', 'pbpc', 'pwi', 'wbi', 'pci', 'bbj', 'bbb', 'qpw', 'wbp', 'wiv', 'pib', 'pbi', 'bbp', 'iiii', 'b', 'j', 'wxw', 'w', 'x', 'c', 'bjwb', 'bbxbw', 'pb', 'cw', 'pbi', 'b', 'pwbibai', 'iib', 'ii', 'pba', 'iw', 'bqb', 'w', 'wpbp', 'iww', 'pwpb', 'i', 'biw', 'p', 'w', 'i', 'p', 'bwpwp', 'wbw', 'pib', 'iw', 'bpb', 'iib', 'bpb', 'pbpwpw', 'bbww', 'i', 'b', 'pibi', 'bwhj', 'i', 'pbw', 'bwbp', 'w', 'bb', 'pw', 'pbw', 'w', 'wipi', 'qbppwp', 'wbpc', 'ww', 'iip', 'iwjp', 'cb', 'jx', 'qbbwww', 'iiwi', 'cwpi', 'j', 'bwiq', 'ip', 'icwpawh', ],
['jbwqiwp', 'ib', 'i', 'wi', 'ibw', 'p', 'pbbc', 'wp', 'ii', 'jw', 'bbi', 'ppxbpb', 'ibp', 'p', 'ivi', 'wb', 'wbib', 'poi', 'pibp', 'p', 'ppwb', 'iiw', 'ix', 'wwwww', 'pb', 'bb', 'i', 'wii', 'wb', 'pp', 'bbpjw', 'ibb', 'wbbp', 'bb', 'ip', 'xbbp', 'bpb', 'wpw', 'ipiiw', 'bp', 'bphwi', 'pw', 'ibiii', 'a', 'pwbii', 'pii', 'pppiw', 'ppbbp', 'bbbawppc', 'xibbi', 'iip', 'bwp', 'bjib', 'wib', 'pb', 'iwap', 'xqi', 'ppb', 'iwp', 'i', 'biwbi', 'bb', 'ihp', 'w', 'pwi', 'wwppp', 'ipwbw', 'p', 'ppwi', 'ppbip', 'bwwpp', 'ibiq', 'bwpc', 'wxp', 'bwpb', 'i', 'pbpbp', 'bp', 'w', 'xwpbwq', 'q', 'i', 'bwi', 'pipp', 'o', 'wc', 'wiw', ],
['p', 'ix', 'bbci', 'bxi', 'pib', 'pcpxpb', 'icip', 'wppiwb', 'www', 'wwi', 'bpib', 'wbwp', 'ipw', 'pwwbp', 'bwbp', 'ipbb', 'p', 'ppbbww', 'iiwwbw', 'iw', 'w', 'p', 'ibp', 'w', 'b', 'bbp', 'bw', 'p', 'wx', 'wpi', 'ib', 'w', 'pibic', 'pi', 'pp', 'wb', 'jw', 'cbwp', 'bp', 'pb', 'bpb', 'piii', 'iipp', 'b', 'bpbpipc', 'ib', 'bip', 'iwi', 'pi', 'wi', 'wi', 'iipb', 'cbpb', 'cw', 'pibbiw', 'iwcp', 'pbpii', 'b', 'w', 'ii', 'w', 'pj', 'bibwiw', 'iwi', 'bwppp', 'wcbbw', 'b', 'iw', 'bpb', 'iip', 'bibp', 'ipwcbx', 'ppiwbb', 'bb', 'bixhp', 'ppwi', 'pw', 'pppb', 'bpwpp', 'cb', 'bbbp', 'ww', 'xqp', 'pip', ],
['pwpbp', 'pxxbp', 'ipiw', 'iwp', 'bwp', 'biw', 'i', 'pw', 'bpc', 'bp', 'bw', 'wwwb', 'ipib', 'ii', 'iaw', 'pbbbww', 'bib', 'wbwbw', 'ibib', 'c', 'iq', 'pbipw', 'wii', 'pbw', 'jqi', 'ij', 'ib', 'w', 'i', 'w', 'pp', 'wpi', 'ihbi', 'vx', 'wp', 'wbpb', 'wwi', 'pi', 'bbpiw', 'i', 'bpib', 'b', 'p', 'bbpi', 'ipib', 'iwiip', 'ppw', 'p', 'bwx', 'cpbpw', 'biw', 'ip', 'p', 'bi', 'p', 'bpwpw', 'bbppp', 'ppp', 'ppppwpix', 'iiw', 'wi', 'xbbwi', 'ihpbbib', 'bb', 'biwpw', 'piwbqi', 'p', 'iwi', 'ppbb', 'wwp', 'pwpip', 'b', 'ivi', 'p', 'iwib', 'bpiwp', 'pbpbv', 'bwb', 'cb', 'pp', 'q', ],
['b', 'ipw', 'bbp', 'pi', 'wip', 'bp', 'biwpb', 'jw', 'ppbw', 'pp', 'ii', 'bipb', 'bpwqp', 'wp', 'b', 'b', 'pwp', 'ipp', 'pi', 'ppi', 'pww', 'wb', 'ib', 'piw', 'ww', 'jpb', 'pwqiw', 'ii', 'pipii', 'pwip', 'vbb', 'ppp', 'bpiwji', 'wvpii', 'piiw', 'ipip', 'wbp', 'bbvj', 'iib', 'obwp', 'p', 'p', 'bipp', 'bw', 'wb', 'bip', 'ibiwivp', 'p', 'pwab', 'qiw', 'ppp', 'xbbp', 'qv', 'pwbic', 'bwjii', 'bj', 'ippbpp', 'pib', 'biii', 'p', 'piiqb', 'iiw', 'j', 'bbcwi', 'p', 'bipp', 'pv', 'wbibb', 'bwi', 'px', 'v', 'wociii', 'bb', 'p', 'biiw', 'ppc', 'ii', 'ib', 'wbbwj', 'wiwb', 'wbww', 'bp', ],
['pic', 'b', 'i', 'ib', 'piwb', 'bpwp', 'pww', 'w', 'b', 'wibw', 'xjpji', 'ww', 'b', 'qb', 'iw', 'pcp', 'wbbw', 'bpwwwpb', 'bibbbbb', 'pwibwiw', 'w', 'jww', 'biw', 'wb', 'wipbpp', 'wpip', 'bb', 'wa', 'pww', 'qiicpp', 'ix', 'iwwi', 'bwb', 'xbbw', 'ip', 'bbp', 'wb', 'jwi', 'ppvc', 'qp', 'iwp', 'wi', 'viip', 'wib', 'pbii', 'b', 'xw', 'wpwpbwp', 'p', 'p', 'wppw', 'bppi', 'bbww', 'pbpbjpi', 'ibp', 'w', 'iw', 'pi', 'vww', 'pib', 'pw', 'pw', 'ibqwp', 'pp', 'pji', 'ipw', 'bw', 'bpp', 'cpi', 'pa', 'iwp', 'qi', 'wpwpi', 'b', 'wbpb', 'bbp', 'wibb', 'op', 'p', 'bpwipi', 'b', 'ob', 'pw', 'pbiii', ],
['ww', 'bp', 'w', 'wx', 'cp', 'ib', 'ppiw', 'bjixp', 'p', 'ww', 'ii', 'ipppi', 'ciw', 'wpipw', 'i', 'bii', 'iiq', 'pwpb', 'pww', 'ibbwwp', 'ipw', 'w', 'w', 'ppi', 'wixpxi', 'ppppw', 'bb', 'iwwppb', 'xi', 'ipbpw', 'pb', 'xwbi', 'pw', 'ib', 'ib', 'w', 'iwb', 'wpib', 'bwo', 'piwi', 'wiwb', 'wih', 'bpwi', 'pbp', 'pix', 'p', 'iw', 'pbiwj', 'bp', 'p', 'wjjp', 'ip', 'biic', 'ipp', 'wx', 'iiwiwaii', 'biwcw', 'wbi', 'pw', 'bi', 'pww', 'wwpw', 'pwi', 'p', 'bijp', 'wpxb', 'ipb', 'pbiwpx', 'p', 'iwwbc', 'bpw', 'bpib', 'pbciw', 'iibpp', 'wiwppi', 'bb', 'bbw', 'oww', 'wwp', 'wwqbij', 'wpiq', 'bbpwi', 'xp', 'bq', 'biw', 'jiw', ],
['pwb', 'iw', 'ba', 'w', 'pjb', 'a', 'vpwij', 'p', 'icw', 'bw', 'www', 'pbicbp', 'ipbx', 'c', 'bw', 'pib', 'pbiip', 'wiwb', 'bpiii', 'wbb', 'w', 'ibx', 'ipi', 'wp', 'wbqb', 'awj', 'ii', 'pbi', 'wb', 'i', 'wpb', 'xppw', 'wwpi', 'aiw', 'wwipp', 'bwp', 'w', 'wq', 'wp', 'oi', 'pibw', 'b', 'wi', 'ciiiip', 'wibpw', 'bb', 'wi', 'wwiij', 'bb', 'ww', 'bii', 'wwiq', 'b', 'jiib', 'pii', 'pbpp', 'iww', 'bp', 'pvj', 'pp', 'b', 'bibb', 'jpp', 'jba', 'i', 'p', 'wpw', 'bb', 'iw', 'w', 'jbbpwi', 'pciw', 'pwp', 'w', 'cwcx', 'bp', 'ipbpw', 'i', 'bipp', 'pbi', 'jwb', 'wwi', 'pbbp', 'bp', 'pbb', 'ibww', 'pbpixp', 'w', 'ib', ],
['bwjbii', 'ww', 'wwi', 'pcw', 'ii', 'b', 'bww', 'whbi', 'bbib', 'bwqi', 'www', 'i', 'ibb', 'wwp', 'w', 'pi', 'pciw', 'qi', 'p', 'wwib', 'bji', 'iqj', 'bpwibp', 'ww', 'wbi', 'b', 'p', 'pbi', 'bpb', 'bwppbw', 'ipb', 'ibp', 'bww', 'iwv', 'bwpw', 'bpwib', 'wpp', 'vbq', 'bjw', 'bpa', 'pwwpbp', 'xbjp', 'wb', 'ww', 'iwb', 'ib', 'p', 'cw', 'pi', 'ipb', 'ppwb', 'xjw', 'pp', 'bi', 'p', 'pii', 'wwbib', 'bpij', 'wbwi', 'iw', 'bb', 'bixp', 'a', 'j', 'obwi', 'obb', 'bwbw', 'wb', 'wiw', 'ai', 'xiw', 'bpp', 'w', 'wb', 'ibwq', 'bpqwb', 'w', 'iwic', 'ii', 'w', 'piiqb', 'i', 'pb', 'ppi', 'iqbqpx', 'ppp', ],
['piq', 'qb', 'ipb', 'ijii', 'ip', 'bip', 'ipwb', 'bwb', 'pib', 'bb', 'xx', 'ib', 'iww', 'pww', 'wwp', 'wbb', 'ii', 'wjppbp', 'b', 'bhi', 'bii', 'b', 'b', 'wq', 'hvi', 'wi', 'bp', 'pcp', 'ipwp', 'pwbw', 'bw', 'b', 'bwb', 'ppij', 'bb', 'i', 'iwbi', 'ipc', 'pw', 'ix', 'wowib', 'bw', 'phbwj', 'iiiw', 'o', 'axi', 'pibwbb', 'ipqi', 'jwi', 'bwibp', 'w', 'ixwii', 'ipwawxw', 'w', 'piwb', 'bi', 'pcp', 'bipip', 'wbpb', 'bbwb', 'ibbb', 'ppbw', 'wip', 'pbi', 'bi', 'ii', 'bppi', 'wp', 'bx', 'wcpib', 'qb', 'obwp', 'wwj', 'w', 'bb', 'i', 'i', 'i', 'pwvb', 'paij', 'wppip', 'bbwciibx', 'bwi', ],
['iib', 'bx', 'pi', 'wbpi', 'wwbpw', 'ixiwpw', 'iiw', 'qbi', 'w', 'piw', 'q', 'i', 'wbb', 'pixw', 'pwbbwpb', 'ww', 'iwiw', 'w', 'bc', 'bwpbw', 'pbb', 'bbwb', 'w', 'pw', 'hwii', 'b', 'bbb', 'pip', 'a', 'wbi', 'i', 'bipicb', 'pwb', 'wpi', 'ibwicb', 'bwcwiw', 'pippb', 'qbp', 'pwi', 'ibpb', 'iippiippp', 'ip', 'pwbp', 'wibcib', 'xi', 'pbwb', 'jw', 'ipbb', 'pwiw', 'bpiw', 'w', 'iwib', 'wbww', 'b', 'x', 'bp', 'w', 'pbb', 'wp', 'qpp', 'w', 'i', 'pbbp', 'w', 'bw', 'wbw', 'wbb', 'wbcb', 'w', 'p', 'pibwibqp', 'iiwi', 'wbww', 'ii', 'wbw', 'iwwii', 'a', 'pwbh', 'pwi', 'ippbi', 'wbq', ],
['ibpi', 'i', 'pwpbp', 'ibio', 'wipbp', 'i', 'wi', 'wwip', 'w', 'ibp', 'b', 'bip', 'jiiww', 'wbqw', 'bvap', 'ii', 'wp', 'bpi', 'ib', 'q', 'x', 'wi', 'bw', 'i', 'iwi', 'ajwx', 'pii', 'bb', 'ii', 'xbp', 'pbc', 'biw', 'ibbb', 'w', 'ww', 'cibqw', 'bi', 'wc', 'wwv', 'w', 'iwbp', 'pq', 'ip', 'bbpiw', 'wbp', 'pbi', 'p', 'bibb', 'p', 'qwc', 'p', 'pwib', 'iw', 'bp', 'pp', 'pii', 'iwbpi', 'iw', 'ipb', 'qp', 'iww', 'a', 'b', 'wwp', 'w', 'bw', 'pb', 'wpw', 'bib', 'wpijjwp', 'w', 'ibpbbiw', 'pbb', 'pp', 'w', 'bp', 'piwii', 'pbpwb', 'p', 'jwbw', 'iiw', ],
['wpb', 'b', 'bpcibp', 'bi', 'p', 'icwp', 'qboc', 'wi', 'wpww', 'ipp', 'bip', 'ipqi', 'p', 'biw', 'ii', 'pi', 'iiw', 'ipwb', 'wbiw', 'pbw', 'ppiw', 'bii', 'bb', 'biwb', 'wip', 'b', 'iiw', 'wi', 'p', 'p', 'iw', 'wiw', 'bp', 'wbwi', 'ji', 'p', 'ii', 'qpbh', 'bi', 'iw', 'ibx', 'pb', 'c', 'piib', 'bibiwwi', 'pwb', 'wop', 'pb', 'b', 'p', 'pibwp', 'ipjw', 'wpwp', 'iw', 'i', 'bp', 'wb', 'ip', 'xbbi', 'bwwb', 'pbcpji', 'bip', 'pwb', 'ipq', 'p', 'ix', 'pw', 'w', 'pji', 'pbpx', 'pvbp', 'ib', 'b', 'wi', 'qbw', 'bbixaw', 'iiiwip', 'ii', 'pjbpw', 'bwbb', 'bwb', 'ww', 'wobix', ],
['obp', 'pwbww', 'piq', 'pp', 'ipbb', 'ipbji', 'wi', 'jw', 'iii', 'pipwiw', 'ibi', 'wb', 'pqw', 'b', 'iiqxi', 'cxbiwi', 'ipbpw', 'ip', 'ii', 'ipw', 'iwii', 'wi', 'bipoi', 'iqq', 'pbicbi', 'wiw', 'wb', 'iw', 'b', 'bpiw', 'ipbqw', 'bwwb', 'wpp', 'pipp', 'pw', 'w', 'ip', 'pbwp', 'wxwpb', 'wii', 'icppbwb', 'ib', 'pp', 'biw', 'ppi', 'w', 'iiwvwi', 'p', 'iw', 'ppbipw', 'bi', 'xbp', 'iwwbp', 'wiqi', 'iwibiw', 'bcp', 'p', 'jbi', 'ixpipwp', 'bbpph', 'piiw', 'bpw', 'pwb', 'ibi', 'q', 'bwcw', 'pwb', 'cbab', 'ip', 'bbp', 'ippp', 'piiwbwb', 'iw', 'p', 'i', 'iw', 'iii', 'ixbb', 'wpp', 'pp', 'biw', 'bvpb', 'p', 'bpp', 'pw', 'p', ],
['pib', 'wphpipa', 'ii', 'bwx', 'bbw', 'ixii', 'bpcw', 'j', 'piw', 'bpbi', 'b', 'wwiwb', 'bi', 'oiwpi', 'bwi', 'jw', 'biw', 'ibbbw', 'cqw', 'pjbw', 'iwc', 'owb', 'ii', 'ibpb', 'iww', 'b', 'iw', 'wwwjw', 'ppwiw', 'bwpw', 'pb', 'ww', 'biw', 'bw', 'bw', 'iwi', 'bp', 'jii', 'ibwwi', 'w', 'pjbpp', 'iwp', 'wwbw', 'bpwp', 'w', 'bp', 'wpww', 'i', 'wc', 'cw', 'iwppi', 'iw', 'iiww', 'w', 'b', 'ib', 'piwpb', 'bbbip', 'wpiq', 'iip', 'ibbp', 'jbi', 'b', 'wwiiw', 'bwiw', 'pppi', 'bp', 'bip', 'biii', 'ipb', 'wab', 'bip', 'wbpb', 'pi', 'pwbw', 'jiwqx', 'ciixi', 'ai', 'w', 'wwwwp', 'wi', 'ibi', 'b', ],
['jpbwp', 'ppxi', 'iipb', 'p', 'jp', 'ow', 'b', 'b', 'ibwb', 'i', 'wb', 'jpbw', 'wiiwwpw', 'iww', 'w', 'pwx', 'i', 'qpi', 'biww', 'iibbp', 'ip', 'iwwa', 'pb', 'wpjib', 'wi', 'wxww', 'wii', 'ib', 'ii', 'ww', 'bwi', 'wiw', 'jb', 'bi', 'bw', 'w', 'ci', 'bwi', 'pvbii', 'bwwi', 'ip', 'pii', 'pb', 'wpb', 'p', 'c', 'w', 'wq', 'xbb', 'pjpc', 'bbb', 'icip', 'iiibp', 'biw', 'ibi', 'bwpw', 'wibp', 'wbwp', 'iw', 'bw', 'bbww', 'pqi', 'wpp', 'qi', 'wpib', 'bp', 'b', 'xw', 'b', 'pj', 'bci', 'wwi', 'wpwiw', 'i', 'bqipqp', 'pb', 'ppi', 'cx', 'wbbc', 'bp', 'ic', 'w', 'wp', 'pbbi', 'wb', 'cpojb', ],
['i', 'jbw', 'pwp', 'ii', 'pw', 'wipq', 'ppi', 'xbp', 'wpxi', 'wb', 'iiib', 'icij', 'pixiiqc', 'ppi', 'wibp', 'bxww', 'wwbi', 'iw', 'ipb', 'wp', 'hcw', 'bw', 'xbci', 'c', 'wii', 'wix', 'o', 'wbp', 'c', 'c', 'ppb', 'b', 'ii', 'xbw', 'w', 'ibwi', 'pwpwbpi', 'ijj', 'pp', 'iwp', 'b', 'bp', 'vpibq', 'iqp', 'pi', 'b', 'ww', 'bbp', 'w', 'iw', 'ipbwp', 'wiwbw', 'b', 'i', 'acwppbpi', 'b', 'wxi', 'ibibb', 'jbpp', 'o', 'w', 'pi', 'b', 'i', 'ii', 'bi', 'ww', 'xiv', 'ipp', 'bwpq', 'vbwp', 'b', 'b', 'ppjp', 'bi', 'hp', 'wp', 'i', 'wpc', 'iv', 'qw', 'bwbi', 'b', 'w', ],
['b', 'pbpx', 'bpppi', 'bi', 'bbpp', 'ii', 'bpwp', 'ppp', 'wbiwp', 'iii', 'p', 'b', 'bixobi', 'wpi', 'wb', 'wwbx', 'wix', 'i', 'bwix', 'ppbbp', 'ppwpw', 'p', 'bbc', 'ppq', 'wa', 'px', 'j', 'iicwc', 'wpcbi', 'pbp', 'ip', 'iwiw', 'biq', 'qwiw', 'xbw', 'xwpwwi', 'b', 'wi', 'x', 'ibbp', 'iw', 'ppip', 'jbpp', 'bbbi', 'wjip', 'bi', 'ipbb', 'wii', 'wiv', 'bi', 'iww', 'bbpp', 'w', 'wiii', 'h', 'i', 'piwp', 'bp', 'p', 'w', 'i', 'iww', 'iii', 'bpwpw', 'ibbibp', 'v', 'bp', 'wpi', 'wiip', 'bwb', 'ii', 'iiwcwpi', 'ppb', 'i', 'w', 'phb', 'i', 'bb', 'pwpbw', 'wbwp', 'awb', 'pppi', 'wiw', 'iib', ],
['jp', 'bpp', 'pii', 'pb', 'bbc', 'pqb', 'ib', 'i', 'pb', 'cpp', 'ppip', 'p', 'ipw', 'iq', 'ibiwi', 'b', 'i', 'p', 'pbpipp', 'w', 'iiip', 'bwiiiwp', 'wiiib', 'bwi', 'xp', 'w', 'i', 'pjwb', 'bbpw', 'ji', 'bibbip', 'wbxi', 'x', 'pbb', 'q', 'bbpi', 'xppcw', 'bc', 'wwp', 'bipbw', 'i', 'b', 'w', 'iwi', 'jpw', 'bpwip', 'pw', 'pqw', 'ocpw', 'w', 'p', 'pp', 'iqpp', 'w', 'ijjw', 'ic', 'pb', 'b', 'bw', 'bb', 'ixpp', 'iw', 'b', 'w', 'w', 'jih', 'ppi', 'bbp', 'pww', 'i', 'wppw', 'pwbi', 'ajw', 'b', 'wpiwp', 'iii', 'b', 'pip', 'wpw', 'biw', 'pixw', 'i', 'ppwbi', 'bwciw', 'p', ],
['cbb', 'pp', 'bw', 'wi', 'pwiww', 'wj', 'ppci', 'ibwb', 'bbbw', 'xppi', 'co', 'wiw', 'bi', 'b', 'iwwwb', 'b', 'v', 'ibip', 'wbw', 'w', 'pwi', 'pbiwi', 'jp', 'ipp', 'bwwb', 'ipbb', 'wq', 'pwqb', 'ibiwi', 'ii', 'wwc', 'qiw', 'bpw', 'bwww', 'c', 'pi', 'w', 'pwb', 'ipb', 'qppp', 'p', 'ibb', 'wcjpi', 'pipb', 'bpbi', 'wi', 'pxp', 'ib', 'pwi', 'bbwc', 'b', 'wwbbbbii', 'p', 'bww', 'iww', 'biip', 'biip', 'bibp', 'q', 'bqp', 'wbw', 'jbq', 'j', 'bbww', 'wxi', 'pb', 'b', 'bwib', 'bww', 'qiwbi', 'wpii', 'bwb', 'wpbw', 'ppb', 'piii', 'iii', 'ijw', 'pwi', 'bb', 'b', 'b', 'p', ],
['qc', 'cqq', 'qxc', 'xqxcj', 'ccj', 'jqxx', 'qcx', 'jhbhc', 'qcx', 'jqjxxc', 'qcj', 'c', 'qxjx', 'jx', 'qqqxcj', 'cj', 'cxqj', 'xjx', 'xj', 'qj', 'xxjc', 'jjq', 'x', 'jccq', 'xq', 'xxjxcqq', 'cc', 'jqqqc', 'ccx', 'xc', 'iocc', 'x', 'ch', 'jxcqc', 'a', 'ccx', 'q', 'jcjjj', 'cxxcj', 'qjq', 'jjccqw', 'xp', 'jq', 'xcjc', 'xq', 'jjjcqx', 'jq', 'cc', 'jq', 'xv', 'jc', 'x', 'c', 'pxxx', 'jjxjc', 'cjjxx', 'jxj', 'jxj', 'v', 'j', 'c', 'j', 'cc', 'xc', 'q', 'q', 'qx', 'cq', 'cjc', 'qcx', 'vc', 'qjxcx', 'xqjx', 'jjcqo', 'jq', 'cxc', 'jjx', 'qc', 'jq', 'ccjj', 'cj', 'qix', 'cvcj', ],
['hxxqc', 'cj', 'qcx', 'jxq', 'q', 'jj', 'bxc', 'jcjx', 'vqj', 'q', 'c', 'ijqc', 'jqcj', 'p', 'qq', 'xo', 'qq', 'cqjq', 'c', 'cq', 'xcjj', 'cqj', 'c', 'jqjcc', 'qx', 'xjcxjq', 'hj', 'cqo', 'jqcjj', 'jjcxijq', 'qxcj', 'c', 'cxqqc', 'xicjq', 'jjxxj', 'q', 'xcj', 'xxqqx', 'qajx', 'jpo', 'q', 'cjixc', 'cc', 'q', 'jjcjh', 'aqcj', 'qqvxj', 'xxx', 'cjq', 'q', 'qcqj', 'cxx', 'jqxj', 'c', 'jx', 'qccqcj', 'pxjq', 'cj', 'jc', 'vh', 'qx', 'jqxxbj', 'awx', 'xqj', 'x', 'x', 'bcqc', 'qxcqjj', 'jxh', 'cc', 'cjjc', 'cxc', 'xc', 'q', 'jcjcj', 'j', 'xjq', 'jxcc', 'qxqx', 'j', 'axj', 'qxcqj', 'qxq', ],
['jccjq', 'jjqx', 'jjxq', 'qcqqx', 'jb', 'ccq', 'xqc', 'cxx', 'jxqq', 'qpq', 'q', 'c', 'cjjwc', 'jjqqxqh', 'jxc', 'wj', 'cjjj', 'jq', 'cq', 'cq', 'jjxx', 'cj', 'qcxqq', 'qxx', 'xq', 'qx', 'wp', 'qj', 'q', 'xjqx', 'c', 'jchq', 'c', 'xcqx', 'x', 'xqqq', 'xjq', 'qxjxjqcx', 'v', 'cx', 'qx', 'cx', 'qjj', 'qcqvjj', 'baxc', 'qj', 'j', 'cq', 'xqjj', 'jxxjqcq', 'cq', 'xxq', 'xx', 'hvjbqjc', 'cjjxoq', 'x', 'xjj', 'h', 'cqx', 'c', 'qqc', 'j', 'xcxq', 'q', 'xch', 'c', 'x', 'jjx', 'qqj', 'x', 'qxqj', 'xqv', 'c', 'jj', 'j', 'jx', 'xjj', 'chqx', 'qjjqq', 'qc', 'pjqq', 'jxx', 'c', 'iq', 'cq', 'hx', 'c', 'qqxc', 'cqqjj', ],
['cqxc', 'cx', 'bxqj', 'ccqjcjc', 'qcqj', 'ixj', 'qcq', 'cq', 'cqc', 'jcc', 'xb', 'cxqq', 'qx', 'qq', 'jx', 'qjjq', 'xj', 'cjqb', 'bq', 'qcx', 'p', 'qcqx', 'bqc', 'xqx', 'jjqj', 'cqjcj', 'ocj', 'xjxjjqx', 'cxq', 'xj', 'q', 'jjxcc', 'cxqq', 'jj', 'xqq', 'xj', 'caqjc', 'qcjjj', 'jc', 'xqxq', 'xcj', 'qccc', 'xx', 'cx', 'qxjjxcj', 'qcx', 'a', 'pq', 'jc', 'xjhjjx', 'qc', 'qc', 'c', 'vjx', 'vqqxx', 'xxc', 'cqoc', 'xp', 'qxqq', 'xciivjc', 'ccq', 'jox', 'xcxqc', 'ix', 'cv', 'jq', 'cq', 'qj', 'cccox', 'qcx', 'wjx', 'qjxxq', 'vxh', 'vq', 'cxq', 'xc', 'xqq', 'qx', 'q', 'xc', 'xcx', 'xcqq', 'cqxc', 'ccwjj', 'cx', 'qcqcc', 'x', ],
['xqj', 'cqi', 'q', 'qqvxq', 'xjcj', 'ccq', 'x', 'qp', 'xqcqjvc', 'xcxi', 'xqbcq', 'hjq', 'jjc', 'jjqqxv', 'qxjjq', 'xjw', 'wx', 'xc', 'xcqc', 'xq', 'cw', 'j', 'qjcc', 'o', 'qaqjj', 'cc', 'xo', 'x', 'qcxpj', 'qxcx', 'xi', 'qcjxxo', 'b', 'ccj', 'qj', 'xxjxq', 'jqc', 'c', 'qxcq', 'jcx', 'qjjcq', 'xq', 'qcjb', 'xc', 'coj', 'qqx', 'jcx', 'jjx', 'cq', 'ijxxjxxq', 'qq', 'jqqj', 'j', 'j', 'qjxc', 'c', 'cccx', 'x', 'i', 'cxcqjq', 'c', 'jjqxc', 'wjccjq', 'jq', 'x', 'j', 'xqxcq', 'qqxxc', 'hcqqv', 'xxax', 'jo', 'cp', 'j', 'cq', 'xxj', 'qqxxcx', 'q', 'cj', 'qxj', 'xjc', 'xcjjqc', 'xqcv', 'cj', 'xc', 'vj', 'xjjx', 'qcq', 'x', 'qcxjc', 'xcx', ],
['jc', 'x', 'jjcccx', 'cciqj', 'qqcj', 'qqjqqx', 'ajjc', 'qxxx', 'jcjb', 'xq', 'j', 'xaqc', 'qqc', 'xw', 'jcc', 'jxcq', 'xqjcx', 'x', 'cq', 'jq', 'xxq', 'h', 'vc', 'jcj', 'qxjqx', 'xxx', 'cc', 'qjj', 'oq', 'jcxx', 'qx', 'qxvbj', 'qqc', 'qjjq', 'jjjo', 'cqc', 'xjj', 'xq', 'q', 'xqhc', 'cjcx', 'jqpjqc', 'ccjc', 'xjc', 'xxcx', 'qxqvjjxcq', 'x', 'q', 'qqjx', 'jcjc', 'cjcx', 'j', 'xc', 'jx', 'jqxqcq', 'jx', 'phx', 'qxjjxq', 'j', 'ccxq', 'qq', 'jx', 'xqc', 'cxc', 'qxjj', 'jq', 'cjxqqx', 'xx', 'xqqq', 'vjqqx', 'jq', 'q', 'jqxj', 'cxc', 'q', 'cc', 'jx', 'cx', 'cjc', 'jpxx', 'jc', 'jq', 'qjc', 'qqcq', 'cxj', 'ccqo', 'jxx', ],
['cqx', 'c', 'qxqcq', 'xjx', 'cqc', 'wc', 'xqcxx', 'qxxxq', 'qq', 'xxcjcqx', 'c', 'jx', 'q', 'xjoxx', 'j', 'qjjxj', 'jx', 'wx', 'pxc', 'cjxq', 'q', 'qqvxx', 'xx', 'xhcj', 'xja', 'qcqqq', 'xq', 'cqjxb', 'x', 'cx', 'cjqj', 'qpcocj', 'j', 'x', 'c', 'cjc', 'qqvx', 'qx', 'cx', 'op', 'j', 'jc', 'cxcxc', 'xqqx', 'q', 'oo', 'x', 'ccjp', 'j', 'qqqc', 'jc', 'jx', 'ccq', 'qqc', 'qx', 'xqq', 'cxqx', 'qcc', 'v', 'j', 'qc', 'x', 'cxj', 'qcqx', 'j', 'cxxc', 'xx', 'o', 'jhcqqcx', 'xcpx', 'qc', 'jxqjj', 'jcj', 'cj', 'cqcj', 'xjx', 'cqj', 'jxxxx', 'p', 'cq', 'cqvw', 'qjcq', ],
['qjcjxq', 'qj', 'qxj', 'qcq', 'xjxq', 'jx', 'cxcj', 'jj', 'c', 'qqx', 'xxxcxxc', 'xqhc', 'xjq', 'qbjq', 'qccc', 'cjcx', 'q', 'ccqjxp', 'q', 'xjcc', 'x', 'xjcxjj', 'qx', 'qc', 'cx', 'jqxj', 'x', 'cx', 'cjxc', 'qhqq', 'jqxx', 'cjqqq', 'ji', 'vqiqc', 'jqc', 'cjq', 'o', 'jjc', 'qjxc', 'qcq', 'q', 'bxjjjo', 'qc', 'w', 'cicxjxj', 'qxa', 'c', 'oxbx', 'xx', 'jcc', 'jx', 'qx', 'vjb', 'xcc', 'qcq', 'xcc', 'jj', 'jcjc', 'x', 'ccjj', 'cqqq', 'ccjax', 'xjxaq', 'jx', 'c', 'qjxxq', 'cjcjxxqjq', 'cxjqc', 'x', 'qqoxq', 'qjoc', 'xj', 'xjca', 'qjjj', 'jxj', 'jccx', 'c', 'c', 'x', 'qqxqx', 'x', 'ccx', 'c', 'jjcxj', 'jjqqj', 'p', 'j', 'jjqc', 'cj', ],
['xqj', 'j', 'xjjj', 'cx', 'xqxj', 'j', 'xqqcx', 'qji', 'qjjb', 'xq', 'xc', 'vxqx', 'jccx', 'jc', 'qoq', 'x', 'jcx', 'x', 'j', 'qxcci', 'xxqjq', 'qq', 'xq', 'qwjxc', 'qx', 'hqq', 'j', 'hxqja', 'qx', 'xj', 'jjjjxq', 'jxxc', 'qqcqj', 'cc', 'x', 'xx', 'cwj', 'jjjxx', 'xjcjx', 'cwq', 'ccjq', 'q', 'jxx', 'xxc', 'ccj', 'qqcq', 'xxjx', 'xjxcc', 'jx', 'qjch', 'oh', 'qqcj', 'jxqcc', 'oj', 'xqqj', 'xqq', 'xx', 'jqqj', 'jj', 'x', 'cqcq', 'xqc', 'jcxxj', 'cjjjx', 'jqwqj', 'cc', 'cx', 'xcc', 'pqx', 'hx', 'xc', 'xjqx', 'cqjx', 'c', 'jqq', 'j', 'cwxq', 'xhc', 'ccxxc', 'xxx', 'wc', 'bj', 'cqxcc', 'qc', 'x', 'aj', 'xc', ],
['xcq', 'cx', 'cccjj', 'cjj', 'xj', 'q', 'qx', 'j', 'cqq', 'xcqjqj', 'jcx', 'x', 'jjq', 'xx', 'xccc', 'qpqcq', 'qj', 'cq', 'jxcqx', 'o', 'qhc', 'cc', 'cc', 'cqjxcc', 'cic', 'axq', 'xqcxjj', 'q', 'ccqjc', 'x', 'xw', 'qqjxccq', 'qqj', 'xqq', 'xw', 'c', 'cc', 'xjj', 'jqjj', 'qq', 'jc', 'jcjjo', 'jc', 'x', 'jxq', 'xjc', 'jvcx', 'q', 'wcq', 'qxx', 'oxjc', 'boxpa', 'jxcx', 'p', 'qj', 'q', 'qxc', 'qj', 'qqxjj', 'xjjx', 'ccq', 'qj', 'j', 'cjcj', 'q', 'jj', 'cpcc', 'xqjq', 'cxq', 'x', 'jc', 'ccqb', 'jqx', 'xjqj', 'c', 'vx', 'bc', 'qxcxc', 'jjj', 'cwxxc', 'xqjxqq', 'qqq', 'j', 'xq', 'qccq', ],
['jjcc', 'vcqjqq', 'xjcxq', 'j', 'jcx', 'jcx', 'w', 'qjc', 'j', 'q', 'cc', 'cc', 'jqc', 'cxqjjc', 'xc', 'cxx', 'cc', 'pcxjxj', 'qq', 'q', 'qx', 'wxcx', 'cxxj', 'ch', 'qjxq', 'jcxj', 'xqhjxxq', 'ic', 'qwx', 'jq', 'jxxx', 'jc', 'xjx', 'x', 'xjc', 'vjq', 'jjjx', 'xqq', 'c', 'cj', 'qc', 'xqcxj', 'cwb', 'j', 'xqjc', 'cqqj', 'jjj', 'qxx', 'b', 'qj', 'wjjq', 'q', 'qj', 'cqcbc', 'xjqqj', 'p', 'cxcxj', 'c', 'cqcx', 'x', 'jj', 'jc', 'xxx', 'jjq', 'bpc', 'cq', 'v', 'jcc', 'jj', 'p', 'cj', 'jcxq', 'xxxx', 'cqxqaq', 'q', 'ccvjxj', 'x', 'q', 'ccj', 'j', 'xqx', 'jqc', 'jqxc', 'q', 'ac', 'cx', 'xj', 'xc', 'j', ],
['cq', 'qcxc', 'jcjj', 'cj', 'x', 'ccc', 'j', 'q', 'ccjc', 'a', 'jjc', 'cxj', 'jqx', 'cjqjxvx', 'q', 'cqqj', 'qjic', 'qcc', 'q', 'qqqc', 'ccq', 'qqjc', 'qcxc', 'x', 'q', 'cjxj', 'jjcc', 'xxqq', 'x', 'ajxc', 'jq', 'cjx', 'xp', 'qc', 'cj', 'cpj', 'qj', 'xx', 'xvq', 'xc', 'cjjq', 'j', 'xcj', 'cxc', 'c', 'qqxxc', 'jqbj', 'chcc', 'x', 'qxpq', 'xb', 'cqcjq', 'q', 'j', 'jxca', 'jqj', 'cxcc', 'aqh', 'cj', 'cx', 'cvq', 'xqxc', 'jcq', 'x', 'jx', 'p', 'ccxjj', 'qjb', 'xj', 'bjcx', 'qj', 'cx', 'jjq', 'cccjcx', 'cj', 'cx', 'qc', 'xxq', 'q', 'c', 'ccqx', 'x', 'qqxjjj', 'xoc', 'xcjj', 'caq', 'vco', 'qqxj', ],
['qqj', 'qjcx', 'v', 'jcxxjj', 'qqj', 'jq', 'c', 'qx', 'pxxx', 'jjxc', 'jq', 'oxbx', 'qjj', 'xhj', 'jqj', 'xxc', 'xcqxq', 'x', 'cic', 'xjjjjx', 'jx', 'c', 'q', 'qq', 'cc', 'vcqiq', 'xq', 'qcq', 'jqxxj', 'ccq', 'wjqi', 'xqq', 'qxxqc', 'hj', 'qqc', 'xxq', 'cq', 'cqjc', 'jhc', 'cjqxoc', 'vcc', 'ocxcx', 'vqq', 'wq', 'v', 'cjjx', 'q', 'qqqx', 'jjqc', 'hw', 'qq', 'xqcj', 'qcqxjqh', 'cqjj', 'ccjc', 'j', 'jjcj', 'cxp', 'q', 'jj', 'qjc', 'xpc', 'cqc', 'c', 'qjjcb', 'ic', 'cjhci', 'jqjx', 'qjqqcqijx', 'qx', 'x', 'jqqjj', 'xjc', 'q', 'xcjjjcq', 'xcjx', 'cjq', 'cjjqjjjj', 'cq', 'i', 'jx', 'jxjqix', ],
['bixjj', 'xxv', 'j', 'jxjcj', 'jjx', 'q', 'x', 'bc', 'qcq', 'cq', 'xj', 'j', 'xacjh', 'q', 'q', 'qjj', 'x', 'qv', 'j', 'qjq', 'cbqcx', 'q', 'j', 'c', 'jx', 'jxqxq', 'c', 'jj', 'qjqq', 'cqj', 'jcxq', 'jx', 'qqxj', 'cq', 'qx', 'acixqa', 'xxc', 'jq', 'jj', 'q', 'xx', 'x', 'xxcc', 'jx', 'c', 'qjqxqxj', 'qc', 'cqqj', 'cqjqjjxc', 'cqq', 'pxqh', 'qxxjc', 'xjjc', 'jxjxj', 'jqc', 'cjx', 'jx', 'xxqj', 'q', 'cq', 'iojxj', 'c', 'jj', 'qcj', 'qq', 'jq', 'xq', 'xxj', 'jqjqiq', 'cjcc', 'qo', 'cxqxc', 'jc', 'bjjx', 'qxjc', 'q', 'xq', 'c', 'xjx', 'hax', 'xjxqxc', 'jqx', 'cqp', 'j', 'xqcv', 'q', 'j', 'jc', 'vc', ],
['qqc', 'xq', 'ccqj', 'c', 'px', 'qw', 'cq', 'hjjqj', 'jp', 'cqj', 'q', 'xijq', 'xx', 'cq', 'jjxx', 'xxjjcc', 'pc', 'qjj', 'pjx', 'cxc', 'ccq', 'wc', 'xjjc', 'jxj', 'qq', 'j', 'x', 'qx', 'qvc', 'q', 'qxj', 'qjc', 'qjj', 'qqj', 'cjjqxxc', 'qwqx', 'wjc', 'wjcaxc', 'jcbx', 'pj', 'c', 'j', 'ccjx', 'xc', 'xqi', 'xxp', 'q', 'xj', 'c', 'c', 'qchpx', 'xcc', 'cc', 'xj', 'jx', 'xpx', 'cxj', 'xqqj', 'jjq', 'j', 'jx', 'pc', 'x', 'xjh', 'jc', 'x', 'cj', 'cjx', 'xqjcjx', 'qcco', 'q', 'xqxj', 'xqxq', 'jqw', 'jcqqx', 'xxqxcx', 'ccp', 'qxjxxh', 'o', 'qjj', 'xqx', 'hccq', 'q', 'jqqqc', 'cjc', ],
['xj', 'jcc', 'cq', 'x', 'jjxx', 'x', 'xqjx', 'xcjq', 'bxjc', 'cqccqxq', 'c', 'ccqq', 'jqj', 'oc', 'xcq', 'c', 'q', 'c', 'p', 'jqqccj', 'ph', 'cqq', 'c', 'qx', 'xcj', 'cq', 'c', 'xjqj', 'jq', 'qqcx', 'cj', 'jqxqcc', 'qxjiq', 'j', 'jjjx', 'x', 'cq', 'jc', 'c', 'c', 'cqc', 'xqq', 'qqxxx', 'qxqc', 'qxpq', 'jjc', 'p', 'jw', 'q', 'jwcc', 'bxx', 'qc', 'xc', 'j', 'j', 'xj', 'cx', 'cxqx', 'qcxqb', 'qc', 'c', 'q', 'cbxc', 'ixoxqcx', 'jqxqj', 'qqjj', 'x', 'q', 'qjqjcv', 'jjx', 'qjx', 'p', 'qxjoqc', 'jcjq', 'aajji', 'cq', 'jc', 'jq', 'hcxj', 'qqxjc', 'j', 'cjx', 'qcc', 'cpx', 'cjqjj', 'ccc', 'ij', 'q', 'qxxp', ],
['xx', 'bcj', 'qxvbjjx', 'axj', 'jxqc', 'xb', 'qxc', 'qjj', 'xc', 'wcx', 'oxj', 'j', 'qcxcx', 'ii', 'jqcqjc', 'q', 'qxj', 'x', 'jj', 'jxq', 'cjqjqj', 'qx', 'xcxxc', 'xxpq', 'cjjjxx', 'qq', 'jhj', 'cqq', 'xxccq', 'xxbhjp', 'qccqcqxq', 'j', 'jijc', 'cc', 'cxx', 'jxxwxc', 'xj', 'cc', 'jccc', 'xc', 'jx', 'q', 'qxjjj', 'qxj', 'jc', 'j', 'qc', 'ccjx', 'cj', 'cxjc', 'jw', 'xq', 'bc', 'cxq', 'xc', 'x', 'jxj', 'x', 'joc', 'qc', 'jvx', 'i', 'jcb', 'c', 'xq', 'xjxjc', 'cjx', 'hx', 'qxxcjx', 'qwj', 'c', 'bcc', 'jqp', 'cjxx', 'ccx', 'qx', 'qpx', 'qcjj', 'xjxq', 'xc', 'qq', 'cqv', 'qxj', 'jhx', 'vjj', 'cxc', 'xj', ],
['cjqcj', 'qqic', 'qqqc', 'x', 'xj', 'c', 'cqx', 'cqj', 'j', 'x', 'cxc', 'ccc', 'xqcj', 'j', 'xhhq', 'x', 'oqccqj', 'cijj', 'cqjpq', 'j', 'cc', 'x', 'c', 'qj', 'x', 'xqj', 'qaqq', 'q', 'jj', 'qj', 'cqx', 'qcjq', 'cqx', 'jj', 'qcj', 'wx', 'jcxc', 'ccvj', 'xjqjc', 'b', 'hcc', 'jjqj', 'jqqc', 'q', 'qqp', 'jjx', 'hxjj', 'xq', 'ccx', 'cq', 'jc', 'qqcq', 'q', 'bcxq', 'cxqqq', 'cxjj', 'qjx', 'x', 'c', 'xc', 'qcj', 'cqjcj', 'w', 'qj', 'jcj', 'c', 'q', 'xqj', 'j', 'qqxxhx', 'q', 'x', 'jjjc', 'xcc', 'cq', 'cjj', 'xxxj', 'jjjxw', 'cjjc', 'xqj', 'jq', 'qqxq', 'cqx', 'jx', 'qcj', 'jjx', ],
['qq', 'xcj', 'jxhw', 'qcq', 'jhqxw', 'xxq', 'q', 'xxc', 'qqcc', 'xjqjx', 'q', 'cx', 'qjjccc', 'p', 'qxj', 'jxpc', 'c', 'pxx', 'jjcx', 'ox', 'jjx', 'xq', 'cj', 'j', 'q', 'hxw', 'qbqccj', 'i', 'a', 'c', 'qj', 'jxxj', 'cjc', 'xx', 'qj', 'qc', 'cxqx', 'jp', 'iq', 'cjj', 'jqqjc', 'cbv', 'jjqqc', 'xq', 'xj', 'qcjq', 'xjq', 'x', 'cajxj', 'jjcx', 'jjqq', 'bxq', 'bj', 'xcqq', 'qqq', 'xcxx', 'w', 'xq', 'x', 'cx', 'q', 'jjxqc', 'acjj', 'qjqxqqq', 'xjj', 'cx', 'qjcqq', 'jjqcc', 'j', 'wq', 'jj', 'q', 'jx', 'qxj', 'jjjc', 'qcjc', ],
['jvvxc', 'qqw', 'qqqj', 'c', 'jjx', 'qxj', 'j', 'jcq', 'vc', 'jca', 'cxxcj', 'cj', 'jcjc', 'ccq', 'x', 'q', 'jp', 'jq', 'cjxc', 'xjc', 'xx', 'jqx', 'qqx', 'j', 'xjcjx', 'xx', 'qqjp', 'q', 'axqqc', 'xxcx', 'jqxc', 'jxq', 'cq', 'xjc', 'jqjj', 'cxx', 'jx', 'xx', 'x', 'w', 'q', 'ccx', 'ji', 'qc', 'coc', 'jj', 'jjx', 'qjcj', 'xxqc', 'jxjj', 'qj', 'hxxj', 'xq', 'jqc', 'xjj', 'jj', 'jc', 'xqqjj', 'cxc', 'qqo', 'cxxwx', 'c', 'ch', 'j', 'cjxc', 'qxjx', 'q', 'qqc', 'jx', 'q', 'qxcx', 'xqx', 'qc', 'xqxx', 'jxqh', 'qxqjx', 'c', 'hj', 'jxcj', 'ojv', 'xxjc', 'cxhj', 'jjq', 'xjcx', 'xbj', ],
['xxc', 'c', 'xjxqx', 'xb', 'x', 'cq', 'x', 'qcxcj', 'jqqx', 'q', 'wxqxj', 'j', 'qcqx', 'xiq', 'jjxx', 'qc', 'xcxcx', 'jxqxj', 'cqc', 'cjc', 'c', 'bjcp', 'c', 'qaj', 'x', 'jxx', 'jjqjx', 'w', 'qcxx', 'ac', 'qj', 'x', 'qbcc', 'cqo', 'c', 'jqq', 'jcqqc', 'cjq', 'xcc', 'jj', 'jqq', 'jjjxqc', 'pq', 'cj', 'jcq', 'cqjja', 'vi', 'jo', 'j', 'cqc', 'xjqq', 'x', 'xqxcqj', 'c', 'qqc', 'xc', 'c', 'cqj', 'bq', 'jqqq', 'jjx', 'bx', 'jq', 'bxjc', 'xqc', 'qq', 'xqc', 'cxc', 'cxqq', 'xqcq', 'cxqwc', 'x', 'qcxq', 'cjb', 'qjxjj', 'xq', 'qxcjqxj', 'cc', 'jj', 'qxoxc', 'jcjxj', 'xcbcxx', 'xcj', 'jjq', 'jq', 'jjxjj', 'xj', 'ccq', 'xj', 'qx', 'xcqc', 'qxco', ],
['jq', 'jj', 'j', 'qx', 'cccxqj', 'wcxqx', 'qx', 'jjxx', 'jc', 'x', 'xi', 'cq', 'c', 'qxji', 'qxjcq', 'cxch', 'cx', 'jqcj', 'cqjq', 'c', 'xcxc', 'x', 'cc', 'jcqj', 'xq', 'cc', 'xxqc', 'xc', 'xoq', 'j', 'jcjc', 'jac', 'x', 'jxx', 'q', 'jc', 'qj', 'xj', 'xc', 'jqv', 'xcxq', 'q', 'qccx', 'xx', 'qjc', 'qc', 'qxc', 'cq', 'jjj', 'jjj', 'jq', 'ccxq', 'woq', 'xqc', 'xcc', 'cc', 'chq', 'jjq', 'xcx', 'cj', 'xcxj', 'qhp', 'qxqj', 'qx', 'xc', 'cxx', 'cjx', 'c', 'jh', 'ocjx', 'jpcjcxq', 'xxxq', 'jji', 'qcc', 'ja', 'q', 'ca', 'q', 'q', 'cjq', 'vjj', 'jq', ],
['qj', 'bxc', 'jqw', 'hcjx', 'c', 'qx', 'c', 'qcjq', 'xqxxaq', 'qxjjq', 'wc', 'xj', 'xj', 'qqj', 'jccj', 'cxcx', 'qc', 'pc', 'xccwc', 'j', 'xhx', 'jx', 'xj', 'jc', 'cc', 'cojx', 'jax', 'qjqjc', 'jx', 'cc', 'cq', 'jj', 'cqq', 'c', 'cjj', 'jc', 'cxc', 'xqjx', 'cxjqj', 'jjcj', 'jjcxxxj', 'c', 'jjqqqp', 'i', 'bcwj', 'xjbjh', 'jxj', 'xjcc', 'q', 'xojxxqcbx', 'j', 'xjq', 'j', 'xjx', 'xxjx', 'ccxb', 'qqxq', 'jjc', 'xx', 'xqq', 'xcjj', 'iq', 'qbqj', 'qx', 'cc', 'qx', 'jj', 'xcjc', 'j', 'x', 'xx', 'qq', 'xxjj', 'xjqj', 'jc', 'cqj', 'j', 'cjc', 'ccqq', 'qcjxj', 'cc', 'i', 'c', 'xx', ],
['q', 'jjic', 'pc', 'cc', 'qoxj', 'q', 'jjqj', 'oc', 'jqci', 'qxicq', 'xj', 'x', 'cxj', 'p', 'cqc', 'ixxcqj', 'qcq', 'jo', 'j', 'xx', 'jj', 'w', 'j', 'xxqx', 'ccj', 'jcc', 'qaccqcj', 'xx', 'jw', 'ccxc', 'xhj', 'j', 'q', 'cqc', 'caj', 'xj', 'qjxx', 'q', 'qccjqj', 'x', 'c', 'jv', 'c', 'xj', 'jj', 'xqx', 'qjq', 'jqcc', 'xc', 'cx', 'qx', 'vcccq', 'qqxcj', 'qcj', 'qq', 'j', 'v', 'c', 'jxo', 'cccc', 'qcccx', 'vcxx', 'x', 'jq', 'j', 'jjcx', 'cxc', 'jqj', 'xx', 'xwqccqj', 'jjcj', 'jcx', 'jc', 'jqxqj', 'xj', 'jjca', 'xq', 'x', 'cxxqqjqx', 'qx', 'x', 'jjq', 'jbqo', 'cj', 'cqx', 'jx', 'ccx', 'jcxq', 'p', ],
['c', 'jqj', 'xx', 'cc', 'qcqbq', 'xjjxcj', 'jxqq', 'jc', 'xqxjj', 'q', 'x', 'i', 'cxqjq', 'xq', 'xxq', 'wc', 'qqx', 'caqc', 'cq', 'qq', 'xxcc', 'cjcjj', 'jj', 'jqqqqjj', 'bj', 'jc', 'xxqc', 'cq', 'xq', 'xp', 'xxxjo', 'ccc', 'j', 'qxjj', 'qo', 'qxcc', 'cqqc', 'qx', 'c', 'qqc', 'cx', 'x', 'a', 'c', 'jqjiqx', 'cjqccjx', 'jj', 'qqx', 'qxc', 'xxjj', 'qjqxv', 'cv', 'xpp', 'xqxj', 'xjj', 'qqjcq', 'xc', 'xxqj', 'qij', 'aqw', 'jxx', 'jjj', 'cj', 'xxx', 'jjjcjx', 'cxcq', 'q', 'qjcxj', 'cqc', 'jqjq', 'qqv', 'qq', 'cjcq', 'jxxj', 'cqq', 'c', 'cpqjcq', 'xcqx', 'cq', 'v', 'qcj', 'qjj', 'q', 'cj', ],
['c', 'xc', 'b', 'cq', 'cj', 'jc', 'cq', 'j', 'wx', 'qccj', 'jqc', 'xj', 'q', 'q', 'qox', 'cxj', 'cxxc', 'hjj', 'pixjc', 'vjj', 'xxqcc', 'o', 'c', 'jjjqpx', 'qc', 'qxv', 'cqq', 'xcj', 'cx', 'qqqxq', 'xqjqq', 'xcqc', 'qc', 'j', 'ccx', 'qj', 'xcv', 'bxq', 'qx', 'j', 'c', 'qcxjcwj', 'ccj', 'xj', 'qcx', 'xc', 'cj', 'jcc', 'xqxqq', 'qjj', 'qqpjq', 'j', 'cx', 'qx', 'hjqqj', 'jcc', 'jj', 'qixcq', 'xjxx', 'x', 'xj', 'qxx', 'jcqqjx', 'xjcjjc', 'q', 'xqxc', 'jvvc', 'cq', 'xx', 'c', 'pj', 'cxjc', 'qjwq', 'j', 'qq', 'xj', 'xjc', 'q', 'jcq', 'cvq', 'qcc', 'x', 'q', 'xjjjxqx', ],
['j', 'pjqc', 'jwix', 'cjxj', 'xjq', 'xj', 'qjij', 'x', 'xj', 'xqjcj', 'v', 'x', 'jcxv', 'xxc', 'x', 'cx', 'cc', 'j', 'qj', 'v', 'x', 'xpcqqc', 'qjq', 'cxc', 'jj', 'xxq', 'jqq', 'xbx', 'qjc', 'xq', 'cxx', 'cq', 'jjjjqq', 'xqjjjq', 'jjx', 'xx', 'qc', 'jxjx', 'jvc', 'jqc', 'aqc', 'jcqq', 'jcx', 'ci', 'j', 'j', 'jxjqj', 'c', 'ojcx', 'xcc', 'xq', 'qjjc', 'qq', 'jcqx', 'j', 'xxxjqh', 'xocjjjojq', 'xax', 'jhwjax', 'qxjpj', 'qqx', 'xcxb', 'bqixvo', 'xjxjc', 'c', 'bc', 'q', 'x', 'xjj', 'qcj', 'jc', 'cqcbxc', 'cqc', 'qjp', 'xqxc', 'cqjqjq', 'qc', 'qxxx', 'ccwj', 'vhq', 'ijq', 'cqxcq', ],
['cq', 'jx', 'j', 'cqhx', 'x', 'hjjc', 'cqix', 'jx', 'xxqq', 'qcj', 'jq', 'xxcqxqc', 'cqx', 'qj', 'jjqqc', 'jcqxc', 'cx', 'xcjcj', 'cjb', 'xj', 'c', 'cxc', 'cj', 'xqqjcq', 'qcx', 'jjxccx', 'jc', 'qqj', 'xqjq', 'xcwiq', 'bo', 'qcq', 'jxcqx', 'jjccq', 'ap', 'x', 'cx', 'c', 'xv', 'cxccx', 'jqi', 'j', 'xq', 'q', 'x', 'cccc', 'wv', 'bq', 'x', 'qj', 'xjj', 'cq', 'cqcj', 'c', 'oiq', 'xqxjqx', 'cpqq', 'xjcj', 'j', 'a', 'jqxxc', 'jqqqcq', 'jxcx', 'q', 'xijjj', 'c', 'jqqq', 'cq', 'px', 'qqxj', 'x', 'jq', 'cq', 'cjj', 'cqcqx', 'jj', 'h', 'c', 'qw', 'qhjj', 'qjxjq', 'j', 'qxcx', 'qjcx', 'xxcj', 'ccw', 'jcw', 'qc', 'xq', ],
['hx', 'jqjqccx', 'cxxxc', 'c', 'qxj', 'c', 'xcxj', 'jjjq', 'jjqxvj', 'qxx', 'cj', 'coj', 'cch', 'cc', 'qc', 'xca', 'qv', 'jqj', 'jjjq', 'cqx', 'ccq', 'q', 'cq', 'c', 'q', 'q', 'qj', 'j', 'xxjq', 'bcjq', 'qxqq', 'jx', 'xcq', 'xc', 'jx', 'xqqq', 'xxq', 'qcqc', 'c', 'jx', 'jq', 'cjjci', 'cxjqcqcjj', 'j', 'jx', 'ccx', 'c', 'qqj', 'jijxjj', 'cjq', 'xjjc', 'j', 'jx', 'cjcxj', 'x', 'xcj', 'jqc', 'cxjx', 'cxx', 'j', 'vccc', 'pcjj', 'qcq', 'xxcj', 'j', 'jxbq', 'oxjo', 'cacc', 'jxxx', 'jcic', 'cqx', 'j', 'jqc', 'aqqc', 'j', 'jc', 'qqxx', 'j', 'qpx', 'q', 'qxxq', 'jq', 'jq', 'jjc', 'xqpcjc', ],
['jxqj', 'aqjqqxix', 'xjj', 'jqxcb', 'jhccx', 'cj', 'qj', 'jqxx', 'cx', 'qqjc', 'xq', 'jj', 'oa', 'cqcq', 'cqc', 'qwq', 'w', 'hxq', 'x', 'cxq', 'w', 'cjcqjc', 'ijxxjj', 'jc', 'x', 'j', 'hqaj', 'hx', 'xqx', 'jc', 'qcj', 'jji', 'wqc', 'xjj', 'ccjx', 'qcx', 'cjc', 'j', 'cx', 'j', 'cqqa', 'xv', 'xx', 'cj', 'xcx', 'xqx', 'qjqqc', 'xqq', 'xqj', 'axjx', 'cxjx', 'qxcacx', 'jj', 'cqjccx', 'q', 'jj', 'qqc', 'joc', 'xqcc', 'qqx', 'c', 'jqqj', 'qqhq', 'xqqxc', 'xjx', 'q', 'xqc', 'qxcx', 'ccxqj', 'jx', 'xcqj', 'cq', 'jxq', 'jx', 'jjx', 'cxq', 'vjq', 'qc', 'cocjja', 'jqqx', 'qqqq', 'q', 'xj', 'jh', 'j', ],
['c', 'cjj', 'xqx', 'ccqhq', 'cxqx', 'q', 'j', 'cjjqcc', 'c', 'jcqq', 'cxqj', 'xcx', 'jqqw', 'qhxj', 'hc', 'qjq', 'j', 'jc', 'x', 'qcc', 'c', 'xjxc', 'xxb', 'qq', 'xixqx', 'cjcqj', 'qxo', 'qxjc', 'jx', 'cj', 'qwj', 'jcqj', 'jxx', 'j', 'qqc', 'xxq', 'cjxjcb', 'c', 'jcq', 'j', 'xxqcq', 'xqq', 'cq', 'jqxx', 'oxpqcx', 'q', 'i', 'jjv', 'cjh', 'c', 'cc', 'iqjq', 'xjcj', 'ccjpqh', 'xj', 'jq', 'bjjc', 'xqjj', 'qq', 'xx', 'qx', 'qq', 'jc', 'qqvq', 'cc', 'cqjc', 'cqh', 'q', 'qccq', 'xhq', 'jj', 'x', 'xjx', 'pxq', 'qx', 'cj', 'cj', 'jv', 'q', 'qjxq', 'xcq', ],
['x', 'jcqq', 'jc', 'vx', 'jjcj', 'j', 'jc', 'jq', 'jq', 'qcq', 'jjj', 'hc', 'jj', 'qx', 'cqqj', 'qjqh', 'qjqcq', 'q', 'cx', 'xc', 'jbxj', 'jbx', 'cxqxq', 'oica', 'cj', 'jjj', 'j', 'ccq', 'jqj', 'ajcq', 'cq', 'qxj', 'h', 'qx', 'o', 'qxi', 'qj', 'ji', 'xc', 'xcxxj', 'cxxjxi', 'jj', 'j', 'ccv', 'cxqcc', 'x', 'jqjq', 'qx', 'xj', 'i', 'cj', 'jcc', 'xxq', 'cjxq', 'qbqcj', 'x', 'jxccj', 'cxxq', 'qvqj', 'jx', 'xx', 'jqjjq', 'cj', 'hcxc', 'cx', 'jajqxq', 'c', 'jxjqq', 'qc', 'bj', 'xcqq', 'cj', 'q', 'jjqqc', 'x', 'xacaw', 'cpq', 'x', 'qx', 'jq', 'qj', 'c', 'jc', 'xxcq', ],
['o', 'jq', 'jjc', 'xj', 'xxjvqc', 'cjc', 'xqpx', 'c', 'qjjqq', 'xxq', 'jcxcjj', 'jc', 'c', 'j', 'xjqj', 'jq', 'cwj', 'ccqx', 'qj', 'jcjq', 'qqjj', 'j', 'cj', 'xqjx', 'jx', 'qjcqax', 'qvcxxx', 'ccx', 'accq', 'ajq', 'aqxxjxx', 'qjv', 'qqc', 'jqqj', 'xjxbx', 'qjxj', 'cq', 'xjjp', 'vccjjc', 'cjc', 'qjq', 'jqc', 'qcjh', 'c', 'xxj', 'qxx', 'ccq', 'xjj', 'xjcx', 'c', 'co', 'pjqj', 'xjqc', 'xjxc', 'q', 'cjq', 'xqjcjcxx', 'xqxq', 'jx', 'cx', 'q', 'xxj', 'qwcjb', 'xq', 'jwpc', 'qc', 'x', 'jxxcc', 'xjqxcx', 'aj', 'cx', 'xqcjjc', 'cqxa', 'qxcx', 'jqx', 'q', 'ccxcxq', 'xqc', 'xjxc', 'chx', 'xjjxq', 'ixjjc', ],
['ax', 'cj', 'j', 'jw', 'qx', 'jx', 'c', 'wx', 'jjwq', 'pqx', 'c', 'jccxjxx', 'xxxq', 'q', 'cax', 'j', 'cqc', 'xxccc', 'q', 'c', 'q', 'xx', 'xqxq', 'x', 'cjcq', 'xc', 'xxj', 'cbcx', 'chccjqq', 'xxcj', 'jcj', 'xjx', 'cjqc', 'cxbqx', 'jj', 'ji', 'jqx', 'hqxoq', 'qjj', 'qcjj', 'j', 'cqx', 'xcx', 'wxqx', 'jjjhc', 'jx', 'xj', 'cxcj', 'qwjxcq', 'ixcqj', 'q', 'c', 'jxqx', 'jcv', 'cxcqx', 'ojc', 'xjx', 'xq', 'jx', 'jxq', 'qcj', 'jxx', 'jvj', 'xqcjxq', 'cq', 'qjjccj', 'cj', 'cx', 'jxcxj', 'q', 'xccq', 'ccpx', 'c', 'j', 'jq', 'iq', 'qoqi', 'x', ],
['cc', 'ax', 'cj', 'xcjxj', 'qqj', 'a', 'xcqjq', 'bqqc', 'jcj', 'xcx', 'jxq', 'hxxq', 'xqj', 'cjqx', 'cbo', 'x', 'xc', 'jqqx', 'jx', 'j', 'aj', 'cxxqx', 'x', 'cxc', 'j', 'qqa', 'cqqpj', 'xq', 'j', 'cxxc', 'ccx', 'cxqjac', 'c', 'jc', 'xx', 'cpxxj', 'vqjx', 'jjcq', 'j', 'jhjq', 'q', 'c', 'ivxx', 'hc', 'xjx', 'cjj', 'hxc', 'c', 'bxj', 'xhx', 'x', 'qj', 'xxiqc', 'c', 'x', 'jc', 'qxjj', 'qxvjqwa', 'j', 'jj', 'jx', 'ix', 'j', 'c', 'xocqqj', 'cj', 'cavj', 'xc', 'jcj', 'wjj', 'cxjjx', 'c', 'c', 'jj', 'c', 'cxx', 'jj', 'jqxcccx', 'qca', 'o', 'i', 'qcjc', 'vx', 'px', 'jxcj', 'xqj', 'jcqjjx', 'xjx', 'ojjxqq', 'xjx', ],
['q', 'cxq', 'v', 'xxjxq', 'jxq', 'qxx', 'xqqxcx', 'qjq', 'qjqc', 'jjqc', 'c', 'jxc', 'cx', 'xj', 'qq', 'xq', 'xxj', 'qc', 'jqq', 'jqqqc', 'qj', 'c', 'oj', 'jxj', 'j', 'xxxqq', 'cxj', 'hijx', 'xx', 'cc', 'qpcw', 'qx', 'i', 'qxccc', 'ccxjx', 'j', 'c', 'q', 'qxx', 'cxxx', 'q', 'jq', 'c', 'cqcw', 'cx', 'jq', 'jpj', 'j', 'xpx', 'jcqj', 'qj', 'cccx', 'jjj', 'qjcj', 'xqq', 'j', 'jj', 'xjbcj', 'hp', 'qbjc', 'xacq', 'cxj', 'jaqjj', 'x', 'qq', 'c', 'x', 'qj', 'cc', 'jqjq', 'c', 'q', 'x', 'xq', 'pcqjq', 'cqxq', 'ccq', 'qhqcq', 'xqqcc', 'qj', 'xhjcq', 'xj', 'wqqqxx', 'jxcbxxj', 'ccaqj', 'jja', 'qxci', 'c', 'qjx', 'xcxqx', 'c', 'cqj', ],
['xxj', 'qqqq', 'qjcw', 'qjq', 'jv', 'q', 'qjq', 'jq', 'jjj', 'qaxj', 'xj', 'j', 'x', 'jxxx', 'qcjj', 'cx', 'c', 'qccjj', 'ccc', 'w', 'q', 'xcc', 'xxhj', 'pqx', 'xo', 'jc', 'c', 'jjvccx', 'cqjxc', 'jx', 'q', 'xcx', 'xj', 'cqb', 'j', 'cj', 'xqi', 'pqccv', 'jx', 'c', 'cjq', 'xqjcx', 'xcxc', 'qxcj', 'jqxc', 'xxj', 'jx', 'qqxjq', 'xcqcxj', 'cqqv', 'qx', 'q', 'qjhcwq', 'jxxj', 'qqco', 'cq', 'qx', 'qc', 'xx', 'jxqo', 'ixcj', 'cx', 'wx', 'jxp', 'c', 'b', 'qcp', 'jc', 'q', 'xx', 'cqx', 'xx', 'cojxjq', 'c', 'jcjccxw', 'jq', 'jj', 'ocqj', 'cqjjq', 'ccqb', 'j', 'pxc', 'qq', 'cpccx', 'wx', 'xxjc', 'ocx', 'cj', 'x', ],
['qc', 'q', 'cqj', 'cx', 'xj', 'qjqqqq', 'cjq', 'qxjjj', 'qj', 'cqqcqxc', 'cjwj', 'jq', 'qqx', 'xx', 'hcq', 'x', 'qxjx', 'jj', 'qwxqa', 'j', 'xqxh', 'xcq', 'j', 'cjq', 'jobj', 'xja', 'ccp', 'jqcqcxc', 'j', 'xqx', 'jj', 'cqqjx', 'ccqo', 'jxj', 'ccja', 'q', 'qq', 'xjxcqjj', 'qhxjc', 'xjqc', 'cjq', 'qq', 'cccxxjq', 'qqjq', 'iqxq', 'j', 'jjxx', 'xxqj', 'cxv', 'jic', 'xqc', 'xjj', 'xcqx', 'qxx', 'qxj', 'jq', 'xqxx', 'jx', 'ccjqq', 'jc', 'qj', 'cxj', 'qjjc', 'j', 'xjqpjx', 'j', 'qcj', 'c', 'jxx', 'x', 'cxc', 'ajx', 'qqq', 'ccbqc', 'qj', 'xcj', 'c', 'cci', 'c', 'xj', 'jpc', 'qjjxcp', 'cxqp', 'xxqqc', 'ic', 'xj', 'j', 'qxjqxq', 'cxjccq', ],
['jx', 'xxjj', 'jqqc', 'jx', 'x', 'jx', 'jc', 'c', 'j', 'qwc', 'qjcjj', 'xqpb', 'xcq', 'cjwc', 'hcjxq', 'xxwjq', 'qxpjcjc', 'q', 'qxa', 'h', 'bcx', 'qxqc', 'qqxq', 'qco', 'xqx', 'bq', 'hxccjxj', 'jq', 'jqcjx', 'ccicc', 'wj', 'ccxjc', 'j', 'jcjxj', 'xjj', 'cxcj', 'vccxqx', 'xhj', 'qj', 'xjx', 'xxc', 'cb', 'jb', 'j', 'qx', 'qijcjcqc', 'qj', 'x', 'icq', 'jq', 'jxqqjqq', 'qx', 'xcjc', 'x', 'xxxj', 'xcxxc', 'qcjcqq', 'xc', 'jq', 'cxccpq', 'ccc', 'qjc', 'xqcj', 'waj', 'cj', 'ox', 'qx', 'ccjbq', 'qcc', 'cj', 'pxx', 'cqcxj', 'jq', 'cjjcqx', 'cx', 'qjqcjpc', 'jvxc', 'ccc', 'x', 'bqjqq', 'cqj', 'jax', 'pxjj', 'cxqc', 'c', 'o', ],
['cx', 'jv', 'cj', 'xxqq', 'qaqc', 'xx', 'jhj', 'p', 'jjj', 'qqc', 'qxcqxw', 'qjx', 'cjphc', 'c', 'jqqqcq', 'jcq', 'xccc', 'jjjjxj', 'qq', 'cccx', 'q', 'x', 'jqjj', 'jq', 'q', 'xqo', 'j', 'cq', 'qi', 'xxcx', 'xxcq', 'qqx', 'xqx', 'xxx', 'jxjj', 'qx', 'xjx', 'xwxjj', 'hcq', 'iqjq', 'x', 'qjc', 'jqxq', 'jqx', 'xja', 'cccjcc', 'qc', 'jcja', 'jjcc', 'qcqxx', 'jjxoq', 'qqq', 'xjcx', 'qjq', 'jxqjjx', 'a', 'cxcx', 'qjqx', 'qvjj', 'cj', 'xa', 'xqcqi', 'qq', 'jjqc', 'xj', 'vjj', 'xjc', 'x', 'cq', 'h', 'cqqcq', 'jx', 'jj', 'qqq', 'icjc', 'q', 'xxq', 'cv', 'xxx', 'xxqxq', 'qqi', 'ajj', 'qjx', 'cj', ],
['qqqc', 'vjah', 'qqxj', 'jqc', 'q', 'bxq', 'cjxxjx', 'jjj', 'q', 'c', 'jx', 'jcc', 'jxx', 'jxcxjjjq', 'q', 'cjxw', 'xc', 'cjjx', 'xq', 'jca', 'cxqx', 'ccq', 'xjqc', 'bxxj', 'q', 'xxjc', 'j', 'jj', 'jqj', 'xcqcc', 'cqjjc', 'oxjoxj', 'qjqxo', 'cxc', 'jj', 'c', 'qx', 'jcjqcb', 'xx', 'cco', 'qjqqx', 'cqc', 'jj', 'jxqxq', 'jjj', 'jj', 'jjxv', 'qj', 'vqxjq', 'j', 'qbx', 'j', 'jxcxq', 'x', 'x', 'qqqq', 'xc', 'oqcc', 'cxq', 'qqj', 'cijc', 'xjxx', 'jxjj', 'x', 'x', 'hqj', 'j', 'qqqw', 'qvccbj', 'jqxqc', 'x', 'cxj', 'wqc', 'jv', 'jx', 'jc', 'xxq', 'xcc', 'jq', 'qc', 'cjcq', 'q', 'j', 'hpcc', 'q', 'cqq', 'qcjcx', 'j', 'x', 'cqqixq', ],
['xj', 'j', 'qoc', 'xcqj', 'a', 'qq', 'bw', 'jxxc', 'jj', 'jjq', 'xxc', 'xqx', 'jcwj', 'xxv', 'qxj', 'cc', 'jxx', 'qcxq', 'cx', 'xxj', 'xqxq', 'qcqcq', 'q', 'jc', 'jc', 'qxjqq', 'qxqcxc', 'qjcjj', 'qaj', 'j', 'xq', 'jbx', 'bqij', 'jj', 'j', 'xcq', 'qc', 'jcj', 'qjj', 'cqcjq', 'qx', 'cc', 'c', 'jph', 'jj', 'qqj', 'xj', 'xqcbx', 'ccqc', 'qi', 'j', 'jcjxqcq', 'cj', 'wcq', 'xxcvxc', 'x', 'q', 'oq', 'qqq', 'q', 'xxc', 'jcxcqcj', 'xqqc', 'jx', 'qc', 'jx', 'cjxc', 'xc', 'cjwc', 'xvj', 'j', 'qcjojcq', 'cqjjcqpw', 'jc', 'qjjcx', 'cjx', 'x', 'x', 'qjxpxqv', 'qq', 'qcc', 'cj', 'xxq', 'qc', 'vjj', 'qxcxj', 'qa', 'cc', ],
['q', 'xjjq', 'j', 'xp', 'cjc', 'xiq', 'cpv', 'qq', 'qcxj', 'xxxxcx', 'q', 'j', 'jjc', 'jcxq', 'cqcq', 'qqcj', 'wj', 'jj', 'qcqjj', 'qocxjj', 'jx', 'xc', 'q', 'pq', 'j', 'xj', 'ixq', 'oq', 'cj', 'jxqcxx', 'jj', 'xjx', 'jci', 'jqqj', 'cjj', 'cccv', 'ccc', 'jxccxj', 'jx', 'cjx', 'oq', 'xqj', 'q', 'cx', 'viqq', 'cj', 'ccq', 'xq', 'ccqjj', 'xjqq', 'jqq', 'ccj', 'jc', 'jjxc', 'xh', 'j', 'xjjqj', 'jqcq', 'qx', 'cxq', 'xj', 'qx', 'xjcqjqx', 'qqq', 'j', 'c', 'jc', 'jj', 'axxcx', 'cqjjc', 'jcjcbxc', 'xwxjc', 'qjj', 'jjqc', 'cccc', 'ax', 'jc', 'aqjcc', 'qjxxcxc', 'c', 'cj', 'cc', 'qq', 'jxcx', 'qcq', 'jqj', ],
['jqxcqx', 'x', 'xqv', 'jcxj', 'qq', 'cq', 'qc', 'oxjqq', 'jq', 'xjjqj', 'ccxq', 'xb', 'x', 'c', 'x', 'bjjq', 'jxcqp', 'p', 'xcc', 'jjc', 'xj', 'w', 'cjwcq', 'q', 'qxjaj', 'hh', 'x', 'x', 'c', 'jqj', 'cjxc', 'xj', 'qxj', 'jqc', 'qqqq', 'qc', 'xqpjbcxx', 'cbqi', 'cx', 'cjj', 'c', 'cb', 'jjx', 'xcxqx', 'j', 'qxjj', 'jqq', 'cj', 'xccccxq', 'cqj', 'c', 'q', 'jv', 'jq', 'qx', 'xjx', 'cj', 'iwqqx', 'q', 'cq', 'q', 'jjxpj', 'vx', 'qvjq', 'q', 'qj', 'x', 'axj', 'oxx', 'jjcxq', 'q', 'j', 'xq', 'q', 'qxj', 'xxx', 'q', 'qjjjx', 'c', 'jqx', 'jjc', 'jq', 'cxj', 'x', ],
['vq', 'qxjxc', 'qjqij', 'c', 'jcq', 'cijxqqj', 'j', 'cjjq', 'j', 'xq', 'xjx', 'vqq', 'xx', 'xx', 'coqx', 'xxcc', 'bxqc', 'jxjjq', 'cqji', 'vw', 'qcac', 'jcw', 'qjpa', 'xojqqxq', 'vjjhx', 'c', 'qxq', 'xqqj', 'cjjx', 'cjccqc', 'xaj', 'x', 'xj', 'jq', 'cc', 'cjqxxqc', 'qqxc', 'qbxcjq', 'xcw', 'cjjqj', 'jx', 'qcwj', 'cccx', 'qc', 'xqjc', 'cqc', 'c', 'xc', 'cxqcqj', 'pcv', 'c', 'jj', 'aqqq', 'x', 'cjqxq', 'oqx', 'qxjcc', 'jq', 'qjq', 'cxv', 'jq', 'ccqxjcq', 'qx', 'xq', 'j', 'cijj', 'cx', 'q', 'jcxxch', 'jx', 'wq', 'cxqx', 'q', 'j', 'cjqp', 'qqcx', 'xa', 'jqcj', 'jcjjjx', 'qb', 'xqcc', 'qc', 'xj', 'cxx', 'cc', 'qjq', 'qx', 'xxqxc', 'xqxx', 'hx', 'jxqq', 'qbcj', 'xcxcqq', 'ox', ],
['xc', 'xjxa', 'c', 'jqxi', 'q', 'cbjc', 'j', 'j', 'jxjx', 'qxxcc', 'jqx', 'jq', 'cccx', 'c', 'c', 'qcj', 'jc', 'ccjj', 'qjjicc', 'j', 'qq', 'cx', 'cjx', 'q', 'qxx', 'qc', 'jj', 'xx', 'jj', 'qcc', 'cxb', 'ojcjqxch', 'p', 'qcxc', 'xxxxx', 'qbhp', 'cxqjxj', 'v', 'qx', 'j', 'j', 'qjqjj', 'xq', 'i', 'cxxj', 'q', 'xxc', 'cq', 'x', 'jj', 'xxj', 'jc', 'xcjj', 'qcj', 'qqj', 'cxcc', 'jxjac', 'jjq', 'cx', 'q', 'jxq', 'iqxj', 'jjc', 'jqpxj', 'jcjjqj', 'cxj', 'cjjc', 'j', 'qxxc', 'cx', 'c', 'c', 'qjcj', 'qx', 'cxjc', 'q', 'cjj', 'cj', 'jqj', 'c', 'qcj', 'jjjjj', 'jb', 'qi', 'xx', 'joj', 'jc', ],
['x', 'jxqjj', 'jci', 'q', 'xcj', 'b', 'qxq', 'qj', 'xj', 'xjqp', 'vc', 'xxc', 'x', 'cx', 'hpcxj', 'jc', 'xjxjoxq', 'xxax', 'ccxqj', 'cc', 'qc', 'o', 'jocxx', 'qq', 'xbcq', 'cj', 'qxqqc', 'jx', 'cc', 'jcvxx', 'co', 'pcx', 'jj', 'xx', 'j', 'cxj', 'jjc', 'xjxa', 'xbx', 'cqjx', 'xqj', 'jc', 'xx', 'xv', 'j', 'qbc', 'b', 'cc', 'qoqj', 'qc', 'qjc', 'cx', 'j', 'xjqp', 'qcx', 'cxjccx', 'qx', 'cq', 'xcx', 'xqjch', 'xqbqjc', 'ccix', 'jq', 'xxcq', 'xjcc', 'cxc', 'j', 'jxcqjx', 'jq', 'qqxc', 'jqq', 'coj', 'qxj', 'qxjaa', 'qxxb', 'qqv', 'x', 'jjjj', 'xcqc', 'pxjcj', 'ojj', 'jj', 'qqx', 'ojcbx', ],
['ocq', 'vqq', 'qxjq', 'qjqqqcx', 'jbj', 'bcc', 'j', 'jj', 'qx', 'xjq', 'qqjxj', 'qcj', 'x', 'jq', 'q', 'x', 'xjqxc', 'j', 'xq', 'qxjjc', 'jjcxq', 'c', 'qqjc', 'ccicqj', 'q', 'x', 'cjaq', 'xjj', 'xq', 'q', 'cq', 'bcc', 'xqjq', 'c', 'xqqx', 'q', 'jqcjqj', 'qqq', 'qqq', 'xc', 'xjq', 'jqxq', 'oq', 'j', 'qqc', 'cx', 'jq', 'c', 'qvxij', 'cj', 'cc', 'jqjqv', 'xcqccq', 'wqc', 'jqc', 'x', 'i', 'b', 'cqq', 'x', 'caijc', 'qjxc', 'j', 'xq', 'qxc', 'qj', 'x', 'qxx', 'xxiq', 'cqj', 'c', 'jq', 'bxvj', 'cxqj', 'xcjo', 'ojx', 'cjx', 'qcj', 'cp', 'qj', 'q', 'jqx', 'cqqq', ],
['xj', 'jx', 'xc', 'j', 'cc', 'xcic', 'jcc', 'cc', 'jc', 'c', 'xcaq', 'xxb', 'jx', 'cxqq', 'jqjqj', 'jqq', 'xqq', 'xcq', 'cqxc', 'qxcjq', 'cij', 'x', 'vxh', 'p', 'ajj', 'cqxx', 'i', 'xjjx', 'c', 'xjx', 'q', 'cxi', 'jjxx', 'jcq', 'jqccx', 'cxqbxx', 'xp', 'jj', 'w', 'jj', 'iqojaj', 'xqc', 'xccxb', 'x', 'oqx', 'xq', 'qxj', 'c', 'qj', 'jq', 'qqc', 'cjj', 'qj', 'cx', 'xj', 'pj', 'qbcjcx', 'jjxhqx', 'cjq', 'qij', 'j', 'qq', 'jwq', 'ccq', 'x', 'wcj', 'qjx', 'jjx', 'cj', 'xcqxx', 'jjx', 'j', 'xjqx', 'c', 'xp', 'cq', 'cccox', 'xq', 'pcqch', 'q', 'cj', 'c', 'h', 'wcb', 'xqxxpqxc', 'jq', 'x', ],
['qxjc', 'cjc', 'qjca', 'jcxqc', 'pqcqc', 'jwcj', 'xc', 'qqxxxcj', 'q', 'caqc', 'xxxq', 'qxjc', 'jxxq', 'cqqxq', 'jj', 'pxx', 'jxq', 'qjxq', 'q', 'xpx', 'c', 'jjq', 'xjj', 'jh', 'cw', 'cqj', 'x', 'qq', 'cjx', 'jqx', 'jx', 'hjc', 'cqccc', 'ccc', 'qch', 'jjhx', 'jiqcq', 'j', 'cpj', 'xcjjc', 'ccchj', 'xcjc', 'qqqx', 'qci', 'ccqc', 'qcxb', 'xqhw', 'qxqc', 'xc', 'xccq', 'q', 'x', 'cccq', 'xwcj', 'xcc', 'qjj', 'jcq', 'c', 'jcx', 'jjx', 'ccvc', 'cjcq', 'qqb', 'xq', 'vxc', 'xjxc', 'cqx', 'cc', 'qc', 'jjhxch', 'cxq', 'jqjcq', 'jjj', 'j', 'c', 'xjx', 'qqq', 'x', 'jx', 'jjj', 'jjcjj', 'x', 'qj', 'jcqqj', 'qqq', ],
]


dictionary = Dictionary(texts)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [6]:
# set up two topic models
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=500,random_state=2, num_topics=4)
badLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=100,random_state=9, num_topics=4)

In [7]:
print(goodLdaModel.print_topics(num_topics=4, num_words=6))

[(0, '0.028*"q" + 0.021*"j" + 0.019*"c" + 0.017*"x" + 0.009*"cq" + 0.009*"cj"'), (1, '0.025*"c" + 0.021*"j" + 0.020*"x" + 0.018*"q" + 0.010*"o" + 0.008*"p"'), (2, '0.024*"p" + 0.023*"w" + 0.020*"b" + 0.018*"v" + 0.018*"i" + 0.017*"a"'), (3, '0.025*"b" + 0.024*"i" + 0.022*"w" + 0.022*"o" + 0.017*"h" + 0.016*"a"')]


In [8]:
# using U_Mass Coherence
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')
badcm = CoherenceModel(model=badLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')

In [9]:

goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_uci')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_npmi')
print(goodcm.get_coherence())

-1.1101196354085279
-0.46259393278699756
0.4985407620559927
0.0033734920972894045


In [10]:

goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=10,random_state=2, num_topics=4)

In [11]:

goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=50,random_state=2, num_topics=4)
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_uci')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_npmi')
print(goodcm.get_coherence())

-1.1790510669800032
-0.7123232017769117
0.3982203857179961
-0.04159572259737095


In [12]:

goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=100,random_state=2, num_topics=4)
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_uci')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_npmi')
print(goodcm.get_coherence())

-1.1126365783949645
-0.40600523797773386
0.5137366140894803
0.005173060683548551


In [13]:
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=200,random_state=2, num_topics=4)
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_uci')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_npmi')
print(goodcm.get_coherence())

-1.0614920228877187
-0.22708157209250546
0.5377289420959748
0.01908119086990269


In [14]:
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=50,random_state=2, num_topics=4)
goodcm = CoherenceModel(model=goodLdaModel, corpus=corpus, dictionary=dictionary, coherence='u_mass')

print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_uci')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')
print(goodcm.get_coherence())
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_npmi')
print(goodcm.get_coherence())

-1.1790510669800032
-0.7123232017769117
0.3982203857179961
-0.04159572259737095


In [15]:
# Visualize topic models
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(badLdaModel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      30.839407        1       1  0.033700 -0.019299
2      27.009455        1       2 -0.077530 -0.020188
0      21.896849        1       3  0.107504  0.009344
1      20.254282        1       4 -0.063673  0.030143, topic_info=     Category        Freq Term       Total  loglift  logprob
term                                                        
12    Default  257.000000    b  257.000000  30.0000  30.0000
257   Default  245.000000    i  245.000000  29.0000  29.0000
234   Default  239.000000    p  239.000000  28.0000  28.0000
30    Default  241.000000    o  241.000000  27.0000  27.0000
135   Default  255.000000    w  255.000000  26.0000  26.0000
43    Default  216.000000    v  216.000000  25.0000  25.0000
0     Default  206.000000    a  206.000000  24.0000  24.0000
18    Default   56.000000   hh   56.000000  23.0000  23.0000
15    Default  188.000000    h  188.000000  22.0000  22.0000
787   Default   63.000000   wi   63.000000  21.0000  21.0000
1299  Default   62.000000   iw   62.000000  20.0000  20.0000
97    Default  158.000000    x  158.000000  19.0000  19.0000
395   Default   51.000000   pp   51.000000  18.0000  18.0000
443   Default  182.000000    q  182.000000  17.0000  17.0000
1379  Default   56.000000   pb   56.000000  16.0000  16.0000
1297  Default   52.000000   ip   52.000000  15.0000  15.0000
1245  Default   55.000000   bi   55.000000  14.0000  14.0000
50    Default   48.000000   vo   48.000000  13.0000  13.0000
110   Default   47.000000   hv   47.000000  12.0000  12.0000
1392  Default   49.000000   ww   49.000000  11.0000  11.0000
860   Default   52.000000   bb   52.000000  10.0000  10.0000
1248  Default   52.000000   bp   52.000000   9.0000   9.0000
119   Default   44.000000   oo   44.000000   8.0000   8.0000
1356  Default   47.000000   wp   47.000000   7.0000   7.0000
1267  Default   53.000000   pw   53.000000   6.0000   6.0000
1404  Default   49.000000   ii   49.000000   5.0000   5.0000
1273  Default   51.000000   wb   51.000000   4.0000   4.0000
53    Default   43.000000   vv   43.000000   3.0000   3.0000
4     Default   52.000000   ao   52.000000   2.0000   2.0000
164   Default   48.000000   va   48.000000   1.0000   1.0000
...       ...         ...  ...         ...      ...      ...
556    Topic4    3.198380  aaa    6.300952   0.9187  -6.9337
56     Topic4   14.996749   aa   39.537560   0.6274  -5.3885
110    Topic4   17.543900   hv   47.771790   0.5951  -5.2316
0      Topic4   60.327106    a  206.744675   0.3651  -3.9966
75     Topic4    4.564919  hvo    9.870358   0.8257  -6.5779
61     Topic4   16.260586   av   45.006107   0.5787  -5.3076
164    Topic4   16.983490   va   48.685783   0.5437  -5.2641
119    Topic4   15.553829   oo   44.389359   0.5481  -5.3520
43     Topic4   55.900433    v  216.554733   0.2425  -4.0728
15     Topic4   44.454357    h  188.517944   0.1521  -4.3019
14     Topic4   41.341740    c  173.073669   0.1650  -4.3745
22     Topic4   13.932703   ho   42.936897   0.4713  -5.4621
3985   Topic4   14.663433   cj   47.176208   0.4283  -5.4110
46     Topic4   15.229569   vh   51.665264   0.3753  -5.3731
38     Topic4   13.793277   ov   45.236881   0.4091  -5.4722
3167   Topic4   12.237382   qj   38.667736   0.4463  -5.5918
97     Topic4   33.631142    x  158.835571   0.0444  -4.5809
29     Topic4   35.919743    j  179.319427  -0.0111  -4.5150
4      Topic4   14.770509   ao   52.062828   0.3370  -5.4037
443    Topic4   34.630497    q  182.704254  -0.0663  -4.5516
1868   Topic4   12.854417   qx   47.475666   0.2903  -5.5426
4043   Topic4   13.006975   jj   49.117302   0.2681  -5.5308
16     Topic4   13.008336   ha   49.618336   0.2580  -5.5307
3988   Topic4   12.275375   cq   45.474621   0.2872  -5.5887
3094   Topic4    9.814730   qq   32.615997   0.3959  -5.8124
2164   Topic4   11.615567   xj   44.923141   0.2442  -5.6440
135    Topic4   26.887442    w  25